In [1]:
import os
os.getcwd()

'C:\\Users\\ArumPark\\Desktop\\fsimonjetz-korean_anaphora-57be778a8ba3\\src'

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [3]:
from helper.mmax_reader import *
from helper.etri_reader import *
from helper.DepTreeForEuroParl import ReadCorpusFast as ReadConll
from helper.misc import id2int, index_in_sent
from pprint import pprint
import pickle
import pandas as pd
from collections import Counter
from IPython.display import display
pd.options.display.max_columns = None
import os
from helper.transcriber import transcribe

parsed_sentences = [None] + list(read_corpus("C:\\Users\\ArumPark\\Desktop\\fsimonjetz-korean_anaphora-57be778a8ba3\\data\\etri_corpus\\one_sent_per_line.txt.dep_parse.out.mod_orig_add_na_2018.txt"))
# 원래 작업했던 txt 파일에 문제가 생겨서 ...2018년 원본 파일로 다시 돌려봄! 070822 이것만 작동, 이걸로 해야할 듯 

annotation_dir = "C:\\Users\\ArumPark\\Desktop\\fsimonjetz-korean_anaphora-57be778a8ba3\\data\\finished_annotation\\"
annotation_dir_test = "C:\\Users\\ArumPark\\Desktop\\fsimonjetz-korean_anaphora-57be778a8ba3\\data\\finished_annotation_test\\"
project_dir = "C:\\Users\\ArumPark\\Desktop\\fsimonjetz-korean_anaphora-57be778a8ba3\\data\\finished_annotation\\etri{}" # template for paths to project directories
subj_predicates = pickle.load(open("C:\\Users\\ArumPark\\Desktop\\fsimonjetz-korean_anaphora-57be778a8ba3\\data\\subj_pred_words.pkl", mode="rb"))


In [4]:
# include all projects
include_projects = [] # project numbers to include

for f in os.listdir(annotation_dir): #test로 check하기 # 다시 원래 dir로
    
    if f.startswith("."):
        continue
    
    include_projects.append(int(f[4:]))

In [5]:
# Define functions
import re
from collections import defaultdict

def get_parse_info(markable):

    #assert len(markable) == 1, "item '{}' is longer than one token".format(str(markable))
    
    #if len(markable) != 1:
        #print("strange tokenization at", markable.id)
    
    hostsent = markable.host("sentence")
    parse_tokens = parsed_sentences[int(hostsent.sentence_id)].tokens
    offset = id2int(hostsent.span[0])
    return [parse_tokens[id2int(i)-offset] for i in markable.span]

def sent_morphs(markable):

    hostsent = markable.host("sentence")
    parse_tokens = parsed_sentences[int(hostsent.sentence_id)].tokens
    for t in parse_tokens:
        sent_morphs = t.morphs
    
    return sent_morphs

def sent_dep(markable):

    hostsent = markable.host("sentence")
    parse_tokens = parsed_sentences[int(hostsent.sentence_id)].tokens
    for t in parse_tokens:
        sent_dep = t.dep
    
    return sent_dep
    
def get_base_morpheme(markable):
    
    assert len(markable) == 1, "item '{}' is longer than one token".format(str(markable))
    
    parse_info = get_parse_info(markable)[0]
    
    return parse_info.morphs[0]


def has_participant_switch(sentence):
    # return True if the previous sentence is by a different speaker
    
    if sentence.host("Utterance").span[0] == sentence.span[0]:
        return 1
    
    return 0
    

def re_morphmatch(morphlist, **kwargs):
    # test if there is any morph for which all kwarg constraints
    # hold; kwargs look like featurename=/regex/
    
    if not kwargs:
        return False
    
    for m in morphlist:
        
        matched = set()
        
        for feature, regex in kwargs.items():
            
            #print("testing if", regex, "is compatible with", feature, "whose value is",  m.get(feature, ""))
            is_match = re.search(regex, m.get(feature, ""))
            matched.add(bool(is_match))
            
        
        if all(matched):
            return 1
        
    return 0


def sentence_noun_map(project):
    # map {sentence_id:candidates}
    # return a dictionary that gives a list of noun candidates for a sentence id
    sent_nouns = defaultdict(set)
    
    for n in project.markables["Noun"]:
        sent_nouns[n.host("sentence").id].add(n)
        
    return sent_nouns

def get_candidates(sent, sent_map, n=3):
    # extract candidates from up to n sentences to the left
    
    candidates = list()
    
    for i in range(n+1):
        
        for cand in sent_map.get("markable_{}".format(id2int(sent.id)-i), []):
            # TO DO: make sure that the sent and the cand are from the same dialog
            if sent.host("Dialog").id == cand.host("Dialog").id:
                candidates.append((i,cand))
        
    return candidates    


def get_is_ana(zero):
    
    if zero.anaphora_type == "extra":
        return 0
    
    elif zero.anaphora_type in ["inter", "intra"]:
        return 1
    
    else:
        return -1
    
    
def is_head(cand_parse_info):
    
    if cand_parse_info.dep == "modi":
        # should cover -와/과, -(이)랑, -(이)나, -하고
        if re_morphmatch(cand_parse_info.morphs, pos="P", subpos="coor", morphpos="FJ"):
            return 1
        else:
            return 0
    
    else:
        return 1
    
def connective_true(morphs):
    if re_morphmatch(morphs, pos="E", subpos="oend", morphpos="EC"):
        return 1
    else:
        return 0    
    
def get_connective_pred(morphs):
    
    if re_morphmatch(morphs, morph="^(고|구|거나|고서|고나서|으?면서도?|으?며|으?려고|으?려|으?러|으?려면|느라고?|는바람에|기위해서?|[아어]서|서|어|아|라|어서그런지|다가|어?다)$", subpos="^(oend|cend)$"):
        return "class_A"  
    elif re_morphmatch(morphs, morph="^(길래|자|자마자)$"):
        return "class_B"
#     elif re_morphmatch(morphs, morph="^[았었]더니$"):
#         return "class_C"
    
    else:
        return "N/A"

def get_connective_pred_str(morphs):
    
    if re_morphmatch(morphs, morph="^(고|구|거나|고서|고나서|으?면서도?|으?며|으?려고|으?려|으?러|으?려면|느라고?|는바람에|기위해서?|[아어]서|서|어|아|라|어서그런지|다가|어?다)$", subpos="^(oend|cend)$"):
        return morphs[1].get('morph')
    else:
        return "N/A"

def get_referents(zero):
    
    #"1sh+(저)", "2sh+(당신)", "1sh-(나)", "2sh-(너)", "1ph+(저희)", "1ph-(우리)", "3s(그/그녀/그것)", 
    # "3p(그들/그녀들/그것들)", "2ph+(당신들)", "2ph-(너희들)", "others"
    if zero.anaphora_type == "extra":
        
        if zero.referent_type == "1sh+":
            return "1sh+"

        elif zero.referent_type == "2sh+":
            return "2sh+"

        elif zero.referent_type == "1sh-":
            return "1sh-"

        elif zero.referent_type == "2sh-":
            return "2sh-"

        elif zero.referent_type == "1ph+":
            return "1ph+"

        elif zero.referent_type == "1ph-":
            return "1ph-"

        elif zero.referent_type == "3s":
            return "3s"

        elif zero.referent_type == "3p":
            return "3p"

        elif zero.referent_type == "2ph+":
            return "2ph+"

        elif zero.referent_type == "2ph-":
            return "2ph-"

        elif zero.referent_type == "other":
            return zero.other_ref_comment

        else: 
            return "N/A"
        
#extra에 대한 것만 referent_type이 있어서 그런거 아닐까?
# extra를 조건으로 넣어보기 /  성공함
# 나중에 같은지 아닌지 조건 넣고
# 복문 규칙까지 넣어야 하는지 고민해 보기 (복문규칙은 이미 완성된 dataframe을 활용해야 하는 건가?)

    

In [6]:
# Actual data generation

datapoints = list()
maxn = len(include_projects)
print("{:.2f}%\r".format(0/maxn), end="")

for i,pnum in enumerate(include_projects, start=1): 
    
    filepath = os.path.join(project_dir.format(pnum), "zero_pronouns.mmax")
    project = MMAXReader(filepath, unique_hosts=["sentence", "Dialog", "Utterance"])
    cand_map = sentence_noun_map(project)
    
    for zero in project.markables["Zero"]: 

        zp_sentence = zero.host("sentence")
        
        parse_tokens = parsed_sentences[int(zp_sentence.sentence_id)].tokens
        ## ['이것을', '작성하아주시겠습니까', '?']
    
        sent_dep = list()
        sent_morphs = list()
        sent_head = list()
        
        for t in parse_tokens: 
            sent_dep.append(t.dep) 
            sent_morphs.append(t.morphs) 
            sent_head.append(t.head)
        
        candidates = get_candidates(zp_sentence, cand_map)
        antecedents = set(a.id for a in zero.resolve("antecedent"))
        antecedents_list = [a for a in zero.resolve("antecedent")]  
        antecedents_str = ''.join(str(s) for s in antecedents_list)
    
        
        zp_markable_ID = zero.id
        
        morphs = get_parse_info(zero)[0].morphs
        sent_morphs = get_parse_info(zp_sentence)[0].morphs
        
        base_morpheme = morphs[0]
        
        # base_morpheme_second = get_connective_pred_str(morphs) 
                
        zero_parse_info = get_parse_info(zero)[0]
        
        datapoint = dict()
            
        datapoint["etri_sent"] = zp_sentence.sentence_id
        datapoint["sent"] = zp_sentence
        datapoint["verb"] = str(zero)
        datapoint["sent_dep"] = sent_dep
        datapoint["sent_morphs"] = sent_morphs
        datapoint["connective"] = get_connective_pred(morphs)
        datapoint["base_morpheme"] = base_morpheme.get("morph", "N/A") # added
        datapoint["base_morpheme_second"] = get_connective_pred_str(morphs) # added
        datapoint["connective_true"] = connective_true(morphs)
        datapoint["verb_roman"] = transcribe(datapoint["verb"], "-")
        datapoint["zp_markable_ID"] = zp_markable_ID
        datapoint["zp_speaker"] = zero.host("Utterance").speaker
        datapoint["tense_zp"] = base_morpheme.get("tense", "N/A")
        datapoint["zp_mood"] = base_morpheme.get("mood", "N/A")
        datapoint["zp_hono"] = base_morpheme.get("hono", "N/A")
        datapoint["zero_type"] = zero.anaphora_type
        datapoint["verb_syn"] = zero_parse_info.dep
        datapoint["verb_head"] = zero_parse_info.head
        datapoint["zero_referents"] = get_referents(zero)
        datapoint["antecedent"] = antecedents_str
        datapoint["applied_rule"] = ""
        datapoint["pos"] = base_morpheme.get("pos", "N/A")
        # objective pred
#         datapoint["obj_pred"] = base_morpheme.get("morph", "") + "다" in subj_predicates["D"]
#         datapoint["obj_pred"] = int(datapoint["obj_pred"])

            
        if datapoint["zp_mood"] == "impe":
            datapoint["subject_by_rule"] = "2sh+"
            datapoint["group"] = "mood derivable"
            datapoint["rule"] = "impe"
            datapoint["rule_order"] = "1"
            datapoint["applied_rule"] += "impe was applied, "
        elif datapoint["zp_mood"] == "sugg":
            datapoint["subject_by_rule"] = "1ph-"
            datapoint["group"] = "mood derivable"
            datapoint["rule"] = "sugg"
            datapoint["rule_order"] = "2"
            datapoint["applied_rule"] += "sugg was applied, "
        elif re_morphmatch(morphs, morph="^(으?세요|으?세여|으?십시오|으?십시요)$", pos="^E$") and datapoint["zp_mood"] == "decl":
            datapoint["subject_by_rule"] = "2sh+"
            datapoint["group"] = "mood derivable"
            datapoint["rule"] = "impe"
            datapoint["rule_order"] = "3"
            datapoint["applied_rule"] += "impe(decl) was applied, "
#         elif re_morphmatch(morphs, morph="^(ㅂ시다|자)$", pos="^E$") and datapoint["zp_mood"] == "decl":
#             datapoint["subject_by_rule"] = "1ph-"
#             datapoint["group"] = "mood derivable"
#             datapoint["rule"] = "sugg"
#             datapoint["applied_rule"] += "sugg(decl) was applied, "
#             # nichts getroffen
        elif re_morphmatch(morphs, morph="^(는게좋|면좋|는게낫|는것이좋|는편이좋)$", pos="^X$") and datapoint["zp_hono"] == "plus":
            datapoint["subject_by_rule"] = "2sh+"
            datapoint["group"] = "suggestory formula"
            datapoint["rule"] = "suggestory periphrastic construction"
            datapoint["rule_order"] = "4"
            datapoint["applied_rule"] += "suggestory(plus) was applied, "
        elif re_morphmatch(morphs, morph="^(는게좋|면좋|는게낫|는것이좋|는편이좋)$", pos="^X$") and datapoint["zp_mood"] == "decl":
            datapoint["subject_by_rule"] = "2sh+"
            datapoint["group"] = "suggestory formula"
            datapoint["rule"] = "suggestory periphrastic construction"
            datapoint["rule_order"] = "5"
            datapoint["applied_rule"] += "suggestory was applied, "
            
        elif re_morphmatch(morphs, morph="^(을것같|ㄹ것같|는것같|ㄴ것같|은듯하)$", pos="^X$") and datapoint["zp_hono"] == "plus":
            datapoint["subject_by_rule"] = "2sh+"
            datapoint["group"] = "guess"
            datapoint["rule"] = "guess periphrastic construction"
            datapoint["rule_order"] = "6"
            datapoint["applied_rule"] += "guess(plus) was applied, "
            
#         elif datapoint["obj_pred"] = base_morpheme.get("morph", "") + "다" in subj_predicates["E"]:
#             datapoint["subject_by_rule"] = "none"
#             datapoint["group"] = "exclude"
#             datapoint["rule"] = "exclude predicate"
#             datapoint["applied_rule"] += "exclude_predicate1 was applied, "
            
        else: 
            datapoint["subject_by_rule"] = "N/A" 
            
        # 2,601 getroffen (seems ok. 11.94% of 17,266)
        
# clau related rules      

        try_str_list = ["보니까", "보니깐", "보니까는"]
        performative_head_list = ["부탁", "권유"]
        cognition_head_list = ["예정","생각","기억","확인","깜박","예상","착각","이해","확실","집중","계획","숙지","오해","파악","동의","사과","깜빡"]
        sensation_head_list = ["시음","민감","구경","시원"]
        feeling_head_list = ["실망","공포","영광","장담","한시름","미안","실례","의아","질색","놀랍","환영"]
        if datapoint["subject_by_rule"] == "N/A":
            if re_morphmatch(morphs, morph="^(아?야하|어야하|아?야되|어야되|고계시|으?시|으셨)$", pos="^(X|E)$") and re_morphmatch(morphs, morph="^([ㄴ는]데|는데|는건데|은데|[ㄴ은는]건데)$", pos="^E$") and datapoint["zp_hono"] == "plus" and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "2sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_neunde"
                datapoint["rule_order"] = "7"
                datapoint["applied_rule"] += "clau(plus_neunde) was applied, "
                # 진짜 적용됐는지 확인하기
            elif re_morphmatch(morphs, morph="^(으?시|아계시|고계시|[ㄹ을]줄알|가시|주무시|으셨)$", pos="^(E|V)$") and re_morphmatch(morphs, morph="^(려?면|으면|다보면)$", pos="^E$") and datapoint["verb_syn"] == "clau": 
                datapoint["subject_by_rule"] = "2sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_myeon"
                datapoint["rule_order"] = "8"
                datapoint["applied_rule"] += "clau(plus_myeon) was applied, "
                # 진짜 적용됐는지 확인하기
                #datapoint["rule_order"] = ""
            elif re_morphmatch(morphs, morph="^으?시$", pos="^E$") and re_morphmatch(morphs, morph="^었$", pos="^E$") and re_morphmatch(morphs, morph="^으면$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "2sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_myeon"
                datapoint["rule_order"] = "9"
                datapoint["applied_rule"] += "clau(plus_myeon) was applied, "
            elif re_morphmatch(morphs, morph="^주무시$", pos="^V$") and re_morphmatch(morphs, morph="^고$", pos="^E$") and datapoint["verb_syn"] == "clau": 
                datapoint["subject_by_rule"] = "2sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_go"
                datapoint["rule_order"] = "10"
                datapoint["applied_rule"] += "clau(plus_go) was applied, "
                # nichts getroffen
            elif re_morphmatch(morphs, morph="^(으?시|[아어]계시)$", pos="^E$") and re_morphmatch(morphs, morph="^다가?$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "2sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_daga"
                datapoint["rule_order"] = "11"
                datapoint["applied_rule"] += "clau(plus_daga) was applied, "
            elif re_morphmatch(morphs, morph="^으?시$", pos="^E$") and re_morphmatch(morphs, morph="^으?면서$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "2sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_myeonseo"
                datapoint["rule_order"] = "12"
                datapoint["applied_rule"] += "clau(plus_myeonseo) was applied, "
            elif re_morphmatch(morphs, morph="^으?시$", pos="^E$") and re_morphmatch(morphs, morph="^어서$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "2sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_eoseo"
                datapoint["rule_order"] = "13"
                datapoint["applied_rule"] += "clau(plus_eoseo) was applied, "
            elif re_morphmatch(morphs, morph="^으?시$", pos="^E$") and re_morphmatch(morphs, morph="^ㄴ후에*$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "2sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_nhu"
                datapoint["rule_order"] = "14"
                datapoint["applied_rule"] += "clau(plus_nhu) was applied, "
            elif re_morphmatch(morphs, morph="^으?시$", pos="^E$") and re_morphmatch(morphs, morph="^기전에$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "2sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_gijeon"
                datapoint["rule_order"] = "15"
                datapoint["applied_rule"] += "clau(plus_gijeon) was applied, "
            elif re_morphmatch(morphs, morph="^으?시$", pos="^E$") and re_morphmatch(morphs, morph="^(ㄴ다음에*|ㄴ뒤에*|ㄴ뒤에도)$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "2sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_ndaum"
                datapoint["rule_order"] = "16"
                datapoint["applied_rule"] += "clau(plus_ndaum) was applied, "
            elif re_morphmatch(morphs, morph="^으?시$", pos="^E$") and re_morphmatch(morphs, morph="^(ㄹ때|ㄹ때에?는|ㄹ때에?도)$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "2sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_lttae"
                datapoint["rule_order"] = "17"
                datapoint["applied_rule"] += "clau(plus_lttae) was applied, "
            elif re_morphmatch(morphs, morph="^으?시$", pos="^E$") and re_morphmatch(morphs, morph="^다니$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "2sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_dani"
                datapoint["rule_order"] = "18"
                datapoint["applied_rule"] += "clau(plus_dani) was applied, "
            elif re_morphmatch(morphs, morph="^으?시$", pos="^E$") and re_morphmatch(morphs, morph="^어야지?$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "2sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_eoya"
                datapoint["rule_order"] = "19"
                datapoint["applied_rule"] += "clau(plus_eoya) was applied, "
            elif re_morphmatch(morphs, morph="^으?시$", pos="^E$") and re_morphmatch(morphs, morph="^더라도?$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "2sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_deolado"
                datapoint["rule_order"] = "20"
                datapoint["applied_rule"] += "clau(plus_deolado) was applied, "
            elif re_morphmatch(morphs, morph="^으?시$", pos="^E$") and re_morphmatch(morphs, morph="^ㄴ?다면$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "2sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_ndamyeon"
                datapoint["rule_order"] = "21"
                datapoint["applied_rule"] += "clau(plus_ndamyeo) was applied, "
            elif re_morphmatch(morphs, morph="^(아?야하|어야하|으?려하|[ㄹ을]터이|을테|[아어]보|[ㄹ을]것이)$", pos="^X$") and re_morphmatch(morphs, morph="^(니|니까|니깐)$", pos="^E$") and datapoint["verb_syn"] == "clau" and datapoint["zp_hono"] == "plus":
                datapoint["subject_by_rule"] = "2sh+" # 1sh+ 였던걸 2sh+로 바꿈 270723
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_nikka"
                datapoint["rule_order"] = "22"
                datapoint["applied_rule"] += "clau(plus_nikka) was applied, "
                # nichts getroffen
            elif datapoint["zp_hono"] == "plus" and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "2sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_plus"
                datapoint["rule_order"] = "23"
                datapoint["applied_rule"] += "clau(plus, at the end) was applied, "
                
            elif re_morphmatch(morphs, morph="^(ㄹ수있|을수있|ㄹ수없|ㄹ수도없|을수없)$", pos="^X$") and re_morphmatch(morphs, morph="^으?니$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "2sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_euni"
                datapoint["rule_order"] = "24"
                datapoint["applied_rule"] += "clau(euni_possibility) was applied, "
                # coverage and precision low
#             elif re_morphmatch(morphs, morph="^(걸리|멀쩡하|멀|나오)$", pos="^(J|V)$") and re_morphmatch(morphs, morph="^(는데)$", pos="^E$"):
#                 datapoint["subject_by_rule"] = "none"
#                 datapoint["group"] = "exclude"
#                 datapoint["rule"] = "exclude predicate"
#                 datapoint["rule_order"] = "25"
#                 datapoint["applied_rule"] += "exclude before clau(neunde1) was applied,"
            elif re_morphmatch(morphs, morph="^(겠|았|었|아?야하|어야하|아?야되|어야되|ㄹ려고?하|으?려고?하|고자하|ㄹ까하|[아어]?드리|고싶|[ㄹ을]생각이|[ㄹ을]지도모르)$", pos="^(E|X)$") and re_morphmatch(morphs, morph="^([ㄴ는]데|는데|는건데|은데|[ㄴ은는]건데)$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "1sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_neunde"
                datapoint["rule_order"] = "26"
                datapoint["applied_rule"] += "clau(neunde1) was applied, "
                # 진짜 적용됐는지 확인하기
            elif re_morphmatch(morphs, morph="^([ㄹ을]건데|려는데)$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "1sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_neunde"
                datapoint["rule_order"] = "27"
                datapoint["applied_rule"] += "clau(neunde2) was applied, "
            elif re_morphmatch(morphs, morph="^(고있|어서그러|던중이|고있는중이|ㄴ것같|은것같|은듯하)$", pos="^X$") and re_morphmatch(morphs, morph="^(ㄴ데|는데|은데)$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "1sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_neunde"
                datapoint["rule_order"] = "28"
                datapoint["applied_rule"] += "clau(neunde3) was applied, "
            elif re_morphmatch(morphs, morph="^(았었|았|었|었었)$", pos="^E$") and re_morphmatch(morphs, morph="^는데$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "1sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_neunde"
                datapoint["rule_order"] = "29"
                datapoint["applied_rule"] += "clau(neunde4) was applied, "
            elif re_morphmatch(morphs, morph="^(은건데|는건데)$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "1sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_neunde"
                datapoint["rule_order"] = "30"
                datapoint["applied_rule"] += "clau(neunde5) was applied, "
            elif re_morphmatch(morphs, morph="^고싶$", pos="^X$") and re_morphmatch(morphs, morph="^지만$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "1sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_jiman"
                datapoint["rule_order"] = "31"
                datapoint["applied_rule"] += "clau(jiman1) was applied, "
                # coverage low
            elif re_morphmatch(morphs, morph="^.*보$", pos="^V$") and re_morphmatch(morphs, morph="^았$", pos="^E$") and re_morphmatch(morphs, morph="^지만$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "1sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_jiman"
                datapoint["rule_order"] = "32"
                datapoint["applied_rule"] += "clau(jiman2) was applied, "
                # coverage low
            elif re_morphmatch(morphs, morph="^고싶$", pos="^X$") and re_morphmatch(morphs, morph="^으니까?$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "1sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_nikka"
                datapoint["rule_order"] = "33"
                datapoint["applied_rule"] += "clau(nikka1) was applied, "
                # eins getroffen
            elif re_morphmatch(morphs, morph="^려니?$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "1sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_nikka"
                datapoint["rule_order"] = "34"
                datapoint["applied_rule"] += "clau(nikka2) was applied, "
                # eins getroffen
            elif re_morphmatch(morphs, morph="^(아?야하|어야하|으?려하|[ㄹ을]터이|을테|[아어]보|[ㄹ을]것이)$", pos="^X$") and re_morphmatch(morphs, morph="^(니|니까|니깐)$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "1sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_nikka"
                datapoint["rule_order"] = "35"
                datapoint["applied_rule"] += "clau(nikka3) was applied, "
                
            elif re_morphmatch(morphs, morph="^(아드리|어드리|드리|축하드리)$", pos="^(X|V)$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "1sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_various"
                datapoint["rule_order"] = "36"
                datapoint["applied_rule"] += "clau(various_deuli) was applied, "                
           
            elif re_morphmatch(morphs, morph="^지말$", pos="^X$") and re_morphmatch(morphs, morph="^고$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "2sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_go"
                datapoint["rule_order"] = "37"
                datapoint["applied_rule"] += "clau(go_variant) was applied, "
            elif re_morphmatch(morphs, morph="^(ㄹ까보|게되|고싶)$", pos="^X$") and re_morphmatch(morphs, morph="^(아|어|어서|아서)$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "1sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_aseo"
                datapoint["rule_order"] = "38"
                datapoint["applied_rule"] += "clau(aseo_variant) was applied, "
            elif re_morphmatch(morphs, morph="^(게되|야하)$", pos="^X$") and re_morphmatch(morphs, morph="^(다니|ㄴ다니)$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "1sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_dani"
                datapoint["rule_order"] = "39"
                datapoint["applied_rule"] += "clau(dani_variant) was applied, "
            elif re_morphmatch(morphs, morph="^려고$", pos="^E$") and re_morphmatch(morphs, morph="^보$", pos="^V$") and re_morphmatch(morphs, morph="^(니|니까)$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "1sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_euni"
                datapoint["rule_order"] = "40"
                datapoint["applied_rule"] += "clau(euni_variant1) was applied, "
            elif [x for x in try_str_list if x in datapoint["verb"]] and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "1sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_euni"
                datapoint["rule_order"] = "41"
                datapoint["applied_rule"] += "clau(euni_variant2) was applied, "
            elif re_morphmatch(morphs, morph="^고있$", pos="^X$") and re_morphmatch(morphs, morph="^(으니|으니까|으니깐)$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "1sh+"
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_euni"
                datapoint["rule_order"] = "42"
                datapoint["applied_rule"] += "clau(euni_variant3) was applied, "
                
            elif re_morphmatch(morphs, morph="^(는줄알|ㄴ줄알|을줄알|ㄹ줄알|ㄹ줄아|잘알|알|아|알게되|잘아|모르|는줄모르|을줄모르|ㄹ줄모르|어야할지모르|는지모르|ㄴ지모르|을지모르|ㄹ지모르|을수있을지모르|알아들|궁금하|고집하)$", pos="^(X|V|J)$") and re_morphmatch(morphs, morph="^(ㄴ데|는데|은데|지만|다가|니까|으니까|니깐|으니깐)$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "1sh+" 
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_cognition"
                datapoint["rule_order"] = "43"
                datapoint["applied_rule"] += "0.1 clau(cognition1) was applied, "
            elif re_morphmatch(morphs, morph="^(잊|잊어버리|믿|알아듣|외우|헷갈리)$", pos="^V$") and re_morphmatch(morphs, morph="^(ㄴ데|는데|은데|지만|다가|니까|으니까|니깐|으니깐)$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "1sh+" 
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_cognition"
                datapoint["rule_order"] = "44"
                datapoint["applied_rule"] += "0.2 clau(cognition2) was applied, "
                # nichts getroffen
            elif [x for x in cognition_head_list if x in datapoint["verb"]] and re_morphmatch(morphs, morph="^(ㄴ데|는데|은데|지만|다가|니까|으니까|니깐|으니깐)$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "1sh+" 
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_cognition"
                datapoint["rule_order"] = "45"
                datapoint["applied_rule"] += "0.3 clau(cognition3) was applied, "
            elif re_morphmatch(morphs, morph="^(보|못보|살펴보|둘러보|보러오|듣|들어서|들|들어보|맛보|배고프|싱거우|간지럽)$", pos="^(V|J)$") and re_morphmatch(morphs, morph="^(ㄴ데|는데|은데|지만|다가|니까|으니까|니깐|으니깐)$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "1sh+" 
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_cognition"
                datapoint["rule_order"] = "46"
                datapoint["applied_rule"] += "0.4 clau(cognition4) was applied, "
            elif [x for x in sensation_head_list if x in datapoint["verb"]] and re_morphmatch(morphs, morph="^(ㄴ데|는데|은데|지만|다가|니까|으니까|니깐|으니깐)$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "1sh+" 
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_cognition"
                datapoint["rule_order"] = "47"
                datapoint["applied_rule"] += "0.5 clau(cognition5) was applied, "
                # nichts getroffen
            elif re_morphmatch(morphs, morph="^(춥|따뜻하|덥|후텁지근하|찝찝하|아프|쓰라리|만지|만져보|느끼)$", pos="^(J|V)$") and re_morphmatch(morphs, morph="^(ㄴ데|는데|은데|지만|다가|니까|으니까|니깐|으니깐)$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "1sh+" 
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_cognition"
                datapoint["rule_order"] = "48"
                datapoint["applied_rule"] += "0.6 clau(cognition6) was applied, "
            elif re_morphmatch(morphs, morph="^(괜찮|고맙|불편하|피곤하|지루하|흥미롭|곤란하|불쾌하|거북하|불안하|힘들|힘드|기힘들|좋|어쩔수없|별수없)$", pos="^(J|X)$") and re_morphmatch(morphs, morph="^(ㄴ데|는데|은데|지만|다가|니까|으니까|니깐|으니깐)$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "1sh+" 
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_cognition"
                datapoint["rule_order"] = "49"
                datapoint["applied_rule"] += "0.7 clau(cognition7) was applied, "
            elif re_morphmatch(morphs, morph="^(좋아하|부담되|기대되|마음에들|기대하|걱정되|걱정하|고민되|고민하|놀라|당황하|놀래|놀랍|행복하|미안하|죄송하|죄송|답답하|부담스럽|의아하|질색이)$", pos="^(V|N)$") and re_morphmatch(morphs, morph="^(ㄴ데|는데|은데|지만|다가|니까|으니까|니깐|으니깐)$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "1sh+" 
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_cognition"
                datapoint["rule_order"] = "50"
                datapoint["applied_rule"] += "0.8 clau(cognition8) was applied, "
            elif re_morphmatch(morphs, morph="^(후회하|감사하|감사드리|감탄하|횡재하|들뜨|떨리|만족하|만족스럽|망설여지|망설이|싫|싫어하)$", pos="^(V|J)$") and re_morphmatch(morphs, morph="^(ㄴ데|는데|은데|지만|다가|니까|으니까|니깐|으니깐)$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "1sh+" 
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_cognition"
                datapoint["rule_order"] = "51"
                datapoint["applied_rule"] += "0.9 clau(cognition9) was applied, "
                # nichts getroffen
            elif [x for x in feeling_head_list if x in datapoint["verb"]] and re_morphmatch(morphs, morph="^(ㄴ데|는데|은데|지만|다가|니까|으니까|니깐|으니깐)$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "1sh+" 
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_cognition"
                datapoint["rule_order"] = "52"
                datapoint["applied_rule"] += "0.10 clau(cognition10) was applied, "
            elif re_morphmatch(morphs, morph="^(익숙하|자부하|망설여지|망설이|아쉽|상관없|즐겁|재미있|재미없|즐거우|무섭|기쁘|행복하|난감하|반갑|환영하|신기하|당황스럽|막막하|안타깝|부럽)$", pos="^(J|V)$") and re_morphmatch(morphs, morph="^(ㄴ데|는데|은데|지만|다가|니까|으니까|니깐|으니깐)$", pos="^E$") and datapoint["verb_syn"] == "clau":
                datapoint["subject_by_rule"] = "1sh+" 
                datapoint["group"] = "clau"
                datapoint["rule"] = "clau_cognition"
                datapoint["rule_order"] = "53"
                datapoint["applied_rule"] += "0.11 clau(cognition11) was applied, "
                
#             elif re_morphmatch(morphs, morph="^으?니$", pos="^E$") and datapoint["verb_syn"] == "clau":
#                 datapoint["subject_by_rule"] = "1sh+"
#                 datapoint["group"] = "clau"
#                 datapoint["rule"] = "clau_euni"
#                 datapoint["applied_rule"] += "clau(euni) was applied, "

            else: 
                datapoint["subject_by_rule"] = "N/A" 
                
                # 2,601 + 1,204 = 3,805 (22.1% of 17,266)
            
            indirect_str_list = ["주시겠","주실수","주시면안","시면안","주겠","줄수","주면안"]
            want_str_list = ["시겠습", "시겠어", "시려고"]
#             want_suffix_list = ["을게요","ㄹ게요","게요","을께요","ㄹ께요","ㄹ게","ㄹ께","ㄹ건데요","ㄹ건가","ㄹ거라","을건데요","는건가요","은건가요","ㄹ겁니까"]

            if datapoint["subject_by_rule"] == "N/A":
                if re_morphmatch(morphs, morph="^ㄴ적이?있|ㄴ적이?없$", pos="^X$") and datapoint["zp_hono"] == "plus":
                    datapoint["subject_by_rule"] = "2sh+"
                    datapoint["group"] = "experience"
                    datapoint["rule"] = "experience periphrastic construction"
                    datapoint["rule_order"] = "54"
                    datapoint["applied_rule"] += "1.1 experience(plus) was applied, "
                elif [x for x in indirect_str_list if x in datapoint["verb"]] and datapoint["zp_mood"] == "ques":
                    datapoint["subject_by_rule"] = "2sh+"
                    datapoint["group"] = "indirect speech act"
                    datapoint["rule"] = "request"
                    datapoint["rule_order"] = "55"
                    datapoint["applied_rule"] += "1.2 indirect(ques) was applied, "
                    # coverage and precision very high
                elif [x for x in want_str_list if x in datapoint["verb"]] and datapoint["zp_mood"] == "ques":
                    datapoint["subject_by_rule"] = "2sh+"
                    datapoint["group"] = "want statement"
                    datapoint["rule"] = "want verbal suffix"
                    datapoint["rule_order"] = "56"
                    datapoint["applied_rule"] += "1.3. want(ques_plus) was applied, "
                    # coverage and precision very high
#                 elif re_morphmatch(morphs, morph="^(는게좋|는편이좋|지않)$", pos="^X$") and re_morphmatch(morphs, morph="^(을것이|을듯싶|는게좋|겠)$", pos="^X$") and datapoint["zp_mood"] == "decl":
#                     datapoint["subject_by_rule"] = "2sh+"
#                     datapoint["group"] = "suggestory formula"
#                     datapoint["rule"] = "suggestory periphrastic construction"
#                     datapoint["applied_rule"] += "5. suggestory(decl) was applied, "
                # 5는 나중에 적용, 정확도가 높지가 않음
                elif re_morphmatch(morphs, morph="^으?면되$", pos="^X$") and re_morphmatch(morphs, morph="^(겠|지않)$", pos="^(E|X)$") and datapoint["zp_hono"] == "plus":
                    datapoint["subject_by_rule"] = "2sh+"
                    datapoint["group"] = "suggestory formula"
                    datapoint["rule"] = "suggestory periphrastic construction"
                    datapoint["rule_order"] = "57"
                    datapoint["applied_rule"] += "1.4 suggestory(plus) was applied, "
                elif re_morphmatch(morphs, morph="^(아?야하|[어여]야하)$", pos="^X$") and re_morphmatch(morphs, morph="^았$", pos="^E$") and datapoint["zp_hono"] == "plus":
                    datapoint["subject_by_rule"] = "2sh+"
                    datapoint["group"] = "suggestory formula"
                    datapoint["rule"] = "suggestory periphrastic construction"
                    datapoint["rule_order"] = "58"
                    datapoint["applied_rule"] += "1.5 suggestory(plus) was applied, "
                    # coverage 진짜 낮음 (어야하 plus 관련된 obligation 다 위로 올라와도 됨)
                elif re_morphmatch(morphs, morph="^(았|었)$", pos="^E$") and re_morphmatch(morphs, morph="^어야지요?$", pos="^E$") and datapoint["zp_hono"] == "plus":
                    datapoint["subject_by_rule"] = "2sh+"
                    datapoint["group"] = "suggestory formula"
                    datapoint["rule"] = "suggestory periphrastic construction"
                    datapoint["rule_order"] = "59"
                    datapoint["applied_rule"] += "1.6 suggestory(plus) was applied, "
                    # coverage 진짜 낮음
                elif re_morphmatch(morphs, morph="^(고[싶지?않?])$", pos="^X$") and datapoint["zp_hono"] == "plus": 
                    datapoint["subject_by_rule"] = "2sh+"
                    datapoint["group"] = "want statement"
                    datapoint["rule"] = "want periphrastic construction/plus"
                    datapoint["rule_order"] = "60"
                    datapoint["applied_rule"] += "1.7 want(plus) was applied, "
                elif re_morphmatch(morphs, morph="^[ㄹ을]건가요?$", pos="^E$") and datapoint["zp_hono"] == "plus": 
                    datapoint["subject_by_rule"] = "2sh+"
                    datapoint["group"] = "want statement"
                    datapoint["rule"] = "want periphrastic construction/plus"
                    datapoint["rule_order"] = "61"
                    datapoint["applied_rule"] += "1.8 want(plus) was applied, "
                    # coverage 진짜 낮음
                elif re_morphmatch(morphs, morph="^[ㄹ을]것이$", pos="^X$") and re_morphmatch(morphs, morph="^ㄴ가요$", pos="^E$") and datapoint["zp_hono"] == "plus": 
                    datapoint["subject_by_rule"] = "2sh+"
                    datapoint["group"] = "want statement"
                    datapoint["rule"] = "want periphrastic construction/plus"
                    datapoint["rule_order"] = "62"
                    datapoint["applied_rule"] += "1.9 want(plus) was applied, "
                    # coverage and precision high
#                 elif re_morphmatch(morphs, morph="^(을게요|ㄹ게요|게요|을께요|ㄹ께요|ㄹ게|ㄹ께|ㄹ건데요|ㄹ건가|ㄹ거라|을건데요|는건가요|은건가요|ㄹ겁니까)$", pos="^E$") and datapoint["zp_hono"] == "plus":
#                     datapoint["subject_by_rule"] = "2sh+"
#                     datapoint["group"] = "want statement"
#                     datapoint["rule"] = "want verbal suffix/plus"
#                     datapoint["applied_rule"] += "10. want(plus) was applied, "
                # 말씀하시는건가요 이런거 섞여있음 나중에 다시 적용 
                #그럼 이걸로 하실겁니까 ? 얼마 충전하실겁니까 ? 혹시 체크카드로 현금인출을하려고하시는건가요 ? (이런거 cover하고 싶었던 듯)
#                 elif re_morphmatch(morphs, morph="^ㄹ것이$", pos="^X$") and datapoint["zp_hono"] == "plus":
#                     datapoint["subject_by_rule"] = "2sh+"
#                     datapoint["group"] = "want statement"
#                     datapoint["rule"] = "want verbal suffix/plus"
#                     datapoint["applied_rule"] += "11. want(plus) was applied, "
                # 이것저것 다 섞여있음 나중에 다시 적용
                    
                else: 
                    datapoint["subject_by_rule"] = "N/A" 
                
                if datapoint["subject_by_rule"] == "N/A": 
                    if re_morphmatch(morphs, morph="^겠$", pos="^E$") and re_morphmatch(morphs, morph="^습니다$", pos="^E$") and datapoint["zp_hono"] == "plus":
                        datapoint["subject_by_rule"] = "2sh+"
                        datapoint["group"] = "want statement"
                        datapoint["rule"] = "want verbal suffix"
                        datapoint["rule_order"] = "63"
                        datapoint["applied_rule"] += "2.1 want(plus) was applied, "
                    elif re_morphmatch(morphs, morph="^(겠|겟)$", pos="^E$") and re_morphmatch(morphs, morph="^습니다$", pos="^E$") and datapoint["zp_mood"] == "decl":
                        datapoint["subject_by_rule"] = "1sh+"
                        datapoint["group"] = "want statement"
                        datapoint["rule"] = "want verbal suffix"
                        datapoint["rule_order"] = "64"
                        datapoint["applied_rule"] += "2.2 want was applied, "
                        # coverage and precision very high
                        # -드리겠-도 이미 포함
                        
                    elif re_morphmatch(morphs, morph="^(려고하|으려고하|사려하|려|려하|으려하|으려고|려고|어보려고하|려고그러|려고요|으려고요|ㄹ려고요|려는것이|고자하|ㄹ려고하|기로하|ㄹ려구요|려구요|사려하)$", pos="^(X|E|V)$") and datapoint["zp_hono"] == "plus":
                        datapoint["subject_by_rule"] = "2sh+"
                        datapoint["group"] = "want statement"
                        datapoint["rule"] = "want periphrastic construction"
                        datapoint["rule_order"] = "65"
                        datapoint["applied_rule"] += "2.2.1 want(plus) was applied, "
                    elif re_morphmatch(morphs, morph="^(려고하|으려고하|사려하|려|려하|으려하|으려고|려고|어보려고하|려고그러|려고요|으려고요|ㄹ려고요|려는것이|고자하|ㄹ려고하|기로하|ㄹ려구요|려구요|사려하)$", pos="^(X|E|V)$") and datapoint["zp_mood"] == "decl":
                        datapoint["subject_by_rule"] = "1sh+"
                        datapoint["group"] = "want statement"
                        datapoint["rule"] = "want periphrastic construction"
                        datapoint["rule_order"] = "66"
                        datapoint["applied_rule"] += "2.2.2 want was applied, "
                        # coverage and precision high
                    elif re_morphmatch(morphs, morph="^(려고하|으려고하|사려하|려|려하|으려하|으려고|려고|어보려고하|려고그러|려고요|으려고요|ㄹ려고요|려는것이|고자하|ㄹ려고하|기로하|ㄹ려구요|려구요|사려하)$", pos="^(X|E|V)$") and datapoint["zp_mood"] == "ques":
                        datapoint["subject_by_rule"] = "2sh+" 
                        datapoint["group"] = "want statement"
                        datapoint["rule"] = "want periphrastic construction"
                        datapoint["rule_order"] = "67"
                        datapoint["applied_rule"] += "2.2.3 want was applied, "
                        
                    elif re_morphmatch(morphs, morph="^[ㄹ을으]려고요?$", pos="^E$") and datapoint["zp_mood"] == "decl":
                        datapoint["subject_by_rule"] = "1sh+"
                        datapoint["group"] = "want statement"
                        datapoint["rule"] = "want verbal suffix"
                        datapoint["rule_order"] = "68"
                        datapoint["applied_rule"] += "2.3 want was applied, "
                        # coverage not that high
                    elif re_morphmatch(morphs, morph="^아가$", pos="^X$") and re_morphmatch(morphs, morph="^되$", pos="^V$") and datapoint["zp_mood"] == "ques":
                        datapoint["subject_by_rule"] = "1sh+"
                        datapoint["group"] = "permission"
                        datapoint["rule"] = "permission periphrastic construction"
                        datapoint["rule_order"] = "69"
                        datapoint["applied_rule"] += "2.4 permission was applied, "
                        # coverage really low
                    elif re_morphmatch(morphs, morph="^(아보|어보|보|가보|해보|써보|타보|찾아보|재보|맛보)$", pos="^(X|V)$") and re_morphmatch(morphs, morph="^(아도되|아도돼|ㄹ수있|면안되)$", pos="^X$") and datapoint["zp_mood"] == "ques":
                        datapoint["subject_by_rule"] = "1sh+"
                        datapoint["group"] = "permission"
                        datapoint["rule"] = "permission periphrastic construction"
                        datapoint["rule_order"] = "70"
                        datapoint["applied_rule"] += "2.5 permission was applied, "
                        # coverage and precision high
                    elif re_morphmatch(morphs, morph="^(아도되|어도되|도되)$", pos="^X$") and datapoint["zp_mood"] == "ques":
                        datapoint["subject_by_rule"] = "1sh+"
                        datapoint["group"] = "permission"
                        datapoint["rule"] = "permission periphrastic construction"
                        datapoint["rule_order"] = "71"
                        datapoint["applied_rule"] += "2.6 permission was applied, "
                        # coverage and precision high
                    elif re_morphmatch(morphs, morph="^(면되|으면되)$", pos="^X$") and re_morphmatch(morphs, morph="^겠$", pos="^E$") and datapoint["zp_mood"] == "decl":
                        datapoint["subject_by_rule"] = "1sh+"
                        datapoint["group"] = "want statement"
                        datapoint["rule"] = "want periphrastic construction"
                        datapoint["rule_order"] = "72"
                        datapoint["applied_rule"] += "2.7 want was applied, "
                    elif re_morphmatch(morphs, morph="^(았|었)$", pos="^E$") and re_morphmatch(morphs, morph="^어야지요?$", pos="^E$") and datapoint["zp_mood"] == "decl":
                        datapoint["subject_by_rule"] = "2sh+"
                        datapoint["group"] = "want statement"
                        datapoint["rule"] = "want periphrastic construction"
                        datapoint["rule_order"] = "73"
                        datapoint["applied_rule"] += "2.8 want was applied, "
                         # nichs getroffen 원래 의도는 20884 곤충은 검역대상인데 신고를 하셨어야지요.

                    elif re_morphmatch(morphs, morph="^(아야하|어야하|여야하|야하)$", pos="^X$") and datapoint["zp_hono"] == "plus":
                        datapoint["subject_by_rule"] = "2sh+"
                        datapoint["group"] = "obligation statement"
                        datapoint["rule"] = "obligation periphrastic construction"
                        datapoint["rule_order"] = "74"
                        datapoint["applied_rule"] += "2.9 obligation(plus) was applied, "
                        # covearge and precision high
                    elif re_morphmatch(morphs, morph="^(아야하|어야하|여야하|야하)$", pos="^X$") and re_morphmatch(morphs, morph="^ㄹ것같$", pos="^X$") and datapoint["zp_mood"] == "decl":
                        datapoint["subject_by_rule"] = "1sh+"
                        datapoint["group"] = "want statement"
                        datapoint["rule"] = "want periphrastic construction"
                        datapoint["rule_order"] = "75"
                        datapoint["applied_rule"] += "2.10 want was applied, "
                        # coverage not that high, one wrong case (ambiguous even for human)
                    elif re_morphmatch(morphs, morph="^(아야하|어야하|여야하|야하)$", pos="^X$") and datapoint["zp_mood"] == "ques":
                        datapoint["subject_by_rule"] = "1sh+"
                        datapoint["group"] = "obligation statement"
                        datapoint["rule"] = "obligation periphrastic construction"
                        datapoint["rule_order"] = "76"
                        datapoint["applied_rule"] += "2.11 obligation was applied, "
                        # covearge and precision high
                        
                    elif re_morphmatch(morphs, morph="^(고싶|고싶지않)$", pos="^X$") and datapoint["zp_mood"] == "ques": 
                        datapoint["subject_by_rule"] = "2sh+"
                        datapoint["group"] = "want statement"
                        datapoint["rule"] = "want periphrastic construction"
                        datapoint["rule_order"] = "77"
                        datapoint["applied_rule"] += "2.12 want was applied, "
                        # coverage very low, two cases
                    elif re_morphmatch(morphs, morph="^을건가요$", pos="^E$") and datapoint["zp_mood"] == "ques": 
                        datapoint["subject_by_rule"] = "2sh+"
                        datapoint["group"] = "want statement"
                        datapoint["rule"] = "want periphrastic construction"
                        datapoint["rule_order"] = "78"
                        datapoint["applied_rule"] += "2.13 want was applied, "
                        # coverage not that high
                    elif re_morphmatch(morphs, morph="^[ㄹ을]것이$", pos="^X$") and re_morphmatch(morphs, morph="^ㄴ가요$", pos="^E$") and datapoint["zp_mood"] == "ques": 
                        datapoint["subject_by_rule"] = "2sh+"
                        datapoint["group"] = "want statement"
                        datapoint["rule"] = "want periphrastic construction"
                        datapoint["rule_order"] = "79"
                        datapoint["applied_rule"] += "2.14 want was applied, "
                        # coverage not that high
                    elif re_morphmatch(morphs, morph="^(을게요|ㄹ게요|게요|을께요|ㄹ께요|ㄹ게|ㄹ께|ㄹ건데요|ㄹ건가|ㄹ거라|을건데요|는건데요|은건데요)?$", pos="^E$") and datapoint["zp_mood"] == "decl":
                        datapoint["subject_by_rule"] = "1sh+"
                        datapoint["group"] = "want statement"
                        datapoint["rule"] = "want verbal suffix"
                        datapoint["rule_order"] = "80"
                        datapoint["applied_rule"] += "2.15 want was applied, "
                        # coverage and precision high
                    elif re_morphmatch(morphs, morph="^(을래요|ㄹ래요)$", pos="^E$") and datapoint["zp_mood"] == "decl":
                        datapoint["subject_by_rule"] = "1sh+"
                        datapoint["group"] = "want statement"
                        datapoint["rule"] = "want verbal suffix"
                        datapoint["rule_order"] = "81"
                        datapoint["applied_rule"] += "2.16 want was applied, "
                        # coverage not that high
                    elif re_morphmatch(morphs, morph="^(을래요|ㄹ래요|ㄹ겁니까)$", pos="^E$") and datapoint["zp_mood"] == "ques":
                        datapoint["subject_by_rule"] = "2sh+"
                        datapoint["group"] = "want statement"
                        datapoint["rule"] = "want verbal suffix"
                        datapoint["rule_order"] = "82"
                        datapoint["applied_rule"] += "2.17 want was applied, "
                        # coverage and precision high
                    elif re_morphmatch(morphs, morph="^ㄹ것이$", pos="^X$") and datapoint["zp_mood"] == "ques":
                        datapoint["subject_by_rule"] = "2sh+"
                        datapoint["group"] = "want statement"
                        datapoint["rule"] = "want verbal suffix"
                        datapoint["rule_order"] = "83"
                        datapoint["applied_rule"] += "2.18 want was applied, "
                        # nichs getroffen (ㄹ것이 plus에서 이미 다 cover되는 듯)
                    elif re_morphmatch(morphs, morph="^(고싶|ㄹ것이|고싶지않)$", pos="^X$") and datapoint["zp_hono"] == "plus": 
                        datapoint["subject_by_rule"] = "2sh+"
                        datapoint["group"] = "want statement"
                        datapoint["rule"] = "want periphrastic construction"
                        datapoint["rule_order"] = "84"
                        datapoint["applied_rule"] += "2.19.1 want(plus) was applied, "
                        # definition of subject is still the problem e.g. 통로를 걸어가셔서 문을 열고 왼쪽으로 꺾으시면 바로 보이실겁니다 . (화장실? 당신?)
#                     elif re_morphmatch(morphs, morph="^(생기|나오|멀|멀쩡하|정교하|기품있|편리하|덜컹거리|싱싱하|저렴하|부드럽|아슬하|저렴하|잠기|지나|사라지|조용하|나뉘|멀쩡하|딱맞|유효하|독하|생기|개운하|고급스러|독특하|좋아보이|뿌옇|특이하|가동하|붐비|저렴|유치하|잘어울리|거대하|부풀어오르|특별하|한산하|훌륭하|열|견고하|터지|동일하|알록달록하|달|어둡|저렴하|대단하|상영하|유명하|저렴하|기품있|경쾌하|튼튼하|화려하|많|밀리|열리|작동하|들어있|닫|간단하|담백하|적당하|남|빠듯하|돌|차|늦|복잡하|기|짧|머|저렴하|아담하|가벼우|어둡|밝|쏠리|부드럽|달|다르|크|작|적|활기차|비리|고소하|저렴하|달|싱싱하|맑|유명하|굳|운치있|혼잡하|독특하|똑같|조용하|한산하|매콤하|새콤하|꼬불꼬불하|다르|징그럽|모자르|시끄럽|심하|초과하|위치하|심해지|높|어울리|고장나|남성스럽|상하|익혀지|멋지|딸리|흔들리|작|싱겁|바뀌|깨지|끝나|이륙하|귀엽|험하|무섭|문제없|번거롭|어렵|쉽|아름답|예쁘|답답하|더럽|만만하|신기하|나|재밌|재미있|지루하|벗겨지|무겁|깨|튀|마르|위험하|이색적이|가볍|느끼하|맛있|불편하|시원하|덥|춥|편하|불편하|출발하|편안하|아프|관계없|싸|비싸|가능하|불가능하|걸리|고쳐지|깨끗하|나오|내리|다니|줄어들|되|들어오|마련되|나아지|맞|매우|맵|멀|가깝|보이|비슷하|비치되|시원하|시작하|없|아니|이|이기|이색적이|있|잘되|충분하|\w+되|\w+어지|\w+히)$", pos="^(J|V|N)$"):
#                         datapoint["subject_by_rule"] = "none"
#                         datapoint["group"] = "exclude"
#                         datapoint["rule"] = "exclude predicate"
#                         datapoint["rule_order"] = "85"
#                         datapoint["applied_rule"] += "2.19.2 exclude_predicate1 was applied, "
                        # many predicates are excluded already here
#                     elif re_morphmatch(morphs, morph="^(어있|아있|아지|어지|지|ㄹ만하|게되어있)$", pos="^X$"):
#                         datapoint["subject_by_rule"] = "none"
#                         datapoint["group"] = "exclude"
#                         datapoint["rule"] = "exclude predicate"
#                         datapoint["rule_order"] = "86"
#                         datapoint["applied_rule"] += "2.19.3 exclude_predicate2 was applied, "
                    elif re_morphmatch(morphs, morph="^(고싶|고싶지않)$", pos="^X$") and datapoint["zp_mood"] == "decl": 
                        datapoint["subject_by_rule"] = "1sh+"
                        datapoint["group"] = "want statement"
                        datapoint["rule"] = "want periphrastic construction"
                        datapoint["rule_order"] = "87"
                        datapoint["applied_rule"] += "2.19.4 want was applied, "
                        # coverage and precion high
                    elif re_morphmatch(morphs, morph="^(ㄹ것이|ㄹ거라)$", pos="^X$") and datapoint["zp_mood"] == "decl": 
                        datapoint["subject_by_rule"] = "1sh+"
                        datapoint["group"] = "want statement"
                        datapoint["rule"] = "want periphrastic construction"
                        datapoint["rule_order"] = "88"
                        datapoint["applied_rule"] += "2.19.5 want was applied, "
#                     elif re_morphmatch(morphs, morph="^(\w+다고하|\w+라고하|다고하)$", pos="^X$"):
#                         datapoint["subject_by_rule"] = "none"
#                         datapoint["group"] = "exclude"
#                         datapoint["rule"] = "exclude predicate"
#                         datapoint["rule_order"] = "89"
#                         datapoint["applied_rule"] += "2.19.6 exclude_hearsay was applied, "
              
                        
                    elif re_morphmatch(morphs, morph="^고도$", pos="^E$") and re_morphmatch(morphs, morph="^싶$", pos="^J$") and datapoint["zp_mood"] == "decl": 
                        datapoint["subject_by_rule"] = "1sh+"
                        datapoint["group"] = "want statement"
                        datapoint["rule"] = "want periphrastic construction"
                        datapoint["rule_order"] = "90"
                        datapoint["applied_rule"] += "2.20 want was applied, "
                        # coverage really low, one case

                    else: 
                        datapoint["subject_by_rule"] = "N/A" 

#                     # 9777 getroffen (56.7% of 17,266)

                    if datapoint["subject_by_rule"] == "N/A": 
                        if re_morphmatch(morphs, morph="^(을까요|ㄹ까요|을까|ㄹ까)$", pos="^E$") and datapoint["zp_hono"] == "plus":  
                            datapoint["subject_by_rule"] = "2sh+"
                            datapoint["group"] = "suggestory formula"
                            datapoint["rule"] = "suggestory verbal suffix"
                            datapoint["rule_order"] = "91"
                            datapoint["applied_rule"] += "3.1.1 suggestory(plus) was applied, "
                        elif re_morphmatch(morphs, morph="^(을까요|ㄹ까요|을까|ㄹ까)$", pos="^E$") and datapoint["zp_mood"] == "ques":  
                            datapoint["subject_by_rule"] = "1sh+"
                            datapoint["group"] = "suggestory formula"
                            datapoint["rule"] = "suggestory verbal suffix"
                            datapoint["rule_order"] = "92"
                            datapoint["applied_rule"] += "3.1.2 suggestory was applied, "
                            # 10991, 11546은 나중에 exclude list로 제외돼야 함! e.g. 죄송하지만 조금만 빨리 가주을까요 ? 원래 데이터 수정해도 반영이 안됨
#                         elif re_morphmatch(morphs, morph="^(는게좋|는게낫|는것이좋|면좋)$", pos="^X$") and datapoint["zp_mood"] == "decl":
#                             datapoint["subject_by_rule"] = "1sh+"
#                             datapoint["group"] = "suggestory formula"
#                             datapoint["rule"] = "suggestory periphrastic construction"
#                             datapoint["applied_rule"] += "3.2 suggestory was applied, "
                            # mixed subjects 일단 보류
#                         elif re_morphmatch(morphs, morph="^(는게좋|는게낫|는것이좋|면좋)$", pos="^X$") and datapoint["zp_mood"] == "ques":
#                             datapoint["subject_by_rule"] = "1sh+"
#                             datapoint["group"] = "suggestory formula"
#                             datapoint["rule"] = "suggestory periphrastic construction"
#                             datapoint["applied_rule"] += "3.2.1 suggestory was applied, "
                            # nichts getroffen
                        elif re_morphmatch(morphs, morph="^(ㄴ적이?있|ㄴ적이?없)$", pos="^X$") and datapoint["zp_mood"] == "decl":
                            datapoint["subject_by_rule"] = "1sh+"
                            datapoint["group"] = "experience"
                            datapoint["rule"] = "experience periphrastic construction"
                            datapoint["rule_order"] = "93"
                            datapoint["applied_rule"] += "3.3 experience was applied, "
                            # coverage not that high
                        elif re_morphmatch(morphs, morph="^(ㄴ적이?있|ㄴ적이?없)$", pos="^X$") and datapoint["zp_mood"] == "ques":
                            datapoint["subject_by_rule"] = "2sh+"
                            datapoint["group"] = "experience"
                            datapoint["rule"] = "experience periphrastic construction"
                            datapoint["rule_order"] = "94"
                            datapoint["applied_rule"] += "3.4 experience was applied, "
                            # nichts getroffen (covered by plus)
                        elif re_morphmatch(morphs, morph="^더니$", pos="^E$") and datapoint["tense_zp"] == "past":
                            datapoint["subject_by_rule"] = "1sh+"
                            datapoint["group"] = "experience"
                            datapoint["rule"] = "experience periphrastic construction"
                            datapoint["rule_order"] = "95"
                            datapoint["applied_rule"] += "3.5 experience was applied, "
                            # coverage not that high
                        elif re_morphmatch(morphs, morph="^([아어]드리)$", pos="^X$") and datapoint["zp_hono"] == "plus":
                            datapoint["subject_by_rule"] = "2sh+"
                            datapoint["group"] = "politeness"
                            datapoint["rule"] = "politeness auxiliary verb"
                            datapoint["rule_order"] = "96"
                            datapoint["applied_rule"] += "3.6 politeness(plus) was applied, "
                            # nichts getroffen
                        elif re_morphmatch(morphs, morph="^(아버리|어버리|잃어버리)$", pos="^(X|V)$") and datapoint["zp_hono"] == "plus":
                            datapoint["subject_by_rule"] = "2sh+"
                            datapoint["group"] = "subjectivication"
                            datapoint["rule"] = "aspect auxiliary verb"
                            datapoint["rule_order"] = "97"
                            datapoint["applied_rule"] += "3.7 subjectivication(plus) was applied, "
                            # coverage not that high
                        elif re_morphmatch(morphs, morph="^(아보|어보|가보|해보|써보|타보|찾아보|재보)$", pos="^(X|V)$") and datapoint["zp_hono"] == "plus":
                            datapoint["subject_by_rule"] = "2sh+"
                            datapoint["group"] = "try"
                            datapoint["rule"] = "modality auxiliary verb"
                            datapoint["rule_order"] = "98"
                            datapoint["applied_rule"] += "3.8 try(plus) was applied, "
                            # clau도 포함됨 e.g.써보시고 

                        else: 
                            datapoint["subject_by_rule"] = "N/A" 
                            
#                         performative_head_list = ["부탁", "권유"]
#                         cognition_head_list = ["예정","생각","기억","확인","깜박","예상","착각","이해","확실","집중","계획","숙지","오해","파악","동의","사과","깜빡"]
#                         sensation_head_list = ["시음","민감","구경","시원"]
#                         feeling_head_list = ["실망","공포","영광","장담","한시름","미안","실례"]
                                               
                        if datapoint["subject_by_rule"] == "N/A":
                            if [x for x in performative_head_list if x in datapoint["verb"]] and datapoint["zp_hono"] == "plus":
                                datapoint["subject_by_rule"] = "2sh+" 
                                datapoint["group"] = "performative"
                                datapoint["rule"] = "performative predicate"
                                datapoint["rule_order"] = "99"
                                datapoint["applied_rule"] += "4.1 performative(plus) was applied, "
                                # nichts getroffen
                            elif re_morphmatch(morphs, morph="^부탁하$", pos="^V$") and re_morphmatch(morphs, morph="^(면되|아도되|ㄹ수있)$", pos="^X$") and datapoint["zp_mood"] == "ques":
                                datapoint["subject_by_rule"] = "1sh+"
                                datapoint["group"] = "permission"
                                datapoint["rule"] = "permission periphrastic construction"
                                datapoint["rule_order"] = "100"
                                datapoint["applied_rule"] += "4.2 permission was applied, "
                                # coverage really low 모닝콜 서비스를 부탁해도 되겠습니까? 이미 위에서 커버됨 2.5+6 permission was applied, / one case
                            elif re_morphmatch(morphs, morph="^(기?원하|바라|기바라|추천하|필요하|필요없|ㄹ필요는없|빌|권하|지않아도되)$", pos="^(V|X|J)$") and datapoint["zp_hono"] == "plus":
                                datapoint["subject_by_rule"] = "2sh+"
                                datapoint["group"] = "performative"
                                datapoint["rule"] = "performative predicate"
                                datapoint["rule_order"] = "101"
                                datapoint["applied_rule"] += "4.3 performative(plus) was applied, "
                                # coverage and precision very high
                                # 나중에 zero referents 틀린 case 찾아볼 때는 "subject_error_analysis_080822.xlxs" 참고
                            elif re_morphmatch(morphs, morph="^(는줄알|ㄴ줄알|을줄알|ㄹ줄알|ㄹ줄아|잘알|알|아|알게되|잘아|모르|는줄모르|을줄모르|ㄹ줄모르|어야할지모르|는지모르|ㄴ지모르|을지모르|ㄹ지모르|을수있을지모르|알아들|궁금하|고집하)$", pos="^(X|V|J)$") and datapoint["zp_hono"] == "plus":
                                datapoint["subject_by_rule"] = "2sh+" 
                                datapoint["group"] = "experience"
                                datapoint["rule"] = "cognition predicate"
                                datapoint["rule_order"] = "102"
                                datapoint["applied_rule"] += "4.4 cognition(plus) was applied, "
                                # coverage and precision high
                            elif re_morphmatch(morphs, morph="^(잊|잊어버리|믿|알아듣|외우|헷갈리)$", pos="^V$") and datapoint["zp_hono"] == "plus":
                                datapoint["subject_by_rule"] = "2sh+"
                                datapoint["group"] = "experience"
                                datapoint["rule"] = "cognition predicate"
                                datapoint["rule_order"] = "103"
                                datapoint["applied_rule"] += "4.5 cognition(plus) was applied, "
                                # coverage low, only 3 cases
                            elif [x for x in cognition_head_list if x in datapoint["verb"]] and datapoint["zp_hono"] == "plus":
                                datapoint["subject_by_rule"] = "2sh+"
                                datapoint["group"] = "experience"
                                datapoint["rule"] = "cognition predicate"
                                datapoint["rule_order"] = "104"
                                datapoint["applied_rule"] += "4.6 cognition(plus) was applied, "
                                # "예정이다", '예정하다'
                                # coverage ok
                            elif re_morphmatch(morphs, morph="^(보|못보|살펴보|둘러보|보러오|듣|들어서|들|들어보|맛보|배고프|싱거우|간지럽)$", pos="^(V|J)$") and datapoint["zp_hono"] == "plus":
                                datapoint["subject_by_rule"] = "2sh+"
                                datapoint["group"] = "experience"
                                datapoint["rule"] = "sensation predicate"
                                datapoint["rule_order"] = "105"
                                datapoint["applied_rule"] += "4.7 sensation(plus) was applied, "
                                # coverage ok
                            elif [x for x in sensation_head_list if x in datapoint["verb"]] and datapoint["zp_hono"] == "plus":
                                datapoint["subject_by_rule"] = "2sh+"
                                datapoint["group"] = "experience"
                                datapoint["rule"] = "sensation predicate"
                                datapoint["rule_order"] = "106"
                                datapoint["applied_rule"] += "4.8 sensation(plus) was applied, "
                                # coverage low, only 2 cases
                            elif re_morphmatch(morphs, morph="^(춥|따뜻하|덥|후텁지근하|찝찝하|아프|쓰라리|만지|만져보|느끼)$", pos="^(J|V)$") and datapoint["zp_hono"] == "plus":
                                datapoint["subject_by_rule"] = "2sh+"
                                datapoint["group"] = "experience"
                                datapoint["rule"] = "sensation predicate"
                                datapoint["rule_order"] = "107"
                                datapoint["applied_rule"] += "4.9 sensation(plus) was applied, "
                                # coverage low, only 3 cases
                            elif re_morphmatch(morphs, morph="^(괜찮|고맙|불편하|피곤하|지루하|흥미롭|곤란하|불쾌하|거북하|불안하|힘들|힘드|기힘들|좋|어쩔수없|별수없)$", pos="^(J|X)$") and datapoint["zp_hono"] == "plus":
                                datapoint["subject_by_rule"] = "2sh+"
                                datapoint["group"] = "experience"
                                datapoint["rule"] = "feeling predicate/plus"
                                datapoint["rule_order"] = "108"
                                datapoint["applied_rule"] += "4.10 feeling(plus) was applied, "
                                # coverage ok
                            elif re_morphmatch(morphs, morph="^(좋아하|부담되|기대되|마음에들|기대하|걱정되|걱정하|고민되|고민하|놀라|당황하|놀래|놀랍|행복하|미안하|죄송하|죄송|답답하|부담스럽|의아하|질색이)$", pos="^(V|N)$") and datapoint["zp_hono"] == "plus":
                                datapoint["subject_by_rule"] = "2sh+"
                                datapoint["group"] = "experience"
                                datapoint["rule"] = "feeling predicate"
                                datapoint["rule_order"] = "109"
                                datapoint["applied_rule"] += "4.11 feeling(plus) was applied, "
                                # coverage not that high, 6 cases
                            elif re_morphmatch(morphs, morph="^(후회하|감사하|감사드리|감탄하|횡재하|들뜨|떨리|만족하|만족스럽|망설여지|망설이|싫|싫어하)$", pos="^(V|J)$") and datapoint["zp_hono"] == "plus":
                                datapoint["subject_by_rule"] = "2sh+"
                                datapoint["group"] = "experience"
                                datapoint["rule"] = "feeling predicate"
                                datapoint["rule_order"] = "110"
                                datapoint["applied_rule"] += "4.12 feeling(plus) was applied, "
                                # nichts getroffen
                            elif [x for x in feeling_head_list if x in datapoint["verb"]] and datapoint["zp_hono"] == "plus":
                                datapoint["subject_by_rule"] = "2sh+"
                                datapoint["group"] = "experience"
                                datapoint["rule"] = "feeling predicate"
                                datapoint["rule_order"] = "111"
                                datapoint["applied_rule"] += "4.13 feeling(plus) was applied, "
                                # nichts getroffen
                            elif re_morphmatch(morphs, morph="^(익숙하|자부하|망설여지|망설이|아쉽|상관없|즐겁|재미있|재미없|즐거우|무섭|기쁘|행복하|난감하|반갑|환영하|신기하|당황스럽|막막하|안타깝|부럽)$", pos="^(J|V)$") and datapoint["zp_hono"] == "plus":
                                datapoint["subject_by_rule"] = "2sh+"
                                datapoint["group"] = "experience"
                                datapoint["rule"] = "feeling predicate"
                                datapoint["rule_order"] = "112"
                                datapoint["applied_rule"] += "4.14 feeling(plus) was applied, "
                                # coverage low, only 2 cases
                            elif re_morphmatch(morphs, morph="^(드시|안드|계시|말씀하|고계시)$", pos="^(V|X)$") and datapoint["zp_hono"] == "plus":
                                datapoint["subject_by_rule"] = "2sh+"
                                datapoint["group"] = "politeness"
                                datapoint["rule"] = "politeness predicate"
                                datapoint["rule_order"] = "113"
                                datapoint["applied_rule"] += "4.15.1 politeness(plus) was applied, "
                                # coverage very high, some cases lower the precision
                            elif re_morphmatch(morphs, morph="^(드시|안드|계시|말씀하|고계시)$", pos="^(V|X)$"):
                                datapoint["subject_by_rule"] = "2sh+"
                                datapoint["group"] = "politeness"
                                datapoint["rule"] = "politeness predicate"
                                datapoint["rule_order"] = "114"
                                datapoint["applied_rule"] += "4.15.2 politeness was applied, "
                                # coverage ok, two cases off
                            elif re_morphmatch(morphs, morph="^말씀하$", pos="^V$") and re_morphmatch(morphs, morph="^아드리$", pos="^X$"):
                                datapoint["subject_by_rule"] = "1sh+"
                                datapoint["group"] = "politeness"
                                datapoint["rule"] = "politeness predicate"
                                datapoint["rule_order"] = "115"
                                datapoint["applied_rule"] += "4.16.1 politeness was applied, "
                            elif re_morphmatch(morphs, morph="^축하드리$", pos="^V$"):
                                datapoint["subject_by_rule"] = "1sh+"
                                datapoint["group"] = "politeness"
                                datapoint["rule"] = "politeness predicate"
                                datapoint["rule_order"] = "116"
                                datapoint["applied_rule"] += "4.16.2 politeness was applied, "
                                
                                # nichts getroffen
#                             elif re_morphmatch(morphs, morph="^(귀엽|험하|무섭|문제없|번거롭|어렵|쉽|아름답|예쁘|답답하|더럽|만만하|신기하|재밌|재미있|지루하|무겁|이색적이|가볍|느끼하|맛있|불편하|시원하|덥|춥|편하|불편하|편안하|아프|관계없|싸|비싸|가능하|불가능하|걸리|고쳐지|깨끗하|나오|내리|다니|되|들어오|마련되|맞|매우|맵|멀|가깝|보이|비슷하|비치되|시원하|시작하|없|아니|이|이기|이색적이|있|잘되|충분하|\w+되|\w+어지|\w+히)$", pos="^(J|V)$"):
#                                 datapoint["subject_by_rule"] = "none"
#                                 datapoint["group"] = "exclude"
#                                 datapoint["rule"] = "exclude predicate"
#                                 datapoint["applied_rule"] += "4.17.1 exclude_predicate1 was applied, "
#                             elif re_morphmatch(morphs, morph="^(어있|아있|아지|어지)$", pos="^X$"):
#                                 datapoint["subject_by_rule"] = "none"
#                                 datapoint["group"] = "exclude"
#                                 datapoint["rule"] = "exclude predicate"
#                                 datapoint["applied_rule"] += "4.17.2 exclude_predicate2 was applied, "
                            elif re_morphmatch(morphs, morph="^(을것이|ㄹ것이|을거이|ㄹ거이)$", pos="^(X|E)$") and datapoint["zp_hono"] == "plus":
                                datapoint["subject_by_rule"] = "2sh+"
                                datapoint["group"] = "want statement"
                                datapoint["rule"] = "want(plus) periphrastic construction"
                                datapoint["rule_order"] = "117"
                                datapoint["applied_rule"] += "4.17.1 want/guess1 was applied, "
                            # coverage ok
                            # definition of subject lower the precision, two cases e.g. 쑥 내려가실거예요 ./ 노랫소리처럼 들리실거예요 .
                            elif re_morphmatch(morphs, morph="^(을것이|ㄹ것이|을거이|ㄹ거이)$", pos="^(X|E)$") and datapoint["zp_mood"] == "decl":
                                datapoint["subject_by_rule"] = "1sh+"
                                datapoint["group"] = "want statement"
                                datapoint["rule"] = "want periphrastic construction"
                                datapoint["rule_order"] = "118"
                                datapoint["applied_rule"] += "4.17.2 want/guess1 was applied, "
                                # coverage ok but precision is really low because of the ambiguity between the first and second person
                            elif re_morphmatch(morphs, morph="^(을것이|ㄹ것이|을거이|ㄹ거이)$", pos="^(X|E)$") and datapoint["zp_mood"] == "ques":
                                datapoint["subject_by_rule"] = "2sh+" 
                                datapoint["group"] = "want statement"
                                datapoint["rule"] = "want periphrastic construction"
                                datapoint["rule_order"] = "119"
                                datapoint["applied_rule"] += "4.17.3 want/guess2 was applied, "   
                            elif re_morphmatch(morphs, morph="^지않을것이$", pos="^X$") and datapoint["zp_mood"] == "decl":
                                datapoint["subject_by_rule"] = "2sh+"
                                datapoint["group"] = "guess"
                                datapoint["rule"] = "guess periphrastic construction"
                                datapoint["rule_order"] = "120"
                                datapoint["applied_rule"] += "4.17.4 guess1 was applied, "
                                # nichts getroffen 
                            elif re_morphmatch(morphs, morph="^진않$", pos="^X$") and re_morphmatch(morphs, morph="^을거이$", pos="^X$") and datapoint["zp_mood"] == "decl":
                                datapoint["subject_by_rule"] = "2sh+"
                                datapoint["group"] = "guess"
                                datapoint["rule"] = "guess periphrastic construction"
                                datapoint["rule_order"] = "121"
                                datapoint["applied_rule"] += "4.17.5 guess2 was applied, "
                                # coverage and precision very low

                            elif [x for x in cognition_head_list if x in datapoint["verb"]] and datapoint["zp_mood"] == "ques":
                                datapoint["subject_by_rule"] = "2sh+"
                                datapoint["group"] = "experience"
                                datapoint["rule"] = "cognition predicate"
                                datapoint["rule_order"] = "122"
                                datapoint["applied_rule"] += "4.18 cognition was applied, "
                                # "확인"이 조금 문제 , 1sh+를 만듦 
                                # coverage ok but precision low because of segments with "확인"
                            elif [x for x in performative_head_list if x in datapoint["verb"]] and datapoint["zp_mood"] == "ques":
                                datapoint["subject_by_rule"] = "2sh+"
                                datapoint["group"] = "performative"
                                datapoint["rule"] = "performative predicate"
                                datapoint["rule_order"] = "123"
                                datapoint["applied_rule"] += "4.19 performative was applied, "
                                # nichts getroffen
                            elif re_morphmatch(morphs, morph="^(기?원하|바라|기바라|추천하|필요하|필요없|ㄹ필요는없|지않아도되|빌|권하)$", pos="^(V|X|J)$") and datapoint["zp_mood"] == "ques":
                                datapoint["subject_by_rule"] = "2sh+"
                                datapoint["group"] = "performative"
                                datapoint["rule"] = "performative predicate"
                                datapoint["rule_order"] = "124"
                                datapoint["applied_rule"] += "4.20 performative was applied, "
                                # coverage not that high, one case of with "필요하다", hard even for human
                            elif re_morphmatch(morphs, morph="^(는줄알|ㄴ줄알|을줄알|ㄹ줄알|ㄹ줄아|잘알|알|아|알게되|잘아|모르|는줄모르|ㄴ즐모르|을줄모르|ㄹ줄모르|어야할지모르|는지모르|을지모르|ㄹ지모르|을수있을지모르|ㄹ수있을지모르|알아들|궁금하|고집하)$", pos="^(X|V)$") and datapoint["zp_mood"] == "ques":
                                datapoint["subject_by_rule"] = "1sh+"
                                datapoint["group"] = "experience"
                                datapoint["rule"] = "cognition predicate"
                                datapoint["rule_order"] = "125"
                                datapoint["applied_rule"] += "4.21 cognition was applied, "
                            elif [x for x in cognition_head_list if x in datapoint["verb"]] and datapoint["zp_mood"] == "ques":
                                datapoint["subject_by_rule"] = "2sh+"
                                datapoint["group"] = "experience"
                                datapoint["rule"] = "cognition predicate"
                                datapoint["rule_order"] = "126"
                                datapoint["applied_rule"] += "4.22 cognition was applied, "
                                # nichts getroffen
                            elif re_morphmatch(morphs, morph="^(잊|잊어버리|믿|알아듣|외우|헷갈리)$", pos="^V$") and datapoint["zp_mood"] == "ques":
                                datapoint["subject_by_rule"] = "2sh+"
                                datapoint["group"] = "experience"
                                datapoint["rule"] = "cognition predicate"
                                datapoint["rule_order"] = "127"
                                datapoint["applied_rule"] += "4.23 cognition was applied, "
                                # coverage not that high
                            elif re_morphmatch(morphs, morph="^(보|못보|살펴보|둘러보|보러오|듣|들어서|들|들어보|맛보|배고프|싱거우|간지럽)$", pos="^V$") and datapoint["zp_mood"] == "ques":
                                datapoint["subject_by_rule"] = "2sh+"
                                datapoint["group"] = "experience"
                                datapoint["rule"] = "sensation predicate"
                                datapoint["rule_order"] = "128"
                                datapoint["applied_rule"] += "4.24 sensation was applied, "
                                # coverage and precision very low
                            elif [x for x in sensation_head_list if x in datapoint["verb"]] and datapoint["zp_mood"] == "ques":
                                datapoint["subject_by_rule"] = "1sh+"
                                datapoint["group"] = "experience"
                                datapoint["rule"] = "sensation predicate"
                                datapoint["rule_order"] = "129"
                                datapoint["applied_rule"] += "4.25 sensation was applied, "
                                # coverage and precision very low, ambiguous cases include between discourse participants and general people e.g. A 코스는 어디를 구경합니까 ?
                            elif re_morphmatch(morphs, morph="^(춥|따뜻하|덥|후텁지근하|찝찝하|아프|쓰라리|만지|만져보|느끼)$", pos="^(J|V)$") and datapoint["zp_mood"] == "ques":
                                datapoint["subject_by_rule"] = "2sh+"
                                datapoint["group"] = "experience"
                                datapoint["rule"] = "sensation predicate"
                                datapoint["rule_order"] = "130"
                                datapoint["applied_rule"] += "4.26 sensation was applied, "
                                # coverage not that high and segmentation problem 25303 이제 안전벨트 풀어도 괜찮은거죠 ? exception일수도
                            elif re_morphmatch(morphs, morph="^(고맙|불편하|피곤하|지루하|흥미롭|곤란하|불쾌하|거북하|불안하|힘들|힘드|기힘들)$", pos="^(J|X)$") and datapoint["zp_mood"] == "ques":
                                datapoint["subject_by_rule"] = "2sh+"
                                datapoint["group"] = "experience"
                                datapoint["rule"] = "feeling predicate"
                                datapoint["rule_order"] = "131"
                                datapoint["applied_rule"] += "4.27 feeling was applied, "
                                # nichts getroffen
                            elif re_morphmatch(morphs, morph="^(좋아하|부담되|기대되|마음에들|기대하|걱정되|걱정하|고민되|고만하|놀라|당황하|놀래|놀랍|행복하|미안하|죄송하|죄송|답답하|부담스럽|의아하|질색이)$", pos="^(V|J|N)$") and datapoint["zp_mood"] == "ques":
                                datapoint["subject_by_rule"] = "2sh+"
                                datapoint["group"] = "experience"
                                datapoint["rule"] = "feeling predicate"
                                datapoint["rule_order"] = "132"
                                datapoint["applied_rule"] += "4.28 feeling was applied, "
                                # coverage very low
                            elif re_morphmatch(morphs, morph="^(후회하|감사하|감사드리|감탄하|횡재하|들뜨|떨리|만족하|만족스럽|망설여지|망설이|싫|싫어하)$", pos="^V$") and datapoint["zp_mood"] == "ques":
                                datapoint["subject_by_rule"] = "2sh+"
                                datapoint["group"] = "experience"
                                datapoint["rule"] = "feeling predicate"
                                datapoint["rule_order"] = "133"
                                datapoint["applied_rule"] += "4.29 feeling was applied, "
                                # nichts getroffen
                            elif [x for x in feeling_head_list if x in datapoint["verb"]] and datapoint["zp_mood"] == "ques":
                                datapoint["subject_by_rule"] = "2sh+"
                                datapoint["group"] = "experience"
                                datapoint["rule"] = "feeling predicate"
                                datapoint["rule_order"] = "134"
                                datapoint["applied_rule"] += "4.30 feeling was applied, "
                                # nichts getroffen
                            elif re_morphmatch(morphs, morph="^(익숙하|자부하|망설여지|망설이|아쉽|상관없|즐겁|재미있|재미없|즐거우|무섭|기쁘|행복하|난감하|반갑|환영하|신기하|당황스럽|막막하|안타깝|부럽)$", pos="^J$") and datapoint["zp_mood"] == "ques":
                                datapoint["subject_by_rule"] = "2sh+"
                                datapoint["group"] = "experience"
                                datapoint["rule"] = "feeling predicate"
                                datapoint["rule_order"] = "135"
                                datapoint["applied_rule"] += "4.31 feeling was applied, "
                                # nichts getroffen
                            else: 
                                datapoint["subject_by_rule"] = "N/A" 

                            if datapoint["subject_by_rule"] == "N/A":
                                if [x for x in cognition_head_list if x in datapoint["verb"]] and datapoint["zp_mood"] == "decl":
                                    datapoint["subject_by_rule"] = "1sh+"
                                    datapoint["group"] = "experience"
                                    datapoint["rule"] = "cognition predicate"
                                    datapoint["rule_order"] = "136"
                                    datapoint["applied_rule"] += "5.1 coginition was applied, "
                                    # coverage high,some ambiguous cases lower the precision
                                elif re_morphmatch(morphs, morph="^(면되|으면되|기만하면되|아도되|어도되|아가|도되|지않아도되|아야하|어야하|여야하|야하|어야할것이|아야할것이|어야할것같|아야되|어야되|야되|아야죠|어야죠|야지요|으면안되|면안되)$", pos="^(X|E)$") and datapoint["zp_hono"] == "plus":
                                    datapoint["subject_by_rule"] = "2sh+"
                                    datapoint["group"] = "permission"
                                    datapoint["rule"] = "permission periphrastic construction/plus"
                                    datapoint["rule_order"] = "137"
                                    datapoint["applied_rule"] += "5.2.1 permission(plus) was applied, "
                                    # coverage and precision very high
                                elif re_morphmatch(morphs, morph="^(아야겠|어야겠)$", pos="^X$") and datapoint["zp_mood"] == "decl": 
                                    datapoint["subject_by_rule"] = "1sh+"
                                    datapoint["group"] = "want statement"
                                    datapoint["rule"] = "want periphrastic construction"
                                    datapoint["rule_order"] = "138"
                                    datapoint["applied_rule"] += "5.2.2 want was applied, "
                                    # coverage ok
                                elif re_morphmatch(morphs, morph="^(아야하|어야하|여야하|야하)$", pos="^X$") and re_morphmatch(morphs, morph="^겠$", pos="^E$") and datapoint["zp_mood"] == "decl": 
                                    datapoint["subject_by_rule"] = "1sh+"
                                    datapoint["group"] = "want statement"
                                    datapoint["rule"] = "want periphrastic construction"
                                    datapoint["rule_order"] = "139"
                                    datapoint["applied_rule"] += "5.2.3 want was applied, "
                                    # coverage and precision high
                                elif [x for x in performative_head_list if x in datapoint["verb"]] and datapoint["zp_mood"] == "decl":
                                    datapoint["subject_by_rule"] = "1sh+"
                                    datapoint["group"] = "performative"
                                    datapoint["rule"] = "performative predicate"
                                    datapoint["rule_order"] = "140"
                                    datapoint["applied_rule"] += "5.2.4 performative was applied, "
                                    # coverage and precision high
                                elif re_morphmatch(morphs, morph="^(기?원하|바라|기바라|추천하|필요하|필요없|ㄹ필요는없|빌|권하)$", pos="^(V|X|J)$") and datapoint["zp_mood"] == "decl":
                                    datapoint["subject_by_rule"] = "1sh+"
                                    datapoint["group"] = "performative"
                                    datapoint["rule"] = "performative predicate"
                                    datapoint["rule_order"] = "141"
                                    datapoint["applied_rule"] += "5.3 performative was applied, "
                                    # coverage and precision high
                                elif re_morphmatch(morphs, morph="^(는줄알|ㄴ줄알|을줄알|ㄹ줄알|ㄹ줄아|잘알|알|아|알게되|잘아|모르|는줄모르|을줄모르|ㄹ줄모르|어야할지모르|는지모르|ㄴ지모르|을지모르|ㄹ지모르|을수있을지모르|알아들|궁금하|고집하)$", pos="^(X|V|J)$") and datapoint["zp_mood"] == "decl":
                                    datapoint["subject_by_rule"] = "1sh+"
                                    datapoint["group"] = "experience"
                                    datapoint["rule"] = "cognition predicate"
                                    datapoint["rule_order"] = "142"
                                    datapoint["applied_rule"] += "5.4 cognition was applied, "
                                    # coverage and precision high
                                elif re_morphmatch(morphs, morph="^(잊|잊어버리|믿|알아듣|외우|헷갈리)$", pos="^V$") and datapoint["zp_mood"] == "decl":
                                    datapoint["subject_by_rule"] = "1sh+"
                                    datapoint["group"] = "experience"
                                    datapoint["rule"] = "cognition predicate"
                                    datapoint["rule_order"] = "143"
                                    datapoint["applied_rule"] += "5.5 cognition was applied, "
                                    # coverage not that high
                                elif re_morphmatch(morphs, morph="^(보|못보|살펴보|둘러보|보러오|듣|들어서|들|들어보|맛보|배고프|싱거우|간지럽)$", pos="^(V|J)$") and datapoint["zp_mood"] == "decl":
                                    datapoint["subject_by_rule"] = "1sh+"
                                    datapoint["group"] = "experience"
                                    datapoint["rule"] = "sensation predicate"
                                    datapoint["rule_order"] = "144"
                                    datapoint["applied_rule"] += "5.6 sensation was applied, "
                                    # coverage high, some cases are off 
                                elif [x for x in sensation_head_list if x in datapoint["verb"]] and datapoint["zp_mood"] == "decl":
                                    datapoint["subject_by_rule"] = "1sh+"
                                    datapoint["group"] = "experience"
                                    datapoint["rule"] = "sensation predicate"
                                    datapoint["rule_order"] = "145"
                                    datapoint["applied_rule"] += "5.7 sensation was applied, "
                                    # coverage and precision very low
                                elif re_morphmatch(morphs, morph="^(춥|따뜻하|덥|후텁지근하|찝찝하|아프|쓰라리|만지|만져보|느끼)$", pos="^(J|V)$") and datapoint["zp_mood"] == "decl":
                                    datapoint["subject_by_rule"] = "1sh+"
                                    datapoint["group"] = "experience"
                                    datapoint["rule"] = "sensation predicate"
                                    datapoint["rule_order"] = "146"
                                    datapoint["applied_rule"] += "5.8 sensation was applied, "
                                    # coverage and precision low, somce cases have the issue with the definition of subject e.g. 따뜻합니다 ./ 네, 손바닥이 부어오르고 쓰라리네요 .
                                elif re_morphmatch(morphs, morph="^(괜찮|고맙|불편하|피곤하|지루하|흥미롭|곤란하|불쾌하|거북하|불안하|힘들|힘드|기힘들|좋|어쩔수없|별수없)$", pos="^(J|X)$") and datapoint["zp_mood"] == "decl":
                                    datapoint["subject_by_rule"] = "1sh+"
                                    datapoint["group"] = "experience"
                                    datapoint["rule"] = "feeling predicate"
                                    datapoint["rule_order"] = "147"
                                    datapoint["applied_rule"] += "5.9 feeling was applied, "
                                    # 괜찮다, 좋다의 다의성으로 인해 off되는 case 많음
                                elif re_morphmatch(morphs, morph="^(좋아하|부담되|기대되|마음에들|기대하|걱정되|걱정하|고민되|고민하|놀라|당황하|놀래|놀랍|행복하|미안하|죄송하|죄송|답답하|부담스럽|의아하|질색이)$", pos="^(V|N)$") and datapoint["zp_mood"] == "decl":
                                    datapoint["subject_by_rule"] = "1sh+"
                                    datapoint["group"] = "experience"
                                    datapoint["rule"] = "feeling predicate"
                                    datapoint["rule_order"] = "148"
                                    datapoint["applied_rule"] += "5.10 feeling was applied, "
                                    # some cases with "마음에 들다" lower the precision and also related to the definition of subject issue
                                elif re_morphmatch(morphs, morph="^(후회하|감사하|감사드리|감탄하|횡재하|들뜨|떨리|만족하|만족스럽|망설여지|망설이|싫|싫어하)$", pos="^(V|J)$") and datapoint["zp_mood"] == "decl":
                                    datapoint["subject_by_rule"] = "1sh+"
                                    datapoint["group"] = "experience"
                                    datapoint["rule"] = "feeling predicate"
                                    datapoint["rule_order"] = "149"
                                    datapoint["applied_rule"] += "5.11 feeling was applied, "
                                    # coverage ok
                                elif [x for x in feeling_head_list if x in datapoint["verb"]] and datapoint["zp_mood"] == "decl":
                                    datapoint["subject_by_rule"] = "1sh+"
                                    datapoint["group"] = "experience"
                                    datapoint["rule"] = "feeling predicate"
                                    datapoint["rule_order"] = "150"
                                    datapoint["applied_rule"] += "5.12 feeling was applied, "
                                    # coverage not that high
                                elif re_morphmatch(morphs, morph="^(익숙하|자부하|망설여지|망설이|아쉽|상관없|즐겁|재미있|재미없|즐거우|무섭|기쁘|행복하|난감하|반갑|신기하|당황스럽|막막하|안타깝|부럽)$", pos="^(J|V)$") and re_morphmatch(morphs, morph="^겠$", pos="^E$"):
                                    datapoint["subject_by_rule"] = "2sh+"
                                    datapoint["group"] = "experience"
                                    datapoint["rule"] = "feeling predicate"
                                    datapoint["rule_order"] = "151"
                                    datapoint["applied_rule"] += "5.13.1 feeling was applied, "
                                    # coverage very low, one case
                                elif re_morphmatch(morphs, morph="^(익숙하|자부하|망설여지|망설이|아쉽|상관없|즐겁|재미있|재미없|즐거우|무섭|기쁘|행복하|난감하|반갑|신기하|당황스럽|막막하|안타깝|부럽)$", pos="^(J|V)$") and datapoint["zp_mood"] == "decl":
                                    datapoint["subject_by_rule"] = "1sh+"
                                    datapoint["group"] = "experience"
                                    datapoint["rule"] = "feeling predicate"
                                    datapoint["rule_order"] = "152"
                                    datapoint["applied_rule"] += "5.13.2 feeling was applied, "
                                    # coverage and precision high
                                elif re_morphmatch(morphs, morph="^(드리)$", pos="^(V)$") and datapoint["zp_mood"] == "decl":
                                    datapoint["subject_by_rule"] = "1sh+"
                                    datapoint["group"] = "politeness"
                                    datapoint["rule"] = "politeness predicate/auxiliary verb"
                                    datapoint["rule_order"] = "153"
                                    datapoint["applied_rule"] += "5.14.1 politeness was applied, "
                                    # coverage ok
                                elif re_morphmatch(morphs, morph="^드리$", pos="^V$") and datapoint["zp_mood"] == "ques":
                                    datapoint["subject_by_rule"] = "1sh+"
                                    datapoint["group"] = "politeness"
                                    datapoint["rule"] = "politeness predicate"
                                    datapoint["rule_order"] = "154"
                                    datapoint["applied_rule"] += "5.14.2 politeness was applied, "
                                    # coverage low, 3 cases
                                elif re_morphmatch(morphs, morph="^드리$", pos="^V$") and datapoint["verb_syn"] == "clau":
                                    datapoint["subject_by_rule"] = "1sh+"
                                    datapoint["group"] = "politeness"
                                    datapoint["rule"] = "politeness predicate"
                                    datapoint["rule_order"] = "155"
                                    datapoint["applied_rule"] += "5.14.3 politeness was applied, "
                                    # nichts getroffen
                                elif re_morphmatch(morphs, morph="^(아드리|어드리)$", pos="^X$"):
                                    datapoint["subject_by_rule"] = "1sh+"
                                    datapoint["group"] = "politeness"
                                    datapoint["rule"] = "politeness auxiliary verb"
                                    datapoint["rule_order"] = "156"
                                    datapoint["applied_rule"] += "5.14.4 politeness was applied, "
                                    # coverage and precision high
                                elif re_morphmatch(morphs, morph="^(가져다드리|알려드리|보여드리|해드리|연락드리|도와드리|돌려드리|전해드리|전화드리|갖다드리)$", pos="^V$"):
                                    datapoint["subject_by_rule"] = "1sh+"
                                    datapoint["group"] = "politeness"
                                    datapoint["rule"] = "politeness predicate"
                                    datapoint["rule_order"] = "157"
                                    datapoint["applied_rule"] += "5.15 politeness was applied, "
                                    # coverage and precision high
                                elif re_morphmatch(morphs, morph="^(아버리|어버리|잃어버리)$", pos="^(X|V)$") and datapoint["zp_mood"] == "decl":
                                    datapoint["subject_by_rule"] = "1sh+"
                                    datapoint["group"] = "subjectification"
                                    datapoint["rule"] = "aspect auxiliary verb"
                                    datapoint["rule_order"] = "158"
                                    datapoint["applied_rule"] += "5.16.1 subjectification was applied, "
                                elif re_morphmatch(morphs, morph="^(아버리|어버리|잃어버리)$", pos="^(X|V)$") and datapoint["zp_mood"] == "ques":
                                    datapoint["subject_by_rule"] = "2sh+"
                                    datapoint["group"] = "subjectification"
                                    datapoint["rule"] = "aspect auxiliary verb"
                                    datapoint["rule_order"] = "159"
                                    datapoint["applied_rule"] += "5.16.2 subjectification was applied, "
                                    # coverage very low, one case
                                elif re_morphmatch(morphs, morph="^(ㄹ수밖에없|는수밖에없)$", pos="^X$") and datapoint["zp_hono"] == "plus":
                                    datapoint["subject_by_rule"] = "2sh+"
                                    datapoint["group"] = "possibility"
                                    datapoint["rule"] = "possibility periphrastic construction"
                                    datapoint["rule_order"] = "160"
                                    datapoint["applied_rule"] += "5.17.1 possibility was applied, "
                                    # coverage really low, one case
                                elif re_morphmatch(morphs, morph="^(ㄹ수밖에없|는수밖에없)$", pos="^X$") and datapoint["zp_mood"] == "decl":
                                    datapoint["subject_by_rule"] = "1sh+"
                                    datapoint["group"] = "possibility"
                                    datapoint["rule"] = "possibility periphrastic construction"
                                    datapoint["rule_order"] = "161"
                                    datapoint["applied_rule"] += "5.17.2 possibility was applied, "
                                    # coverage low, four cases
#                                 elif re_morphmatch(morphs, morph="^(아야하|어야하|여야하|야하)$", pos="^X$") and re_morphmatch(morphs, morph="^겠$", pos="^E$") and datapoint["zp_hono"] == "plus": 
#                                     datapoint["subject_by_rule"] = "2sh+"
#                                     datapoint["group"] = "want statement"
#                                     datapoint["rule"] = "want periphrastic construction"
#                                     datapoint["applied_rule"] += "5.18.1 want(plus) was applied, "
#                                 elif re_morphmatch(morphs, morph="^(아야하|어야하|여야하|야하)$", pos="^X$") and re_morphmatch(morphs, morph="^겠$", pos="^E$") and datapoint["zp_mood"] == "decl": 
#                                     datapoint["subject_by_rule"] = "1sh+"
#                                     datapoint["group"] = "want statement"
#                                     datapoint["rule"] = "want periphrastic construction"
#                                     datapoint["applied_rule"] += "5.18.2 want was applied, "
                                elif re_morphmatch(morphs, morph="^(아야하|어야하|여야하|야하)$", pos="^X$") and re_morphmatch(morphs, morph="^겠$", pos="^E$") and datapoint["verb_syn"] == "clau": 
                                    datapoint["subject_by_rule"] = "1sh+"
                                    datapoint["group"] = "want statement"
                                    datapoint["rule"] = "want periphrastic construction"
                                    datapoint["rule_order"] = "162"
                                    datapoint["applied_rule"] += "5.18 want was applied, "
                                    # nichts getroffen
                                elif re_morphmatch(morphs, morph="^야하$", pos="^X$") and re_morphmatch(morphs, morph="^는데요?$", pos="^E$") and datapoint["zp_mood"] == "decl": 
                                    datapoint["subject_by_rule"] = "1sh+"
                                    datapoint["group"] = "obligation statement"
                                    datapoint["rule"] = "obligation periphrastic construction"
                                    datapoint["rule_order"] = "163"
                                    datapoint["applied_rule"] += "5.19 obligation was applied, "
                                    # coverage low, 4 cases
                                elif re_morphmatch(morphs, morph="^어보$", pos="^X$") and re_morphmatch(morphs, morph="^려고요$", pos="^E$") and datapoint["zp_mood"] == "decl":
                                    datapoint["subject_by_rule"] = "1sh+"
                                    datapoint["group"] = "try"
                                    datapoint["rule"] = "modality auxiliary verb"
                                    datapoint["rule_order"] = "164"
                                    datapoint["applied_rule"] += "5.20 try was applied, "
                                    # nichts getroffen
                                elif re_morphmatch(morphs, morph="^어보$", pos="^X$") and re_morphmatch(morphs, morph="^려고요$", pos="^E$") and datapoint["zp_mood"] == "ques":
                                    datapoint["subject_by_rule"] = "2sh+"
                                    datapoint["group"] = "try"
                                    datapoint["rule"] = "modality auxiliary verb"
                                    datapoint["rule_order"] = "165"
                                    datapoint["applied_rule"] += "5.21 try was applied, "
                                    # nichts getroffen
                                    
                                else: 
                                    datapoint["subject_by_rule"] = "N/A" 


                                if datapoint["subject_by_rule"] == "N/A":
                                    if re_morphmatch(morphs, morph="^(아야겠|어야겠|어야|야하|아야)$", pos="^X$") and datapoint["zp_hono"] == "plus": 
                                        datapoint["subject_by_rule"] = "2sh+"
                                        datapoint["group"] = "obligation statement"
                                        datapoint["rule"] = "obligation periphrastic construction/plus"
                                        datapoint["rule_order"] = "166"
                                        datapoint["applied_rule"] += "6.1 obligation(plus) was applied, "
                                        # coverage really low, one case
                                    elif re_morphmatch(morphs, morph="^(려고하|으려고하|사려하|려|려하|으려하|으려고|려고|어보려고하|려고그러|려고요|으려고요|ㄹ려고요|려는것이|고자하|ㄹ려고하|기로하|ㄹ려구요|려구요|을것이|ㄹ것이|을거이|ㄹ거이)$", pos="^(X|E)$") and datapoint["zp_hono"] == "plus":
                                        datapoint["subject_by_rule"] = "2sh+"
                                        datapoint["group"] = "want statement"
                                        datapoint["rule"] = "want periphrastic construction/plus"
                                        datapoint["rule_order"] = "167"
                                        datapoint["applied_rule"] += "6.2 obligation(plus) was applied, "
                                        # nichts getroffen
                                    elif re_morphmatch(morphs, morph="^(아야하|어야하|여야하)$", pos="^X$") and re_morphmatch(morphs, morph="^겠$", pos="^E$") and datapoint["zp_hono"] == "plus": 
                                        datapoint["subject_by_rule"] = "2sh+"
                                        datapoint["group"] = "obligation statement"
                                        datapoint["rule"] = "obligation periphrastic construction/plus"
                                        datapoint["rule_order"] = "168"
                                        datapoint["applied_rule"] += "6.3 obligation(plus) was applied, "
                                        # nichts getroffen
                                    elif re_morphmatch(morphs, morph="^가시$", pos="^V$") and re_morphmatch(morphs, morph="^(어야하|면되)$", pos="^X$"): 
                                        datapoint["subject_by_rule"] = "2sh+"
                                        datapoint["group"] = "obligation statement"
                                        datapoint["rule"] = "obligation periphrastic construction/plus"
                                        datapoint["rule_order"] = "169"
                                        datapoint["applied_rule"] += "6.4 obligation(plus) was applied, "
                                        # coverage really low 동사 하나만 cover하기 때문, 3 cases
                                    elif re_morphmatch(morphs, morph="^으면하$", pos="^X$") and datapoint["zp_hono"] == "plus": 
                                        datapoint["subject_by_rule"] = "2sh+" 
                                        datapoint["group"] = "want statement"
                                        datapoint["rule"] = "want periphrastic construction/plus"
                                        datapoint["rule_order"] = "170"
                                        datapoint["applied_rule"] += "6.5 want(plus) was applied, "
                                        # coverage ok
                                    elif re_morphmatch(morphs, morph="^(으면안되|면안되)$", pos="^X$") and datapoint["zp_mood"] == "ques":
                                        datapoint["subject_by_rule"] = "1sh+"
                                        datapoint["group"] = "permission"
                                        datapoint["rule"] = "permission periphrastic construction"
                                        datapoint["rule_order"] = "171"
                                        datapoint["applied_rule"] += "6.6 permission was applied, "
                                        # coverage low, 3 cases
                                    elif re_morphmatch(morphs, morph="^(으면안되|면안되)$", pos="^X$") and datapoint["zp_mood"] == "decl":
                                        datapoint["subject_by_rule"] = "2sh+"
                                        datapoint["group"] = "prohibition"
                                        datapoint["rule"] = "prohibition periphrastic construction"
                                        datapoint["rule_order"] = "172"
                                        datapoint["applied_rule"] += "6.7 prohibition was applied, "
                                        # coverage low, one case off about 3rd. person
                                    elif re_morphmatch(morphs, morph="^야하$", pos="^X$") and re_morphmatch(morphs, morph="^(ㅂ니다|아요)$", pos="^E$"): 
                                        datapoint["subject_by_rule"] = "2sh+"
                                        datapoint["group"] = "obligation statement"
                                        datapoint["rule"] = "obligation periphrastic construction"
                                        datapoint["rule_order"] = "173"
                                        datapoint["applied_rule"] += "6.8 obligation was applied, "
                                        # coverage ok, some 1sh+ lower the precision, hard even for human
                                        
                                    else: 
                                        datapoint["subject_by_rule"] = "N/A" 

                                    if datapoint["subject_by_rule"] == "N/A":

#                                         if re_morphmatch(morphs, morph="^(아야겠|어야겠)$", pos="^X$") and datapoint["zp_mood"] == "decl": 
#                                             datapoint["subject_by_rule"] = "1sh+"
#                                             datapoint["group"] = "want statement"
#                                             datapoint["rule"] = "want periphrastic construction"
#                                             datapoint["applied_rule"] += "7.1 want was applied, "
                                        if re_morphmatch(morphs, morph="^(아야겠|어야겠)$", pos="^X$") and datapoint["verb_syn"] == "clau": 
                                            datapoint["subject_by_rule"] = "1sh+"
                                            datapoint["group"] = "want statement"
                                            datapoint["rule"] = "want periphrastic construction"
                                            datapoint["rule_order"] = "174"
                                            datapoint["applied_rule"] += "7.1 want was applied, "
                                            # nichts getroffen
                                        elif re_morphmatch(morphs, morph="^(으면하|ㄹ까하)$", pos="^X$") and datapoint["zp_mood"] == "decl":
                                            datapoint["subject_by_rule"] = "1sh+" 
                                            datapoint["group"] = "want statement"
                                            datapoint["rule"] = "want periphrastic construction"
                                            datapoint["rule_order"] = "175"
                                            datapoint["applied_rule"] += "7.2 want was applied, "
                                            # coverage low, 8 cases
                                        elif re_morphmatch(morphs, morph="^(을뿐이|ㄹ뿐이|는것뿐이|ㄴ것뿐이)$", pos="^X$") and datapoint["zp_mood"] == "decl":
                                            datapoint["subject_by_rule"] = "1sh+"  
                                            datapoint["group"] = "experience"
                                            datapoint["rule"] = "judgement periphrastic construction"
                                            datapoint["rule_order"] = "176"
                                            datapoint["applied_rule"] += "7.3 judgement was applied, "
                                            # coverage really low, 2 cases, one off
                                        else: 
                                             datapoint["subject_by_rule"] = "N/A" 

                                        if datapoint["subject_by_rule"] == "N/A":
#                                             if re_morphmatch(morphs, morph="^(면되|으면되|기만하면되|아도되|어도되|아가|도되|지않아도되|아야하|어야하|여야하|야하|어야할것이|아야할것이|어야할것같|아야되|어야되|야되|아야죠|어야죠|야지요|으면안되|면안되)$", pos="^(X|E)$") and datapoint["zp_hono"] == "plus":
#                                                 datapoint["subject_by_rule"] = "2sh+"
#                                                 datapoint["group"] = "permission"
#                                                 datapoint["rule"] = "permission periphrastic construction/plus"
#                                                 datapoint["applied_rule"] += "8.1 permission(plus) was applied, "
#                                                 # coverage and precision very high
                                            if re_morphmatch(morphs, morph="^(아보|어보|보|가보|해보|써보|타보|찾아보|재보|맛보)$", pos="^(X|V)$") and datapoint["zp_mood"] == "decl":
                                                datapoint["subject_by_rule"] = "1sh+"
                                                datapoint["group"] = "try"
                                                datapoint["rule"] = "modality auxiliary verb"
                                                datapoint["rule_order"] = "177"
                                                datapoint["applied_rule"] += "8.1 try was applied, "
                                                # coverage ok, two cases off
                                            elif re_morphmatch(morphs, morph="^(아보|어보|보|가보|해보|써보|타보|찾아보|재보|맛보)$", pos="^X$") and datapoint["zp_mood"] == "ques":
                                                datapoint["subject_by_rule"] = "2sh+"
                                                datapoint["group"] = "try"
                                                datapoint["rule"] = "modality auxiliary verb"
                                                datapoint["rule_order"] = "178"
                                                datapoint["applied_rule"] += "8.2 try was applied, "
                                                # coverage really low, one case
#                                             elif re_morphmatch(morphs, morph="^(면되|으면되|기만하면되|아도되|어도되|아가|도되|지않아도되|아야하|어야하|여야하|야하|어야할것이|아야할것이|어야할것같|아야되|어야되|야되|아야죠|어야죠|야지요|으면안되|면안되)$", pos="^(X|E)$") and datapoint["zp_mood"] == "decl":
#                                                 datapoint["subject_by_rule"] = "2sh+"
#                                                 datapoint["group"] = "permission"
#                                                 datapoint["rule"] = "permission periphrastic construction"
#                                                 datapoint["applied_rule"] += "8.4 permission was applied, "
#                                                 # precision not that high (사람도 context 없이 어려움)
                                            elif re_morphmatch(morphs, morph="^(어야|아야|야하)$", pos="^X$") and datapoint["zp_mood"] == "decl": 
                                                datapoint["subject_by_rule"] = "1sh+"
                                                datapoint["group"] = "want statement"
                                                datapoint["rule"] = "want periphrastic construction"
                                                datapoint["rule_order"] = "179"
                                                datapoint["applied_rule"] += "8.3 want was applied, "
                                                # coverage really low, one case
                                            elif re_morphmatch(morphs, morph="^(면되|으면되|기만하면되|아도되|어도되|아가|도되|지않아도되|아야하|어야하|여야하|야하|어야할것이|아야할것이|어야할것같|아야되|어야되|야되|아야죠|어야죠|야지요|으면안되|면안되)$", pos="^(X|E)$") and datapoint["zp_mood"] == "decl":
                                                datapoint["subject_by_rule"] = "2sh+"
                                                datapoint["group"] = "permission"
                                                datapoint["rule"] = "permission periphrastic construction"
                                                datapoint["rule_order"] = "180"
                                                datapoint["applied_rule"] += "8.4 permission was applied, "
                                                # precision not that high (사람도 context 없이 어려움)/ 도메인 특성상 상대방에게 permission을 말하는 경우가 더 많을 것 같아서 2인칭으로
                                                # 1인칭 30, 2인칭 69 
                                            elif re_morphmatch(morphs, morph="^(면되|으면되|기만하면되|아도되|어도되|아가|도되|지않아도되|아야하|어야하|여야하|야하|어야할것이|아야할것이|어야할것같|아야되|어야되|야되|아야죠|어야죠|야지요|으면안되|면안되)$", pos="^X$") and datapoint["zp_mood"] == "ques":
                                                datapoint["subject_by_rule"] = "1sh+"
                                                datapoint["group"] = "permission"
                                                datapoint["rule"] = "permission periphrastic construction"
                                                datapoint["rule_order"] = "181"
                                                datapoint["applied_rule"] += "8.5 permission was applied, "
                                               # coverage and precision quite high
                                                
                                            else: 
                                                 datapoint["subject_by_rule"] = "N/A" 

                                            if datapoint["subject_by_rule"] == "N/A":
                                                if re_morphmatch(morphs, morph="^(ㄹ수있|을수있|ㄹ수는있|을수는있|ㄹ수도있|을수도있|ㄹ수가있|을수가있|ㄹ수없|ㄹ수도없|을수없|ㄹ수는없|을수는없|ㄹ수밖에없|을수밖에없|ㄴ수밖에없|는수밖에없|ㄹ수가없|을수가없|ㄹ리가없|을리가없|ㄹ필요는없|을필요는없|ㄹ필요가없|ㄹ필요없|지않아도되|을필요없|ㄹ필요가있|을필요가있|ㄹ뻔하|을뻔하)$", pos="^X$") and datapoint["zp_hono"] == "plus":
                                                    datapoint["subject_by_rule"] = "2sh+"
                                                    datapoint["group"] = "possibility"
                                                    datapoint["rule"] = "possibility periphrastic construction/plus"
                                                    datapoint["rule_order"] = "182"
                                                    datapoint["applied_rule"] += "9.1 possibility(plus,total) was applied, "
                                                    # coverage and precision high
                                                elif re_morphmatch(morphs, morph="^(나하|ㄹ뻔하|을뻔하)$", pos="^X$") and datapoint["zp_mood"] == "decl":
                                                    datapoint["subject_by_rule"] = "1sh+"
                                                    datapoint["group"] = "possibility"
                                                    datapoint["rule"] = "possibility periphrastic construction"
                                                    datapoint["rule_order"] = "183"
                                                    datapoint["applied_rule"] += "9.2.1 possibility was applied, "
                                                    # coverage low, 4 cases
                                                elif re_morphmatch(morphs, morph="^(ㄹ수있|을수있|ㄹ수는있|을수는있|ㄹ수도있|을수도있|ㄹ수가있|을수가있)$", pos="^X$") and re_morphmatch(morphs, morph="^(겠)$", pos="^E$") and datapoint["zp_mood"] == "decl":
                                                    datapoint["subject_by_rule"] = "1sh+"
                                                    datapoint["group"] = "possibility"
                                                    datapoint["rule"] = "possibility periphrastic construction"
                                                    datapoint["rule_order"] = "184"
                                                    datapoint["applied_rule"] += "9.2.2 possibility was applied, "
                                                    # nichts getroffen
                                                elif re_morphmatch(morphs, morph="^(ㄹ수있|을수있|ㄹ수는있|을수는있|ㄹ수도있|을수도있|ㄹ수가있|을수가있)$", pos="^X$") and datapoint["zp_mood"] == "decl":
                                                    datapoint["subject_by_rule"] = "2sh+"
                                                    datapoint["group"] = "possibility"
                                                    datapoint["rule"] = "possibility periphrastic construction"
                                                    datapoint["rule_order"] = "185"
                                                    datapoint["applied_rule"] += "9.2.3 possibility was applied, "
                                                    # coverage ok
                                                elif re_morphmatch(morphs, morph="^(ㄹ수없|ㄹ수도없|을수없|ㄹ수는없|을수는없|ㄹ수밖에없|을수밖에없|ㄴ수밖에없|는수밖에없|ㄹ수가없|을수가없|ㄹ리가없|을리가없)$", pos="^X$") and datapoint["zp_mood"] == "decl":
                                                    datapoint["subject_by_rule"] = "1sh+"
                                                    datapoint["group"] = "impossibility"
                                                    datapoint["rule"] = "impossibility periphrastic construction"
                                                    datapoint["rule_order"] = "186"
                                                    datapoint["applied_rule"] += "9.3 impossibility was applied, "
                                                    # coverage ok, some cases are off because of the ambiguity between first and second person, hard even for human
                                                elif re_morphmatch(morphs, morph="^(ㄹ수없|ㄹ수도없|을수없|ㄹ수는없|을수는없|ㄹ수밖에없|을수밖에없|ㄴ수밖에없|는수밖에없|ㄹ수가없|을수가없|ㄹ리가없|을리가없)$", pos="^X$") and datapoint["zp_mood"] == "decl":
                                                    datapoint["subject_by_rule"] = "1sh+"
                                                    datapoint["group"] = "unnecessary"
                                                    datapoint["rule"] = "unnecessary periphrastic construction"
                                                    datapoint["rule_order"] = "187"
                                                    datapoint["applied_rule"] += "9.4 Unnecessary was applied, "
                                                    # nichts getroffen 
                                                elif re_morphmatch(morphs, morph="^(ㄹ수있을것같|을수있을것같|ㄹ수있다는것이|을수있다는것이|ㄹ수있을것이|을수있을것이)$", pos="^X$") and datapoint["zp_hono"] == "plus":
                                                    datapoint["subject_by_rule"] = "2sh+"
                                                    datapoint["group"] = "possibility"
                                                    datapoint["rule"] = "possibility periphrastic construction"
                                                    datapoint["rule_order"] = "188"
                                                    datapoint["applied_rule"] += "9.5.1 possibility+guess(plus) was applied, "
                                                    # coverage ok
                                                elif re_morphmatch(morphs, morph="^(ㄹ수있을것같|을수있을것같|ㄹ수있다는것이|을수있다는것이|ㄹ수있을것이|을수있을것이)$", pos="^X$") and datapoint["zp_mood"] == "decl":
                                                    datapoint["subject_by_rule"] = "2sh+"
                                                    datapoint["group"] = "possibility"
                                                    datapoint["rule"] = "possibility periphrastic construction"
                                                    datapoint["rule_order"] = "189"
                                                    datapoint["applied_rule"] += "9.5.2 possibility+guess was applied, "
                                                    # coverage and precision low, 9 cases
                                                elif re_morphmatch(morphs, morph="^(ㄹ수있|을수있|ㄹ수는있|을수는있|ㄹ수도있|을수도있|ㄹ수가있|을수가있|ㄹ수없|ㄹ수도없|을수없|ㄹ수는없|을수는없|ㄹ수밖에없|을수밖에없|ㄴ수밖에없|는수밖에없|ㄹ수가없|을수가없|ㄹ리가없|을리가없|ㄹ필요는없|을필요는없|ㄹ필요가없|ㄹ필요없|을필요없|지않아도되|ㄹ필요가있|을필요가있|ㄹ뻔하|을뻔하)$", pos="^X$") and datapoint["zp_mood"] == "ques":
                                                    datapoint["subject_by_rule"] = "1sh+"
                                                    datapoint["group"] = "possibility"
                                                    datapoint["rule"] = "possibility periphrastic construction/plus"
                                                    datapoint["rule_order"] = "190"
                                                    datapoint["applied_rule"] += "9.6 possibility was applied, "
                                                    # coverage quite high, some cases are off because of the ambiguity between first and second person, hard even for human
                                                    
#                                                 if re_morphmatch(morphs, morph="^(을수있|을수가있|을수도있|을수는있|ㄹ수는있|ㄹ수있을것같|ㄹ수가있|ㄹ수도있|ㄹ수는있|ㄹ수있다는것이|을수있을것이|ㄹ수있을것이|을수없|을수가없|을수는없|ㄹ수없|ㄹ수가없|을수가없|ㄹ수는없|는수밖에없|ㄹ수밖에없|을뻔하|ㄹ뻔하)$", pos="^X$") and datapoint["zp_hono"] == "plus":
#                                                     datapoint["subject_by_rule"] = "2sh+"
#                                                     datapoint["group"] = "possibility"
#                                                     datapoint["rule"] = "possibility periphrastic construction/plus"
#                                                     datapoint["applied_rule"] += "9.1 possibility(plus) was applied, "
#                                                 elif re_morphmatch(morphs, morph="^(을뻔하|ㄹ뻔하)$", pos="^X$") and datapoint["zp_mood"] == "decl":
#                                                     datapoint["subject_by_rule"] = "1sh+"
#                                                     datapoint["group"] = "possibility"
#                                                     datapoint["rule"] = "possibility periphrastic construction"
#                                                     datapoint["applied_rule"] += "9.2 possibility was applied, "
#                                                 elif re_morphmatch(morphs, morph="^(을수있|을수도있|을수는있|ㄹ수는있|ㄹ수있을것같|ㄹ수도있|ㄹ수는있|ㄹ수있다는것이|을수있을것이|ㄹ수있을것이|ㄹ수있을것이|을수없|을수는없|ㄹ수없|ㄹ수는없|는수밖에없|ㄹ수밖에없|을뻔하|ㄹ뻔하)$", pos="^X$") and datapoint["zp_mood"] == "decl":
#                                                     datapoint["subject_by_rule"] = "2sh+"
#                                                     datapoint["group"] = "possibility"
#                                                     datapoint["rule"] = "possibility periphrastic construction"
#                                                     datapoint["applied_rule"] += "9.3 possibility was applied, "
#                                                     # precision not that high (사람도 context 없이 어려움), 을수가/ㄹ수가 를 여기 rule에서 제외해보기 1sh+의 특징일 수도
#                                                 elif re_morphmatch(morphs, morph="^(을수있|을수가있|을수도있|을수는있|ㄹ수는있|ㄹ수있을것같|ㄹ수가있|ㄹ수도있|ㄹ수는있|ㄹ수있다는것이|을수있을것이|ㄹ수있을것이|ㄹ수있을것이|을수없|을수가없|을수는없|ㄹ수없|ㄹ수가없|을수가없|ㄹ수는없|는수밖에없|ㄹ수밖에없)$", pos="^X$") and datapoint["zp_mood"] == "ques":
#                                                     datapoint["subject_by_rule"] = "1sh+"
#                                                     datapoint["group"] = "possibility"
#                                                     datapoint["rule"] = "possibility periphrastic construction"
#                                                     datapoint["applied_rule"] += "9.4 possibility was applied, "
#                                                 elif re_morphmatch(morphs, morph="^(을수있|을수가있|을수도있|을수는있|ㄹ수는있|ㄹ수있을것같|ㄹ수가있|ㄹ수도있|ㄹ수는있|ㄹ수있다는것이|을수있을것이|ㄹ수있을것이|ㄹ수있을것이|을수없|을수가없|을수는없|ㄹ수없|ㄹ수가없|을수가없|ㄹ수는없|는수밖에없|ㄹ수밖에없)$", pos="^X$") and datapoint["zp_mood"] == "decl":
#                                                     datapoint["subject_by_rule"] = "1sh+"
#                                                     datapoint["group"] = "possibility"
#                                                     datapoint["rule"] = "possibility periphrastic construction"
#                                                     datapoint["applied_rule"] += "9.5 possibility was applied, "
                                                    
                                                else: 
                                                    datapoint["subject_by_rule"] = "N/A" 

                                                if datapoint["subject_by_rule"] == "N/A":
                                                    if re_morphmatch(morphs, morph="^(아주|어주|주|가져다주|찍어주|안해주|틀어주|도와주|넣어주|거슬러주)$", pos="^(X|V)$") and datapoint["zp_hono"] == "plus":
                                                        datapoint["subject_by_rule"] = "2sh+"
                                                        datapoint["group"] = "benefactive"
                                                        datapoint["rule"] = "modality auxiliary verb/plus"
                                                        datapoint["rule_order"] = "191"
                                                        datapoint["applied_rule"] += "10.1 benefactive(plus) was applied, "
                                                        # coverage and precision high
                                                    elif re_morphmatch(morphs, morph="^(아놓|어놓|열어놓|정해놓|걸어놓)$", pos="^(X|V)$") and datapoint["zp_hono"] == "plus":
                                                        datapoint["subject_by_rule"] = "2sh+"
                                                        datapoint["group"] = "subjectivication"
                                                        datapoint["rule"] = "aspect auxiliary verb/plus"
                                                        datapoint["rule_order"] = "192"
                                                        datapoint["applied_rule"] += "10.2 subjectivication(plus) was applied, "
                                                        # coverage low, two cases
                                                    elif "다놓" in datapoint["verb"] and datapoint["zp_hono"] == "plus":
                                                        datapoint["subject_by_rule"] = "2sh+" 
                                                        datapoint["group"] = "subjectivication"
                                                        datapoint["rule"] = "aspect auxiliary verb/plus"
                                                        datapoint["rule_order"] = "193"
                                                        datapoint["applied_rule"] += "10.3 subjectivication(plus) was applied, "
                                                        # nichts getroffen
                                                    elif re_morphmatch(morphs, morph="^(아두|어두|걸어두|세워두|놓아두|맡겨두|넣어두)$", pos="^(X|V)$") and datapoint["zp_hono"] == "plus":
                                                        datapoint["subject_by_rule"] = "2sh+"
                                                        datapoint["group"] = "subjectivication"
                                                        datapoint["rule"] = "aspect auxiliary verb/plus"
                                                        datapoint["rule_order"] = "194"
                                                        datapoint["applied_rule"] += "10.4 subjectivication(plus) was applied, "
                                                        # coverage low, 3 cases
                                                    elif re_morphmatch(morphs, morph="^(아놓|어놓|열어놓|정해놓|걸어놓)$", pos="^(X|V)$") and datapoint["zp_mood"] == "ques":
                                                        datapoint["subject_by_rule"] = "2sh+"
                                                        datapoint["group"] = "subjectivication"
                                                        datapoint["rule"] = "aspect auxiliary verb"
                                                        datapoint["rule_order"] = "195"
                                                        datapoint["applied_rule"] += "10.5 subjectivication(plus) was applied, "
                                                        # coverage very low, 1 case
                                                    elif "다놓" in datapoint["verb"] and datapoint["zp_mood"] == "ques":
                                                        datapoint["subject_by_rule"] = "2sh+" 
                                                        datapoint["group"] = "subjectivication"
                                                        datapoint["rule"] = "aspect auxiliary verb"
                                                        datapoint["rule_order"] = "196"
                                                        datapoint["applied_rule"] += "10.6 subjectivication was applied, "
                                                        # nichts getroffen
                                                    elif re_morphmatch(morphs, morph="^(아두|어두|걸어두|세워두|놓아두|맡겨두|넣어두)$", pos="^(X|V)$") and datapoint["zp_mood"] == "ques":
                                                        datapoint["subject_by_rule"] = "2sh+"
                                                        datapoint["group"] = "subjectivication"
                                                        datapoint["rule"] = "aspect auxiliary verb"
                                                        datapoint["rule_order"] = "197"
                                                        datapoint["applied_rule"] += "10.7 subjectivication was applied, "
                                                        # coverage very low, one case, even off
                                                    elif re_morphmatch(morphs, morph="^(아주|어주|주|가져다주|찍어주|안해주|틀어주|도와주|넣어주|거슬러주)$", pos="^(X|V)$") and datapoint["zp_mood"] == "ques":
                                                        datapoint["subject_by_rule"] = "2sh+"
                                                        datapoint["group"] = "benefactive"
                                                        datapoint["rule"] = "modality auxiliary verb"
                                                        datapoint["rule_order"] = "198"
                                                        datapoint["applied_rule"] += "10.8 benefactive was applied, "
                                                        # coverage ok, some cases are off because of the ambiguity between second person and general e.g. 기름은 넣어주나요 ?
                                                    elif re_morphmatch(morphs, morph="^(아주|어주|주|가져다주|찍어주)$", pos="^(X|V)$") and re_morphmatch(morphs, morph="^어야하$", pos="^X$"):
                                                        datapoint["subject_by_rule"] = "2sh+"
                                                        datapoint["group"] = "obligation statement"
                                                        datapoint["rule"] = "obligation periphrastic construction"
                                                        datapoint["rule_order"] = "199"
                                                        datapoint["applied_rule"] += "10.9 benefactive was applied, "
                                                        # nichts getroffen
                                                    elif re_morphmatch(morphs, morph="^(아보|어보|가보|해보|써보|타보|찾아보|재보)$", pos="^(X|V)$") and datapoint["zp_mood"] == "ques":
                                                        datapoint["subject_by_rule"] = "2sh+"
                                                        datapoint["group"] = "try"
                                                        datapoint["rule"] = "modality auxiliary verb"
                                                        datapoint["rule_order"] = "200"
                                                        datapoint["applied_rule"] += "10.10 try was applied, "
                                                        # nichts getroffen
                                                    else: 
                                                        datapoint["subject_by_rule"] = "N/A" 

                                                    if datapoint["subject_by_rule"] == "N/A":
                                                        if re_morphmatch(morphs, morph="^겠$", pos="^E$") and datapoint["zp_hono"] == "plus":
                                                            datapoint["subject_by_rule"] = "2sh+" 
                                                            datapoint["group"] = "guess/future/will"
                                                            datapoint["rule"] = "guess/future/will infix"
                                                            datapoint["rule_order"] = "201"
                                                            datapoint["applied_rule"] += "11.1 guess/future/will(plus) was applied, "
                                                            # coverage ok
                                                        elif re_morphmatch(morphs, morph="^겠$", pos="^E$") and datapoint["zp_mood"] == "decl":
                                                            datapoint["subject_by_rule"] = "1sh+" 
                                                            datapoint["group"] = "guess/future/will"
                                                            datapoint["rule"] = "guess/future/will infix"
                                                            datapoint["rule_order"] = "202"
                                                            datapoint["applied_rule"] += "11.2 guess/future/will was applied, "
                                                            # coverage ok, some cases off because of the issue with the definition of subject e.g. 정말 매력적이겠네요 . 
                                                        elif "겠습니까" in datapoint["verb"] and datapoint["zp_mood"] == "ques":
                                                            datapoint["subject_by_rule"] = "2sh+"
                                                            datapoint["group"] = "will"
                                                            datapoint["rule"] = "will infix"
                                                            datapoint["rule_order"] = "203"
                                                            datapoint["applied_rule"] += "11.3 will was applied, "
                                                            # coverage low, 4 cases, even one off
                                                        elif re_morphmatch(morphs, morph="^(아놓|어놓|열어놓|정해놓|걸어놓)$", pos="^(X|V)$") and datapoint["zp_mood"] == "decl":
                                                            datapoint["subject_by_rule"] = "1sh+"
                                                            datapoint["group"] = "subjectivication"
                                                            datapoint["rule"] = "aspect auxiliary verb"
                                                            datapoint["rule_order"] = "204"
                                                            datapoint["applied_rule"] += "11.4 subjectivication was applied, "
                                                            # coverage ok, some cases off because of the general subject e.g. 네, 입구 쪽에 테이블을 쭉 펼쳐놨어요 .
                                                        elif re_morphmatch(morphs, morph="^(아두|어두|걸어두|세워두|놓아두|맡겨두|넣어두)$", pos="^(X|V)$") and datapoint["zp_mood"] == "decl":
                                                            datapoint["subject_by_rule"] = "1sh+"
                                                            datapoint["group"] = "subjectivication"
                                                            datapoint["rule"] = "aspect auxiliary verb"
                                                            datapoint["rule_order"] = "205"
                                                            datapoint["applied_rule"] += "11.5 subjectivication was applied, "
                                                            # coverage ok
                                                        elif re_morphmatch(morphs, morph="^(아주|어주|주|가져다주|찍어주|안해주|틀어주|도와주|넣어주|거슬러주)$", pos="^(X|V)$") and datapoint["zp_mood"] == "decl":
                                                            datapoint["subject_by_rule"] = "1sh+"
                                                            datapoint["group"] = "benefactive"
                                                            datapoint["rule"] = "modality auxiliary verb"
                                                            datapoint["rule_order"] = "206"
                                                            datapoint["applied_rule"] += "11.6 guess was applied, "
                                                            # coverage low, precision very low
                                                        elif re_morphmatch(morphs, morph="^어다$", pos="^E$") and re_morphmatch(morphs, morph="^놓$", pos="^V$"):
                                                            datapoint["subject_by_rule"] = "1sh+"
                                                            datapoint["group"] = "subjectivication"
                                                            datapoint["rule"] = "aspect auxiliary verb"
                                                            datapoint["rule_order"] = "207"
                                                            datapoint["applied_rule"] += "11.7 subjectivication was applied, "
                                                            # nichts getroffen
                                                        elif re_morphmatch(morphs, morph="^(을것같|ㄹ것같|는것같|ㄴ것같|은것같|은듯하|ㄴ거같|을거같|ㄹ거같|는거같|던것같)$", pos="^X$") and datapoint["zp_mood"] == "decl":
                                                            datapoint["subject_by_rule"] = "1sh+"
                                                            datapoint["group"] = "guess"
                                                            datapoint["rule"] = "guess periphrastic construction"
                                                            datapoint["rule_order"] = "208"
                                                            datapoint["applied_rule"] += "11.8 guess was applied, "
                                                            # coverage는 높지만 여전히 subject가 무엇인지의 정의에 따라 정답률이 달라질 수 있음
                                                                                                                        
                                                        else: 
                                                            datapoint["subject_by_rule"] = "N/A" 
                                                        
                                                        exclude_list = ["dobj", "modi", "scom", "subj"]
                                                        if datapoint["subject_by_rule"] == "N/A":
                                                            if re_morphmatch(morphs, morph="^고있$", pos="^X$") and datapoint["zp_mood"] == "decl":
                                                                datapoint["subject_by_rule"] = "1sh+" 
                                                                datapoint["group"] = "progress"
                                                                datapoint["rule"] = "progress aspect"
                                                                datapoint["rule_order"] = "209"
                                                                datapoint["applied_rule"] += "12.1 progress was applied, "
                                                                # coverage quite high, but some off cases because of general subejct and third person
                                                            elif re_morphmatch(morphs, morph="^고있$", pos="^X$") and datapoint["zp_mood"] == "ques":
                                                                datapoint["subject_by_rule"] = "2sh+" 
                                                                datapoint["group"] = "progress"
                                                                datapoint["rule"] = "progress aspect"
                                                                datapoint["rule_order"] = "210"
                                                                datapoint["applied_rule"] += "12.2 progress was applied, "
                                                                # coverage ok, but some off cases because of general subejct and third person
                                                            elif re_morphmatch(morphs, morph="^게되$", pos="^X$") and datapoint["zp_mood"] == "decl":
                                                                datapoint["subject_by_rule"] = "1sh+" 
                                                                datapoint["group"] = "passive"
                                                                datapoint["rule"] = "passive aspect"
                                                                datapoint["rule_order"] = "211"
                                                                datapoint["applied_rule"] += "12.3 passive was applied, "
                                                                # coverage ok
                                                            elif datapoint["zp_hono"] == "plus":
                                                                datapoint["subject_by_rule"] = "2sh+" 
                                                                datapoint["group"] = "honorification"
                                                                datapoint["rule"] = "hono total"
                                                                datapoint["rule_order"] = "212"
                                                                datapoint["applied_rule"] += "12.4 hono(at the end) was applied, "
#                                                             elif datapoint["verb_syn"] in exclude_list:
#                                                                 datapoint["subject_by_rule"] = "none"
#                                                                 datapoint["group"] = "exclude_verb_syn"
#                                                                 datapoint["rule"] = "exclude"
#                                                                 datapoint["rule_order"] = "213"
#                                                                 datapoint["applied_rule"] += "exclude was applied, " 
                                                            else: 
                                                                datapoint["subject_by_rule"] = "N/A" 
                                                            


        datapoints.append(datapoint) 
        
        print("{:.2f}%\r".format(100*i/maxn), end="")  
        


In [7]:
df2 = pd.DataFrame(datapoints) 

exclude_items = ['is_ana', 'verb_roman', 'zp_speaker','sent_morphs', "pos"]
# 'is_ana' 왜인지 inter인 것의 value가 1

#rearrange columns
columns = ['etri_sent',
           'sent',
           'verb',
           'applied_rule',
           'subject_by_rule',
           'zero_referents',
           'antecedent',
           'group',
           'rule_order',
           'rule',
           'connective',
           'base_morpheme',
           'base_morpheme_second',
           'connective_true',
           'zp_hono',
           'zp_mood',
           'sent_dep',
           'sent_morphs',
           'verb_roman',
           'zp_markable_ID',
           'is_ana',
           'zero_type',
           'zp_speaker',
           'tense_zp',
           'verb_syn',
           'verb_head']

columns = [c for c in columns if not c in exclude_items]

for c in list(df2):
    if not c in columns+exclude_items:
        print("Warning: did you miss column '{}'?".format(c))
        
        
df2 = df2[columns]
#df2.to_csv("df2_connective_test_230323.csv", encoding = "utf-8-sig") 

#print(df2.shape) # (17266, 13)
#df.head()
freq = df2.groupby(['group']).count() 
#print(freq)

print(df2.shape) # (17266, 20)

df2

(17266, 22)


,etri_sent,sent,verb,applied_rule,subject_by_rule,zero_referents,antecedent,group,rule_order,rule,connective,base_morpheme,base_morpheme_second,connective_true,zp_hono,zp_mood,sent_dep,zp_markable_ID,zero_type,tense_zp,verb_syn,verb_head
0,121,"산은 너무 멀고, 이쪽 길로 가면 숲을 지나서 돌아올수있어요 .",지나서,,N/A,2sh+,,NaN,NaN,NaN,class_A,지나,서,1,N/A,N/A,"[subj, modi, clau, modi, modi, clau, dobj, cla...",markable_5010,extra,N/A,clau,9
1,253,갇혀있으니 너무 무서워요 .,무서워요,"5.13.2 feeling was applied,",1sh+,1sh+,,experience,152,feeling predicate,N/A,무섭,N/A,0,N/A,decl,"[clau, modi, root, punc]",markable_5131,extra,pres,root,0
2,396,"현재 치료 중이 아니시면 괜찮으시구요, 귀마개를 착용하시는 편이 더 좋겠네요 .","괜찮으시구요,","clau(plus, at the end) was applied,",2sh+,2sh+,,clau,23,clau_plus,N/A,괜찮,N/A,1,plus,N/A,"[modi, modi, scom, clau, clau, dobj, modi, sub...",markable_5252,extra,N/A,clau,7
3,586,그럼 담요를 준비해드릴까요 ?,준비해드릴까요,"3.1.2 suggestory was applied,",1sh+,1sh+,,suggestory formula,92,suggestory verbal suffix,N/A,준비하,N/A,0,N/A,ques,"[modi, dobj, root, punc]",markable_5373,extra,pres,root,0
4,784,준비해드리겠습니다 .,준비해드리겠습니다,"2.2 want was applied,",1sh+,1sh+,,want statement,64,want verbal suffix,N/A,준비하,N/A,0,N/A,decl,"[root, punc]",markable_5494,extra,futu,root,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17261,8455,불편을 끼쳐드려 죄송합니다 .,끼쳐드려,"clau(various_deuli) was applied,",1sh+,1ph+,,clau,36,clau_various,class_A,끼치,어드리,1,N/A,N/A,"[dobj, clau, root, punc]",markable_5080,extra,N/A,clau,3
17262,8458,영화채널도 있는지 궁금합니다 .,궁금합니다,"5.4 cognition was applied,",1sh+,1sh+,,experience,142,cognition predicate,N/A,궁금하,N/A,0,N/A,decl,"[subj, clau, root, punc]",markable_5083,extra,pres,root,0
17263,8460,유료채널을 보려면 어떻게 해야합니까 ?,보려면,,N/A,1sh+,,NaN,NaN,NaN,class_A,보,려면,1,N/A,N/A,"[dobj, clau, modi, root, punc]",markable_5084,extra,N/A,clau,4
17264,8456,룸에 비치된 TV로 외국 채널을 볼수있을까요 ?,볼수있을까요,"2.5 permission was applied,",1sh+,1sh+,,permission,70,permission periphrastic construction,N/A,보,N/A,0,N/A,ques,"[modi, modi, modi, modi, dobj, root, punc]",markable_5081,extra,futu,root,0


In [8]:
# 281023

#df2.to_csv("proposed_conjunction_change_281023.csv", encoding = "utf-8-sig")

In [9]:
# 1. exclude 제거
# 2. error_category 제거
# 3. subject error 찾아서 group1 final 만들기
# 4. group 2 적용
# 5. error analysis 만들기

# 1. exclude 제거

import pandas as pd
import numpy as np

# df2_exclude에 Filter 열 추가 
conditionlist = [
    (df2['subject_by_rule'] != 'N/A') & (df2['verb_syn'] == 'root'),
    (df2['subject_by_rule'] != 'N/A') & (df2['verb_syn'] != 'root'),
    (df2['subject_by_rule'] == 'N/A')] 
choicelist = ['A', 'B', 'C']

df2['Filter'] = np.select(conditionlist, choicelist, default='Not Specified')

df2.shape # (17266, 21)

(17266, 23)

In [12]:
# Filter 추가한 df2에서 각 Filter별로 제외돼야 하는 case 먼저 제외 -> df2_exclude 만들기

#exclude_A

exclude_A = pd.read_excel('Filter A_null_count_2_to_1_Error_27042022.xlsx') 
exclude_A_idx = exclude_A['idx']
exclude_A_idx_val = exclude_A_idx.values 
exclude_A_idx_val_list = exclude_A_idx_val.tolist() 

exclude_A_rest = pd.read_excel('Filter A_Error_rest_28042022.xlsx') 
exclude_A_rest_idx = exclude_A_rest['idx']
exclude_A_rest_idx_val = exclude_A_rest_idx.values 
exclude_A_rest_idx_val_list = exclude_A_rest_idx_val.tolist() # 86개의 제외될 predicate의 idx의 list

#exclude_B

exclude_B = pd.read_excel('Filter B_Error_28042022.xlsx') 
exclude_B_idx = exclude_B['idx']
exclude_B_idx_val = exclude_B_idx.values 
exclude_B_idx_val_list = exclude_B_idx_val.tolist() # 111개의 제외될 predicate의 idx의 list

#exclude_C

exclude_C = pd.read_excel('Filter C_Error_28042022.xlsx') 
exclude_C_idx = exclude_C['idx']
exclude_C_idx_val = exclude_C_idx.values 
exclude_C_idx_val_list = exclude_C_idx_val.tolist() # 647 

#exclude_add

exclude_add = pd.read_excel('case2A_B_C_02052022.xlsx', sheet_name = 'exclude') 
exclude_add_idx = exclude_add['idx']
exclude_add_idx_val = exclude_add_idx.values 
exclude_add_idx_val_list = exclude_add_idx_val.tolist()

#exclude_group2
exclude_group2 = pd.read_excel('rule_group2_analysis.xlsx', sheet_name = 'exclude') 
exclude_group2_idx = exclude_group2['idx']
exclude_group2_idx_val = exclude_group2_idx.values 
exclude_group2_idx_val_list = exclude_group2_idx_val.tolist() # 230522 더 추가함!!

# print(len(exclude_group2_idx_val_list)) # 217

# exclude_total
exclude_total = pd.read_csv('exclude_total_excel_06052022.csv') # exclude가 반영이 안됨!
exclude_total_idx = exclude_total['idx']
exclude_total_idx_val = exclude_total_idx.values 
exclude_total_idx_val_list = exclude_total_idx_val.tolist() # 1,152
#print(len(set(exclude_total_idx_val_list))) # 1,145

exclude_total_idx_list = exclude_A_idx_val_list + exclude_A_rest_idx_val_list + exclude_B_idx_val_list + exclude_C_idx_val_list + exclude_add_idx_val_list + exclude_total_idx_val_list + exclude_group2_idx_val_list
exclude_total_idx_list_set = set(exclude_total_idx_list)

print(len(exclude_total_idx_list)) # 2555
print(len(exclude_total_idx_list_set)) # 1384 

df2_exclude = df2[df2.index.isin(exclude_total_idx_list_set) == False]
df2_exclude.shape #(15883, 21) (17266-15883 = 1383)

# df2_exclude.to_csv("NRS1_implement_150224.csv", encoding = "utf-8-sig")


2555
1384


In [11]:
# exclude_total_idx_list_set index와 df2_exclude_True index의 difference를 찾아보기
df2_exclude_True = df2[df2.index.isin(exclude_total_idx_list_set) == True]
# print(df2_exclude_True.shape) # (1383, 21)

# 위의 seperate된 exclude 관련 파일을 하나로 모아서, df2_exclude_True의 index만 지닌 애들을 뽑아서 exclude_total_idx_list_set과 비교하면 
# 2개의 차이가 무엇인지도 알 수 있고, exclude에 대한 error 내기도 더 편함
# "exclude_error_compilation_130922.xlsx"

exclude_all = pd.read_excel("exclude_error_compilation_130922.xlsx", sheet_name = None, index_col=0)

exclude_all_df = pd.concat(exclude_all)

print(exclude_all_df.shape) #(2555, 4) duplicates 제거 해야 함
# exclude_all_df.to_csv("exclude_all_df_130922.csv", encoding = "utf-8-sig") 

# "exclude_all_df_dedupilicate_130922.csv" 1383개 1개의 차이가 뭔지 index 비교해보기 (굳이 필요 없음)

exclude_all_dedup_review_df = pd.read_csv("exclude_all_df_review_140922.csv")

exclude_all_dedup_review_df_idx = exclude_all_dedup_review_df.index.tolist()
print(len(exclude_all_dedup_review_df_idx)) # 1383




(2555, 3)
1383


In [12]:
# exclude_all_dedup_df"를 error를 기준으로 freq 찾고, 표로 만들고 예문 추가
error_freq = exclude_all_dedup_review_df['error'].value_counts() 
print(exclude_all_dedup_review_df['error'].size) # 1383 richtig

# error_freq.to_csv("error_frequence_140922.csv", encoding = "utf-8-sig") 

1383


In [13]:
# df2_exclude (15883) 우선 csv로 만들기! 
# df2_exclude.to_csv("df2_exclude_061022.csv", encoding = "utf-8-sig") 
# df2_exclude.to_csv("df2_exclude_230322.csv", encoding = "utf-8-sig") 

# 전체 15883
subject_by_rule_freq = df2_exclude['subject_by_rule'].value_counts() 
print(subject_by_rule_freq)


subject_by_rule_group_by_freq = df2_exclude.groupby(['subject_by_rule','antecedent']).count() 

# subject_by_rule_freq.to_csv("subject_by_rule_freq_180922.csv", encoding = "utf-8-sig") 
# subject_by_rule_group_by_freq.to_csv("subject_by_rule_group_by_freq_051022.csv", encoding = "utf-8-sig") 

2sh+    6739
1sh+    5386
N/A     2045
none    1669
1ph-      44
Name: subject_by_rule, dtype: int64


In [14]:
# https://wotres.tistory.com/entry/%EB%B6%84%EB%A5%98-%EC%84%B1%EB%8A%A5-%EC%B8%A1%EC%A0%95%ED%95%98%EB%8A%94%EB%B2%95-Accuracy-Precision-Recall-F1-score-ROC-AUC-in-python

# Precision (실제 양성수 (할당한 주어가 실제 맞는 경우)/ 양성이라고 판단한 수 (특정 주어라고 판단한 수)) 모델이 얼마나 정밀한가, 진짜 양성만을 잘 고르는가?
# TP / TP+FP

df2_zero_referent_count = df2_exclude.groupby(["subject_by_rule","zero_referents"])["zero_referents"].count()
df2_antecedent_count = df2_exclude.groupby(["subject_by_rule","antecedent"])["antecedent"].count()
# df2_zero_referent_antecedent_count = df2_exclude.groupby(["subject_by_rule","zero_referents","antecedent"])["zero_referents","antecedent"].count()


# df2_antecedent = df2_exclude["antecedent"].groupby(df2_exclude["subject_by_rule"])
# df2_antecedent_dict = dict(list(df2_antecedent))

# df2_antecedent

# df2_zero_referent_count.to_csv("df2_zero_referent_count_051022.csv", encoding = "utf-8-sig") 
# df2_antecedent_count.to_csv("df2_antecedent_count_051022.csv", encoding = "utf-8-sig") 



In [15]:
# df2_exclude에서 바로 규칙 정확도 계산해보기 180922 이걸로 다시 표 만들기 
# Accuracy: 전체 샘플 중 맞게 예측한 샘플 수 (TP+TN / Total)

# 1. simple string match (case1)

df2_case1 = df2_exclude[df2_exclude.subject_by_rule == df2_exclude.zero_referents] 
df2_case1_idx = df2_case1.index.tolist()
print(len(df2_case1_idx)) # 11422

print(df2_case1.shape) # (11422, 21)

# df2_case1.to_csv("df2_case1_180922.csv", encoding = "utf-8-sig") 

# 2.  singular <-> plural, person match (case2) 

# df2_not_case1 = df2_total_group1_for_analysis[df2_total_group1_for_analysis.subject_by_rule != df2_total_group1_for_analysis.zero_referents] 
# print(df2_not_case1.shape) # (4470, 23) 15892 - 11422 = 4470 richtig

# df2_not_case1_idx = df2_not_case1.index.tolist()
# # print(len(df2_not_case1_idx)) # 4470

df2_first_person = df2_exclude[df2_exclude['subject_by_rule'] == '1sh+'] 
df2_second_person = df2_exclude[df2_exclude['subject_by_rule'] == '2sh+'] 

print(df2_first_person.shape) #(5382, 21)
print(df2_second_person.shape) #(6739, 21)

df2_first_person_match = df2_first_person[(df2_first_person['zero_referents'] == '1ph-') | (df2_first_person['zero_referents'] == '1ph+')] 
df2_second_person_match = df2_second_person[(df2_second_person['zero_referents'] == '2ph+') | (df2_second_person['zero_referents'] == '2sh-')]

print(df2_first_person_match.shape) # (195, 21)
print(df2_second_person_match.shape) # (39, 21)

df2_first_person_match_idx = df2_first_person_match.index.tolist()
df2_second_person_match_idx = df2_second_person_match.index.tolist()

print(len(df2_first_person_match_idx)) # 195
print(len(df2_second_person_match_idx)) # 39

# df2_first_person_match.to_csv("df2_first_person_match_180922.csv", encoding = "utf-8-sig") 
# df2_second_person_match.to_csv("df2_second_person_match_180922.csv", encoding = "utf-8-sig") 

# 3. subject_by_rule과 antecedent 같은 경우 (case3) 

df2_referent_match_idx = df2_first_person_match_idx + df2_second_person_match_idx + df2_case1_idx
print(len(df2_referent_match_idx)) # 11656 (195+39+11422 = 11656 richtig)

df2_case3_check = df2_exclude[df2_exclude.index.isin(df2_referent_match_idx) == False]

print(df2_case3_check.shape) # (4227, 21) 15883-11656 = 4227 richtig
# df2_case3_check.to_csv("df2_case3_check_180922.csv", encoding = "utf-8-sig") 
# 여기에서 N/A none 3718 (2049+1669 = 3718 richtig)
# 실제로 체크해야 하는 case 509 richtig -> "df2_case3_check_filter_for_referent_180922.csv" (manuell 만듦)

# # df2_case3_check 에서 3개의 subcases 
# # 3.1 subject_by_rule "1sh+" & antecedent "{저}", "{저는}", "{저도}", "{저희가}", "{저희들도}", "{전}", "{제가}" (total: 131)
# # 3.2 subject_by_rule "1ph-" & antecedent "{우리}" (total: 1)
# # 3.3 subject_by_rule "2sh+" # no match (total: 0)

# "df2_case3_index_180922.xlsx" (총 134개의 index가 저장되어 있음)

df2_case3 = pd.read_excel('df2_case3_index_180922.xlsx') 
df2_case3_idx = df2_case3['idx']
df2_case3_idx_val = df2_case3_idx.values 
df2_case3_idx_val_list = df2_case3_idx_val.tolist() 

print(len(df2_case3_idx_val_list)) # 134 richtig


df2_case3_antecedent_match = df2_case3_check[df2_case3_check.index.isin(df2_case3_idx_val_list) == True]
print(df2_case3_antecedent_match.shape) # (134, 21) richtig

# df2_case3_antecedent_match.to_csv("df2_case3_antecedent_match_260922.csv", encoding = "utf-8-sig") 
referent_match_all_idx = df2_case1_idx + df2_first_person_match_idx + df2_second_person_match_idx + df2_case3_idx_val_list
print(len(referent_match_all_idx)) # 11790 richtig


11423
(11423, 23)
(5386, 23)
(6739, 23)
(195, 23)
(39, 23)
195
39
11657
(4226, 23)
134
(134, 23)
11791


In [16]:
# True Positive Case by class (적용된 rule 등에 대해서 분석할 때 용이하게 하기 위해서 311022)
# df2_exclude (15883, 21)

# 1. "subject_by_rule": 1ph-, "zero_referents": 1ph- (+ person matching)

class1_TP_string = df2_exclude[(df2_exclude['subject_by_rule'] == '1ph-') & (df2_exclude['zero_referents'] == '1ph-')] # (42, 21)
class1_TP_antecedent = df2_exclude[(df2_exclude['subject_by_rule'] == '1ph-') & (df2_exclude['antecedent'] == '우리')] # (1, 21)
class1_TP = pd.concat([class1_TP_string, class1_TP_antecedent]) # (43, 21) richtig


# 2. "subject_by_rule": 1sh+, "zero_referents": 1sh+ (+ person matching, antecedent matching, from false negative)

class2_TP_string = df2_exclude[(df2_exclude['subject_by_rule'] == '1sh+') & (df2_exclude['zero_referents'] == '1sh+')] # (4853, 21)

class2_antecedent_list = ["저", "저는", "저도", "저희가", "저희는", "저희들도", "전", "제가"]
class2 = df2_exclude[(df2_exclude['subject_by_rule'] == '1sh+')] # (5382, 21)
class2_TP_antecedent = df2_exclude[(df2_exclude['subject_by_rule'] == '1sh+') & df2_exclude['antecedent'].isin(class2_antecedent_list)] # (131, 21)
class2_TP_person_matching = class2[(class2['zero_referents'] == "1ph-") | (class2['zero_referents'] == "1ph+")] # (195, 21)
class2_FN_idx_list = [2513, 15475, 15484]
class2_TP_from_FN = class2[class2.index.isin(class2_FN_idx_list) == True] # (3, 21)

class2_TP = pd.concat([class2_TP_string, class2_TP_antecedent, class2_TP_person_matching, class2_TP_from_FN]) # (5182, 21) richtig

# 3. "subject_by_rule": 2sh+, "zero_referents": 2sh+ (+ person matching, from false negative)

class3_TP_string = df2_exclude[(df2_exclude['subject_by_rule'] == '2sh+') & (df2_exclude['zero_referents'] == '2sh+')] # (6527, 21)
class3 = df2_exclude[(df2_exclude['subject_by_rule'] == '2sh+')] # (6739, 21)
class3_TP_person_matching = class3[(class3['zero_referents'] == "2ph+") | (class3['zero_referents'] == "2sh-")] # (39, 21)
class3_FN_idx_list = [4797, 2525]
class3_TP_from_FN = class3[class3.index.isin(class3_FN_idx_list) == True] # (2, 21)

class3_TP = pd.concat([class3_TP_string, class3_TP_person_matching, class3_TP_from_FN]) # (6568, 21) richtig
                                 

class_1_2_3_TP = pd.concat([class1_TP, class2_TP, class3_TP]) # (11793, 21)
    
#4. "subject_by_rule": 1sh+ or "subject_by_rule": 2sh+ / others (except "none" 1669 or "N/A" 2049) should be 142, 106 (should be total: 248, 5 exception)

df2_exclude_none = df2_exclude[(df2_exclude['subject_by_rule'] == 'none')] # (1669, 21)
df2_exclude_NA = df2_exclude[(df2_exclude['subject_by_rule'] == 'N/A')] # (2049, 21)

df2_exclude_none_NA = pd.concat([df2_exclude_none, df2_exclude_NA]) # (3718, 21) richtig

# not TP and not none & NA (what left)

# DF=DF1[~DF1.isin(DF2)].dropna(how = 'all')

left_case1 = df2_exclude[~df2_exclude.isin(df2_exclude_none_NA)].dropna(how = 'all') # (12165, 21) 15883-3718 = 12165 richtig
left_case2 = left_case1[~left_case1.isin(class_1_2_3_TP)].dropna(how = 'all') # (372, 21) 12165-11793 = 372 richtig

# left_case2.to_csv("left_case2_311022.csv", encoding = "utf-8-sig") 
# class 4는 총 248개로 계산되는데 왜 남아 있는 것은 372개인지 확인해야 함 (124개의 차이)

# 1ph-	2sh+ (1)
# 1sh+	2sh+ (58)
# 1sh+	zero_referents_string (54) e.g. 사람들
# 1sh+	antecedent_string (88) 
# 2sh+	1ph-,1ph+,1sh+ (65)
# 2sh+	zero_referents_string (46)
# 2sh+	antecedent_string (60)

# total: 372 richtig

# 아마 precision, recall, f-measure 다시 계산해야 할 듯!


# class1_TP.to_csv("class1_TP_011122.csv", encoding = "utf-8-sig") 
# class2_TP.to_csv("class2_TP_011122.csv", encoding = "utf-8-sig") 
# class3_TP.to_csv("class3_TP_011122.csv", encoding = "utf-8-sig") 
# left_case2.to_csv("left_case2_011122.csv", encoding = "utf-8-sig") 

In [17]:
# 각 클래스 별로 통계

class1_TP_rule_freq = class1_TP['group'].value_counts() 
print(class1_TP['group'].size) # 43 richtig

class2_TP_rule_freq = class2_TP['group'].value_counts() 
print(class2_TP['group'].size) # 5182 richtig

class3_TP_rule_freq = class3_TP['group'].value_counts() 
print(class3_TP['group'].size) # 6568 richtig


left_case2_rule_freq = left_case2['group'].value_counts() 
print(left_case2['group'].size) # 372 richtig

#class1_TP_rule_freq.to_csv("class1_TP_rule_freq_011122.csv", encoding = "utf-8-sig") 
#class2_TP_rule_freq.to_csv("class2_TP_rule_freq_011122.csv", encoding = "utf-8-sig") 
#class3_TP_rule_freq.to_csv("class3_TP_rule_freq_011122.csv", encoding = "utf-8-sig") 
#left_case2_rule_freq.to_csv("left_case2_rule_freq_011122.csv", encoding = "utf-8-sig") 


43
5183
6568
375


In [21]:
# df2 관련 richtig check 120923

class1_TP_rule_order = class1_TP['rule_order'].value_counts() 
class2_TP_rule_order = class2_TP['rule_order'].value_counts() 
class3_TP_rule_order = class3_TP['rule_order'].value_counts() 

#class1_TP_rule_order.to_csv("class1_TP_rule_order_120923.csv", encoding = "utf-8-sig") 
# class2_TP_rule_order.to_csv("class2_TP_rule_order_120923.csv", encoding = "utf-8-sig") 
# class3_TP_rule_order.to_csv("class3_TP_rule_order_120923.csv", encoding = "utf-8-sig") 

In [27]:
# getroffen이 0인 rule_order (rule 이름 가져오기_120923)

rule_order_stat_csv = pd.read_csv("df2_exclude_rule_subject_value_count_120923.csv", index_col=0)

rule_order_empty = rule_order_stat_csv['empty']
rule_order_empty_dropna = rule_order_empty.dropna(axis=0)
rule_order_empty_dropna_val = rule_order_empty_dropna.values 
rule_order_empty_dropna_val_list = rule_order_empty_dropna_val.tolist() # 932 richtig

rule_order_empty_dropna_val_list

[9.0,
 10.0,
 22.0,
 30.0,
 44.0,
 47.0,
 50.0,
 51.0,
 52.0,
 73.0,
 83.0,
 94.0,
 96.0,
 99.0,
 110.0,
 111.0,
 115.0,
 116.0,
 119.0,
 121.0,
 123.0,
 126.0,
 131.0,
 133.0,
 134.0,
 135.0,
 155.0,
 162.0,
 164.0,
 165.0,
 167.0,
 168.0,
 174.0,
 187.0,
 193.0,
 196.0,
 199.0,
 200.0,
 207.0]

In [28]:
df2_exclude = pd.read_csv("df2_exclude_230322.csv", index_col=0) 

In [31]:
df2_exclude_empty_rule = df2_exclude[df2_exclude.rule_order.isin(rule_order_empty_dropna_val_list) == True]

df2_exclude.dtypes # 당연히 df2에는 정보가 없음. 적용되지 않았으므로 
# before_exclude_annotation_error_verb_syn = before_exclude_annotation_error['verb_syn'].value_counts() 

# before_exclude_annotation_error_verb_syn

etri_sent                 int64
sent                     object
verb                     object
applied_rule             object
subject_by_rule          object
zero_referents           object
antecedent               object
group                    object
rule_order              float64
rule                     object
connective               object
base_morpheme            object
base_morpheme_second     object
connective_true           int64
zp_hono                  object
zp_mood                  object
sent_dep                 object
zp_markable_ID           object
zero_type                object
tense_zp                 object
verb_syn                 object
verb_head                 int64
Filter                   object
dtype: object

In [17]:
# 각 클래스 별로 통계 (Falsche Vorhersagen)

FV = pd.read_csv("Falsche_Vorhersagen_271222.csv", index_col=0)
print(FV.shape) # (372, 22) richtig

classA_FV = FV[FV["Klasse"] == "A"]
print(classA_FV.shape) # (1, 22) richtig

classB_FV = FV[FV["Klasse"] == "B"]
print(classB_FV.shape) # (58, 22) richtig

classC_FV = FV[FV["Klasse"] == "C"]
print(classC_FV.shape) # (65, 22) richtig

classDB_FV = FV[FV["Klasse"] == "D-B"]
print(classDB_FV.shape) # (142, 22) richtig

classDC_FV = FV[FV["Klasse"] == "D-C"]
print(classDC_FV.shape) # (106, 22) richtig

classA_FV_rule_freq = classA_FV['group'].value_counts() 
classB_FV_rule_freq = classB_FV['group'].value_counts() 
classC_FV_rule_freq = classC_FV['group'].value_counts() 
classDB_FV_rule_freq = classDB_FV['group'].value_counts() 
classDC_FV_rule_freq = classDC_FV['group'].value_counts() 

#classA_FV_rule_freq.to_csv("classA_FV_rule_freq_271222.csv", encoding = "utf-8-sig") 
#classB_FV_rule_freq.to_csv("classB_FV_rule_freq_271222.csv", encoding = "utf-8-sig") 
#classC_FV_rule_freq.to_csv("classC_FV_rule_freq_271222.csv", encoding = "utf-8-sig") 
#classDB_FV_rule_freq.to_csv("classDB_FV_rule_freq_271222.csv", encoding = "utf-8-sig") 
#classDC_FV_rule_freq.to_csv("classDC_FV_rule_freq_271222.csv", encoding = "utf-8-sig") 

(372, 22)
(1, 22)
(58, 22)
(65, 22)
(142, 22)
(106, 22)


In [18]:
# 각 클래스의 (class2-3) ranking 1-3 Regelgruppe의 실제 regel (Falsche Vorhersagen)

# Class B: group value_counts (ranking 1-3)

classB_FV_first_three_ranking = ['want statement', 'experience', 'possibility']
classB_FV_rules = classB_FV[classB_FV.group.isin(classB_FV_first_three_ranking)] 
print(classB_FV_rules.shape) # (37, 22) richtig

classB_FV_rules_value_count = classB_FV_rules["rule"].value_counts() 
# classB_FV_rules_value_count.to_csv("classB_FV_rules_value_count_271222.csv", encoding = "utf-8-sig") 

# Class C: group value_counts (ranking 1-2)

classC_FV_first_two_ranking = ['permission', 'possibility']
classC_FV_rules = classC_FV[classC_FV.group.isin(classC_FV_first_two_ranking)] 
print(classC_FV_rules.shape) # (44, 22) richtig

classC_FV_rules_value_count = classC_FV_rules["rule"].value_counts() 
#classC_FV_rules_value_count.to_csv("classC_FV_rules_value_count_271222.csv", encoding = "utf-8-sig") 

# Class DB: group value_counts (ranking 1-3)

classDB_FV_first_three_ranking = ['experience', 'progress', 'want statement']
classDB_FV_rules = classDB_FV[classDB_FV.group.isin(classDB_FV_first_three_ranking)] 
print(classDB_FV_rules.shape) # (90, 22) richtig

classDB_FV_rules_value_count = classDB_FV_rules["rule"].value_counts() 
#classDB_FV_rules_value_count.to_csv("classDB_FV_rules_value_count_271222.csv", encoding = "utf-8-sig") 

# Class DC: group value_counts (ranking 1-5)

classDC_FV_first_five_ranking = ['clau', 'honorification', 'permission', 'want statement', 'mood derivable']
classDC_FV_rules = classDC_FV[classDC_FV.group.isin(classDC_FV_first_five_ranking)] 
print(classDC_FV_rules.shape) # (68, 22) richtig

classDC_FV_rules_value_count = classDC_FV_rules["rule"].value_counts() 
#classDC_FV_rules_value_count.to_csv("classDC_FV_rules_value_count_271222.csv", encoding = "utf-8-sig") 


(37, 22)
(44, 22)
(90, 22)
(68, 22)


In [19]:
classA_FV_referents = classA_FV['zero_referents'].value_counts()  
print("classA_FV_referents:", classA_FV_referents)

classA_FV_referents: 2sh+    1
Name: zero_referents, dtype: int64


In [20]:
# 각 클래스 별로 referents & antecedents 통계

class1_TP_referents = class1_TP['zero_referents'].value_counts() 
print("class1_TP_referents:", class1_TP_referents)
class1_TP_antecedent = class2_TP['antecedent'].value_counts() 
print("class1_TP_antecedent:", class1_TP_antecedent)

class2_TP_referents = class2_TP['zero_referents'].value_counts() 
print("class2_TP_referents:", class2_TP_referents)
class2_TP_antecedent = class2_TP['antecedent'].value_counts() 
print("class2_TP_antecedent:", class2_TP_antecedent)

class3_TP_referents = class3_TP['zero_referents'].value_counts() 
print("class3_TP_referents:", class3_TP_referents)
class3_TP_antecedent = class3_TP['antecedent'].value_counts() 
print("class2_TP_antecedent:", class3_TP_antecedent)



class1_TP_referents: 1ph-    42
Name: zero_referents, dtype: int64
class1_TP_antecedent:         5049
제가        79
저희가       18
저도        13
저는         9
전          8
우리는        2
저희는        2
저          1
저희들도       1
Name: antecedent, dtype: int64
class2_TP_referents: 1sh+    4853
1ph+     153
1ph-      42
2sh+       1
Name: zero_referents, dtype: int64
class2_TP_antecedent:         5049
제가        79
저희가       18
저도        13
저는         9
전          8
우리는        2
저희는        2
저          1
저희들도       1
Name: antecedent, dtype: int64
class3_TP_referents: 2sh+    6527
2ph+      37
2sh-       2
Name: zero_referents, dtype: int64
class2_TP_antecedent:          6566
관광객으로       1
분은          1
Name: antecedent, dtype: int64


In [21]:
# 각 클래스의 (class2-3) ranking 1-3 Regelgruppe의 실제 regel

# Class 2: group value_counts (ranking 1-3)

class2_TP_first_three_ranking = ['want statement', 'suggestory formula', 'clau']
class2_TP_rules = class2_TP[class2_TP.group.isin(class2_TP_first_three_ranking)] # (3270, 21) richtig
class2_TP_rules_value_count = class2_TP_rules["rule"].value_counts() 

# class2_TP_rules_value_count.to_csv("class2_TP_rules_value_count_011122.csv", encoding = "utf-8-sig") 

# Class 3: group value_counts (ranking 1-3)

class3_TP_first_three_ranking = ['mood derivable', 'clau', 'want statement']
class3_TP_rules = class3_TP[class3_TP.group.isin(class3_TP_first_three_ranking)] # (4056, 21) richtig
class3_TP_rules.shape
class3_TP_rules_value_count = class3_TP_rules["rule"].value_counts() 

# class3_TP_rules_value_count.to_csv("class3_TP_rules_value_count_011122.csv", encoding = "utf-8-sig") 


In [22]:
# 주어가 틀린 경우 (class 4) error analysis

# classDB_FV

classDB_FV_referents = classDB_FV['zero_referents'].value_counts() 
#print("classDB_FV_referents:", classDB_FV_referents)

classDB_FV_antecedent = classDB_FV['antecedent'].value_counts() 
#print("classDB_FV_antecedent:", classDB_FV_antecedent)

# classDC_FV

classDC_FV_referents = classDC_FV['zero_referents'].value_counts() 
#print("classDC_FV_referents:", classDC_FV_referents)

classDC_FV_antecedent = classDC_FV['antecedent'].value_counts() 
#print("classDC_FV_antecedent:", classDC_FV_antecedent)

#classDB_FV_referents.to_csv("classDB_FV_referents_value_count_010223.csv", encoding = "utf-8-sig") 
#classDB_FV_antecedent.to_csv("classDB_FV_antecedent_value_count_010223.csv", encoding = "utf-8-sig") 
#classDC_FV_referents.to_csv("classDC_FV_referents_value_count_010223.csv", encoding = "utf-8-sig") 
#classDC_FV_antecedent.to_csv("classDC_FV_antecedent_value_count_010223.csv", encoding = "utf-8-sig") 


In [23]:
# 주어가 맞는 경우 rule group count

rule_freq_case1 = df2_case1['group'].value_counts() 
print(df2_case1['group'].size) # 381 richtig

rule_freq_first_person_match = df2_first_person_match['group'].value_counts() 
print(df2_first_person_match['group'].size) # 67 richtig

rule_freq_second_person_match = df2_second_person_match['group'].value_counts() 
print(df2_second_person_match['group'].size) # 67 richtig

rule_freq_case3 = df2_case3_antecedent_match['group'].value_counts() 
print(df2_case3_antecedent_match['group'].size) # 233 richtig

# rule_freq_case1.to_csv("rule_freq_case1_260922.csv", encoding = "utf-8-sig") 
# rule_freq_first_person_match.to_csv("rule_freq_first_person_match_260922.csv", encoding = "utf-8-sig") 
# rule_freq_second_person_match.to_csv("rule_freq_second_person_match_260922.csv", encoding = "utf-8-sig") 
# rule_freq_case3.to_csv("rule_freq_case3_260922.csv", encoding = "utf-8-sig") 

11422
195
39
134


In [24]:
# Error Analysis 061022

#1.1ph- as 2sh+ (count: 1)
#2.1sh+ as 2sh+ (count: 59)
df2_exclude_error_2 = df2_exclude[(df2_exclude['subject_by_rule'] == "1sh+") & (df2_exclude['zero_referents'] == "2sh+")]
print(df2_exclude_error_2.shape) # (59, 21)

df2_exclude_error_2_group = df2_exclude_error_2["group"].value_counts()
# print(df2_exclude_error_2_group)

# df2_exclude_error_2_group.to_csv("df2_exclude_error_2_group_061022.csv", encoding = "utf-8-sig") 

#3.1sh+ as etc. (count: 54) (the rest: 90/ total: 144)

df2_exclude_error_3 = df2_exclude[(~(df2_exclude['zero_referents'] == "1ph-") & ~(df2_exclude['zero_referents'] == "1ph+") & ~(df2_exclude['zero_referents'] == "1sh+") & ~(df2_exclude['zero_referents'] == "2sh+")) & (df2_exclude['subject_by_rule'] == "1sh+")]
print(df2_exclude_error_3.shape) # (275, 21) 필드값 없음까지!
df2_exclude_error_3_1 = df2_exclude_error_3[df2_exclude_error_3['zero_referents'].notnull()]
print(df2_exclude_error_3_1.shape) # (54, 21) 필드값 있는 경우만 

df2_exclude_error_3_1_group = df2_exclude_error_3_1["group"].value_counts()
df2_exclude_error_3_1_zero_referents = df2_exclude_error_3_1.groupby("zero_referents")["group"].value_counts()

# print(df2_exclude_error_3_1_group)

# df2_exclude_error_3_1_group.to_csv("df2_exclude_error_3_1_group_061022.csv", encoding = "utf-8-sig") 
# df2_exclude_error_3_1_zero_referents.to_csv("df2_exclude_error_3_1_zero_referents_071022.csv", encoding = "utf-8-sig") 

# df2_exclude_error_3_2 = df2_exclude_error_3[(~(df2_exclude_error_3['antecedent'] == "저") & ~(df2_exclude['antecedent'] == "저는") & ~(df2_exclude['antecedent'] == "저도") & ~(df2_exclude['antecedent'] == "저희가") & ~(df2_exclude['antecedent'] == "저희들도") & ~(df2_exclude['antecedent'] == "전") & ~(df2_exclude['antecedent'] == "제가")) & (df2_exclude['subject_by_rule'] == "1sh+")]
#"저", "저는", "저도", "저희가", "저희는", 저희들도", "전", "제가"

df2_exclude_error_3_antecedent = df2_exclude[(df2_exclude['subject_by_rule'] == "1sh+")]
print(df2_exclude_error_3_antecedent.shape) # (5382, 21) 필드값 없음까지!
df2_exclude_error_3_antecedent_notnull = df2_exclude_error_3_antecedent[df2_exclude_error_3_antecedent['antecedent'].str.strip().astype(bool)]
print(df2_exclude_error_3_antecedent_notnull.shape) # (221, 21) 필드값 있는 경우만!

df2_exclude_error_3_2 = df2_exclude_error_3_antecedent_notnull[~(df2_exclude_error_3_antecedent_notnull['antecedent'] == "저희는") & ~(df2_exclude_error_3_antecedent_notnull['antecedent'] == "저") & ~(df2_exclude_error_3_antecedent_notnull['antecedent'] == "저는") & ~(df2_exclude_error_3_antecedent_notnull['antecedent'] == "저도") & ~(df2_exclude_error_3_antecedent_notnull['antecedent'] == "저희가") & ~(df2_exclude_error_3_antecedent_notnull['antecedent'] == "저희들도") & ~(df2_exclude_error_3_antecedent_notnull['antecedent'] == "전") & ~(df2_exclude_error_3_antecedent_notnull['antecedent'] == "제가")]
print(df2_exclude_error_3_2.shape) # (90, 21) 필드값 있는 경우만!

df2_exclude_error_3_2_group = df2_exclude_error_3_2["group"].value_counts()
df2_exclude_error_3_2_antecedent = df2_exclude_error_3_2.groupby("antecedent")["group"].value_counts()
df2_exclude_error_3_2_group_antecedent = df2_exclude_error_3_2.groupby("group")["antecedent"].value_counts()

# df2_exclude_error_3_2_group.to_csv("df2_exclude_error_3_2_group_061022.csv", encoding = "utf-8-sig") 
# df2_exclude_error_3_2_antecedent.to_csv("df2_exclude_error_3_2_antecedent_071022.csv", encoding = "utf-8-sig")
# df2_exclude_error_3_2_group_antecedent.to_csv("df2_exclude_error_3_2_group_antecedent_071022.csv", encoding = "utf-8-sig") 



#(1ph-, 1ph+, 1sh+)
df2_exclude_error_4 = df2_exclude[(df2_exclude['subject_by_rule'] == "2sh+") & ((df2_exclude['zero_referents'] == "1ph-") | (df2_exclude['zero_referents'] == "1ph+") | (df2_exclude['zero_referents'] == "1sh+"))]
print(df2_exclude_error_4.shape) # (65, 21)

df2_exclude_error_4_group = df2_exclude_error_4["group"].value_counts()
# df2_exclude_error_4_group.to_csv("df2_exclude_error_4_group_061022.csv", encoding = "utf-8-sig") 

# zero_referents가 1ph-, 1ph+, 1sh+, 2ph+, 2sh-, 2sh+ 가 아닌 subject_by_rule "2sh+"
df2_exclude_error_4_1 = df2_exclude[(~(df2_exclude['zero_referents'] == "2ph+") & ~(df2_exclude['zero_referents'] == "2sh-") & ~(df2_exclude['zero_referents'] == "1ph-") & ~(df2_exclude['zero_referents'] == "1ph+") & ~(df2_exclude['zero_referents'] == "1sh+") & ~(df2_exclude['zero_referents'] == "2sh+")) & (df2_exclude['subject_by_rule'] == "2sh+")]
print(df2_exclude_error_4_1.shape) # 필드값 없는 것 까지 

df2_exclude_error_4_1_notnull = df2_exclude_error_4_1[df2_exclude_error_4_1['zero_referents'].notnull()]
print(df2_exclude_error_4_1_notnull.shape) # (46, 21)

df2_exclude_error_4_1_notnull_group = df2_exclude_error_4_1_notnull["group"].value_counts()
# df2_exclude_error_4_1_notnull_group.to_csv("df2_exclude_error_4_1_notnull_group_061022.csv", encoding = "utf-8-sig") 

df2_exclude_error_4_antecedent = df2_exclude[(df2_exclude['subject_by_rule'] == "2sh+")]
print(df2_exclude_error_4_antecedent.shape) # (6739, 21) 필드값 없음까지!
df2_exclude_error_4_antecedent_notnull = df2_exclude_error_4_antecedent[df2_exclude_error_4_antecedent['antecedent'].str.strip().astype(bool)]
print(df2_exclude_error_4_antecedent_notnull.shape) # (62, 21) 필드값 있는 경우만!

df2_exclude_error_4_antecedent_notnull_group = df2_exclude_error_4_antecedent_notnull["group"].value_counts()
# df2_exclude_error_4_antecedent_notnull_group.to_csv("df2_exclude_error_4_antecedent_notnull_group_061022.csv", encoding = "utf-8-sig") 

# 1ph-, 1ph+, 1sh+, 2ph+, 2sh-, 2sh+ 제외, 그리고 필드값 없음 제외
df2_exclude_error_5 = df2_exclude[(df2_exclude['subject_by_rule'] == "2sh+") & (~(df2_exclude['zero_referents'] == "1ph-") & ~(df2_exclude['zero_referents'] == "1ph+") & ~(df2_exclude['zero_referents'] == "1sh+") & ~(df2_exclude['zero_referents'] == "2ph+") & ~(df2_exclude['zero_referents'] == "2sh-") & ~(df2_exclude['zero_referents'] == "2sh+"))]
print(df2_exclude_error_5.shape) # 108, 21) 필드값 없는 것 까지

df2_exclude_error_5_1  = df2_exclude_error_5[df2_exclude_error_5['zero_referents'].notnull()]
print(df2_exclude_error_5_1.shape) # (46, 21) 필드값 없는 것 까지

df2_exclude_error_5_1_zero_referents = df2_exclude_error_5_1.groupby("zero_referents")["group"].value_counts()
# df2_exclude_error_5_1_zero_referents.to_csv("df2_exclude_error_5_1_zero_referents_081022.csv", encoding = "utf-8-sig") 



(59, 23)
(275, 23)
(54, 23)
(5382, 23)
(221, 23)
(90, 23)
(65, 23)
(108, 23)
(46, 23)
(6739, 23)
(62, 23)
(108, 23)
(46, 23)


In [25]:
# none (1.669) 220223 

df3_none_change = pd.read_csv("df3_rule_applied_220223.csv", index_col=0)  # referents의 값이 없거나 값이 틀린 case 수정함 220223 
# (15883, 21) richtig
# df3_none_change.to_csv("df3_none_change_220223.csv", encoding = "utf-8-sig") 


df3_none_change_none = df3_none_change[df3_none_change["subject_by_rule"] == "none"]
df3_none_change_none.shape #(1669, 21) richtig
# df3_none_change_none.to_csv("df3_none_change_none_220223.csv", encoding = "utf-8-sig") 


df3_none_change_none_zero_referents = df3_none_change_none["zero_referents"].value_counts()
# df3_none_change_none_zero_referents.to_csv("df3_none_change_none_zero_referents_220223.csv", encoding = "utf-8-sig") 

df3_none_change_none_antecedent = df3_none_change_none["antecedent"].value_counts()
# df3_none_change_none_antecedent.to_csv("df3_none_change_none_antecedent_220223.csv", encoding = "utf-8-sig") 


In [26]:
# none data type added 220223

df3_none_type_added = pd.read_csv("df3_none_type_added_220223.csv", index_col=0)  # referents의 type을 추가함
# (1669, 22) richtig  열 하나가 더 늘어남

df3_none_type_added_rule_order = df3_none_type_added.groupby("type")["rule_order"].value_counts()

# df3_none_type_added_rule_order.to_csv("df3_none_type_added_rule_order_220223.csv", encoding = "utf-8-sig") 

df3_none_type_added_rule_order_value_count = df3_none_type_added["rule_order"].value_counts()

# df3_none_type_added_rule_order_value_count

df3_none_type_added_rule_order_type = df3_none_type_added.groupby("rule_order")["type"].value_counts()

# df3_none_type_added_rule_order_type.to_csv("df3_none_type_added_rule_order_type_220223.csv", encoding = "utf-8-sig") 

df3_none_type_added_rule_order_213 = df3_none_type_added[df3_none_type_added["rule_order"] == 213]

# df3_none_type_added_rule_order_213_verb_syn = df3_none_type_added_rule_order_213["verb_syn"].value_counts()

# df3_none_type_added_rule_order_213_verb_syn.to_csv("df3_none_type_added_rule_order_213_verb_syn_220223.csv", encoding = "utf-8-sig") 



In [27]:
# N/A (2.049) analysis 150323
# df2_exclude_061022 에서 NA가 할당된 case만 따로 csv로 만들기, 2.049여야 함

#df2_exclude_orig = pd.read_csv("df2_exclude_061022.csv", index_col=0)

df2_exclude_orig = pd.read_csv("df2_exclude_230322.csv", index_col=0)

# df2_exclude_orig.shape # (15883, 21) rightig

df2_NA = df2_exclude_orig[df2_exclude_orig["subject_by_rule"].isnull()]

df2_NA.shape # (2049, 21) richtig
 
# df2_NA.to_csv("df2_NA_check_260922.csv", encoding = "utf-8-sig") 
#df2_NA.to_csv("df2_NA_check_230322.csv", encoding = "utf-8-sig") 

(2049, 23)

In [28]:
# N/A analysis 1 (root) 160323


# df2_NA_orig = pd.read_csv("df2_NA_check_260922.csv", index_col=0)
df2_NA_orig = pd.read_csv("df2_NA_check_230322.csv", index_col=0)

# df2_NA_orig.shape # (2049, 17) richtig

df2_NA_root = df2_NA_orig[df2_NA_orig["verb_syn"] == "root"]

# df2_NA_root.shape # (854, 17) richtig

# df2_NA_root에 새로운 열 추가 

conditionlist = [
    (df2_NA_root['zp_mood'] == 'decl'),
    (df2_NA_root['zp_mood'] == 'ques')] 

choicelist = ['1sh+', '2sh+']

df2_NA_root['new_rule_root'] = np.select(conditionlist, choicelist, default='Not Specified')

# df2_NA_root.shape # (17266, 21)

# df2_NA_root.to_csv("df2_NA_check_root_new_rule_160323.csv", encoding = "utf-8-sig") 

<ipython-input-28-f877c4105f33>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_NA_root['new_rule_root'] = np.select(conditionlist, choicelist, default='Not Specified')


In [29]:
# N/A analysis 1 right prediction (root new rule 정확도 체크) 160323

# df2_NA_root에서 시작하면 됨

first_person_list = ['1ph-', '1ph+', '1sh+']
first_person_ant_list = ['저는', '제가', '저도', '저희가', '저희는', '전']
second_person_list = ['2ph-', '2ph+', '2sh+']


df2_NA_root_ich = df2_NA_root[df2_NA_root['new_rule_root'] == '1sh+']

# df2_NA_root_ich.shape # (579, 18) richtig

df2_NA_root_Sie = df2_NA_root[df2_NA_root['new_rule_root'] == '2sh+']

# df2_NA_root_Sie.shape # (275, 18) richtig


df2_NA_root_first_person_match = df2_NA_root_ich[df2_NA_root_ich.zero_referents.isin(first_person_list) == True]
df2_NA_root_second_person_match = df2_NA_root_Sie[df2_NA_root_Sie.zero_referents.isin(second_person_list) == True]

# df2_NA_root_first_person_match.shape # (400, 18) richtig
# df2_NA_root_second_person_match.shape # (46, 18) richtig

conditionlist = [
    (df2_NA_root['new_rule_root'] == '1sh+') & (df2_NA_root['zero_referents'].isin(first_person_list) == True),
    (df2_NA_root['new_rule_root'] == '1sh+') & (df2_NA_root['antecedent'].isin(first_person_ant_list) == True),
    (df2_NA_root['new_rule_root'] == '2sh+') & (df2_NA_root['zero_referents'].isin(second_person_list) == True),
    (df2_NA_root['new_rule_root'] == '1sh+') & (df2_NA_root['zero_referents'].isin(first_person_list) == False),
    (df2_NA_root['new_rule_root'] == '2sh+') & (df2_NA_root['zero_referents'].isin(second_person_list) == False)] 

choicelist = ['richtig_first', 'richtig_first_ant', 'richtig_second', 'wrong_first', 'wrong_second']

df2_NA_root['prediction'] = np.select(conditionlist, choicelist, default='N/A')

# df2_NA_root.head()

# df2_NA_root.to_csv("df2_NA_check_root_prediction_160323.csv", encoding = "utf-8-sig") 

df2_NA_root_predic = df2_NA_root['prediction'].value_counts() 

richtig_list = ["richtig_first", "richtig_first_ant"]

df2_NA_root_predic_first = df2_NA_root[df2_NA_root["prediction"].isin(richtig_list) == True]
df2_NA_root_predic_first_value_count = df2_NA_root_predic_first["zero_referents"].value_counts() 
df2_NA_root_predic_first_ante_value_count = df2_NA_root_predic_first["antecedent"].value_counts() 

# df2_NA_root_predic_first_value_count.to_csv("df2_NA_root_predic_first_value_count_new_160323.csv", encoding = "utf-8-sig") 

# df2_NA_root_predic_first_ante_value_count.to_csv("df2_NA_root_predic_first_ant_value_count_new_160323.csv", encoding = "utf-8-sig") 


df2_NA_root_predic_second = df2_NA_root[df2_NA_root["prediction"] == "richtig_second"]
df2_NA_root_predic_second_value_count = df2_NA_root_predic_second["zero_referents"].value_counts() 

# df2_NA_root_predic_second_value_count.to_csv("df2_NA_root_predic_second_value_count_160323.csv", encoding = "utf-8-sig") 

# df2_NA_root_predic.to_csv("df2_NA_check_root_prediction_STAT_160323.csv", encoding = "utf-8-sig") 



<ipython-input-29-b7f0585fb9ef>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_NA_root['prediction'] = np.select(conditionlist, choicelist, default='N/A')


In [30]:
# N/A analysis 1 wrong prediction (root new rule 정확도 체크) 160323

df2_NA_root_predic_wrong_first = df2_NA_root[df2_NA_root['prediction'] == 'wrong_first']
df2_NA_root_predic_wrong_second = df2_NA_root[df2_NA_root['prediction'] == 'wrong_second']

# df2_NA_root_predic_wrong_first.shape # (151, 19) richtig
# df2_NA_root_predic_wrong_second.shape # (229, 19) richtig


df2_NA_root_predic_wrong_first_referents_value_count = df2_NA_root_predic_wrong_first["zero_referents"].value_counts() 
df2_NA_root_predic_wrong_first_ant_value_count = df2_NA_root_predic_wrong_first["antecedent"].value_counts() 

df2_NA_root_predic_wrong_second_referents_value_count = df2_NA_root_predic_wrong_second["zero_referents"].value_counts() 
df2_NA_root_predic_wrong_second_ant_value_count = df2_NA_root_predic_wrong_second["antecedent"].value_counts() 

#df2_NA_root_predic_wrong_first_referents_value_count.to_csv("wrong_first_referents_value_count_160323.csv", encoding = "utf-8-sig") 
#df2_NA_root_predic_wrong_first_ant_value_count.to_csv("wrong_first_antecedent_value_count_160323.csv", encoding = "utf-8-sig") 
#df2_NA_root_predic_wrong_second_referents_value_count.to_csv("wrong_second_referents_value_count_160323.csv", encoding = "utf-8-sig") 
#df2_NA_root_predic_wrong_second_ant_value_count.to_csv("wrong_second_antecedent_value_count_160323.csv", encoding = "utf-8-sig") 

In [31]:
# N/A analysis 2 (clau) 160323

df2_NA_clau = df2_NA_orig[df2_NA_orig["verb_syn"] == "clau"]

# df2_NA_clau.shape # (1195, 17) richtig

# df2_NA_clau에 새로운 열 추가 

df2_NA_clau["new_rule_clau"] = np.where(df2_NA_clau["connective"] == "class_A", "copy","Not Specified")
# df2_NA_clau.head()

# df2_NA_clau.to_csv("df2_NA_check_clau_new_rule_230323.csv", encoding = "utf-8-sig") 

# index에 따라 base_morpheme과 base_morpheme_second를 추가 가능?


<ipython-input-31-974c902d2d47>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_NA_clau["new_rule_clau"] = np.where(df2_NA_clau["connective"] == "class_A", "copy","Not Specified")


In [32]:
# N/A analysis 2 (clau new rule 정확도 체크) 290322

# 복사한 주어가 같냐 안 같냐

# 1. 기존에 clau 테스트 한 파일 불러와서 idx 저장

clau_one = pd.read_csv('clau_count_one_add_subject_and_verb_29052022.csv') 
clau_one_idx = clau_one['idx']
clau_one_idx_val = clau_one_idx.values 
clau_one_idx_val_list = clau_one_idx_val.tolist() 


# len(clau_one_idx_val_list) # 432 richtig

# 2. 현재 clau test 파일 불러와서 connective가 class_A인 케이스의 idx 저장하고 1의 idx가 2의 idx에 다 포함되는지 확인

clau_check = pd.read_csv('df2_NA_check_clau_new_rule_230323.csv') 
clau_class_A = clau_check[clau_check["connective"] == "class_A"]

# clau_class_A.shape # (740, 25) richtig

clau_class_A_idx = clau_class_A['idx']
clau_class_A_idx_val = clau_class_A_idx.values 
clau_class_A_idx_val_list = clau_class_A_idx_val.tolist()

# len(clau_class_A_idx_val_list) # 740 richtig

# 3. 리스트 값 비교

set1 = set(clau_one_idx_val_list)
set2 = set(clau_class_A_idx_val_list)

set_intersection = set1.intersection(set2)

# len(set_intersection) # 370

# 432가 아니라 370이군!!




In [33]:
# 기존 코드 활용해서 clau_one과 주어 복사 가능한지 시도해보기 290323

# df2_exclude 대신 df2_exclude_orig 활용하기
# df2_exclude_orig # "df2_exclude_230322.csv", index_col=0

df4 = df2_exclude_orig[df2_exclude_orig["Filter"] == 'C']
# print(df4.shape) # (2049, 23) NA의 수와 같음, richtig

df4 = df4.drop(['subject_by_rule','group','rule'], axis=1)
# df4 = df4.sort_values(by=['connective', 'verb_syn'])
# df4.to_csv("df4_290323.csv", encoding = "utf-8-sig") 

# Filter A와 B인 case들과 class_A와의 접점을 찾아야 하는 이유는 꼭 복사될 주어가 정답이 아니더라도
# 복사할 수 있는 주어가 있을 수 있다는 것이기 때문에
# 정답이 아닌 것은 나중에 정확도 평가할 때 Error analysis를 하면 될 듯

df2_FilterA_B = df2_exclude_orig[(df2_exclude_orig["Filter"] == 'A') | (df2_exclude_orig["Filter"] == 'B')] 
# print(df2_FilterA_B.shape) # (13834, 23) Resolutionsystem의 input 수와 같음, richtig

# Filter A와 B인 case와 class_A에서 중복되는 etri_sent있는지 확인하고, 해당 df에 대해서 Filter A와 B의 분포 살펴보기

df2_FilterA_B_etri_sent = df2_FilterA_B.etri_sent
df2_FilterA_B_etri_sent_val = df2_FilterA_B_etri_sent.values 
df2_FilterA_B_etri_sent_val_list = df2_FilterA_B_etri_sent_val.tolist() # 13834
# print(len(df2_FilterA_B_etri_sent_val_list))
df2_FilterA_B_etri_sent_val_list_set = set(df2_FilterA_B_etri_sent_val_list) # 12480
# print(len(df2_FilterA_B_etri_sent_val_list_set))

# print("df2_FilterA_B_etri_sent_val_list:", len(df2_FilterA_B_etri_sent_val_list))
# print("df2_FilterA_B_etri_sent_val_list_set:", len(df2_FilterA_B_etri_sent_val_list_set))

df4_class_A = df4[df4.connective == 'class_A'] 
# print(df4_class_A.shape) # (740, 20) class A의 connective를 지닌 case 740 richtig
# df4_class_A.to_csv("df4_class_A_290323.csv", encoding = "utf-8-sig") 

df4_class_A_etri_sent = df4_class_A.etri_sent
df4_class_A_etri_sent_val = df4_class_A_etri_sent.values 
df4_class_A_etri_sent_val_list = df4_class_A_etri_sent_val.tolist() # 740
df4_class_A_etri_sent_val_list_set = set(df4_class_A_etri_sent_val_list) # 714

# print(len(df4_class_A_etri_sent_val_list))
# print(len(df4_class_A_etri_sent_val_list_set))

intersection_df2_FilterA_B_and_classA = list(set(df2_FilterA_B_etri_sent_val_list) & set(df4_class_A_etri_sent_val_list))
# print(intersection_df2_FilterA_B_and_classA)
# print("intersection_df2_FilterA_B_and_classA:", len(intersection_df2_FilterA_B_and_classA)) # 575 (740개의 77.70%)

# df4_class_A_check의 경우 class A에 속하는 conjunction을 지니고 있지만 resolutionsystem에서 규칙을 적용 받지 못한 etri_sentence에 관한 데이터임

df4_class_A_check = df4_class_A[df4_class_A.etri_sent.isin(intersection_df2_FilterA_B_and_classA) == False] # (146, 20)
# df4_class_A_check의 경우 복사할 주어가 있는지 또는 이미 존재하는 주어가 있는지 등을 체크해 볼 수 있음  
# df4_class_A_check.to_csv("df4_class_A_check_290323.csv", encoding = "utf-8-sig") 
## 중복되는 etri_sent가 있을 수 있음
## "df4_class_A_check_2_comment_23052022.xslx"에 comment 적어놓음 (230522)


In [34]:
## 우선 FilterA,B와 FilterC면서 class A인 케이스 간의 etri_sent의 intersection이였던 case의 clue를 check하면서 
## 이미 할당된 subject가 있는 경우 이것을 자동으로 복사할 수 있는지 check하는 단계 290323

classA_clue_check = df4_class_A[df4_class_A.etri_sent.isin(intersection_df2_FilterA_B_and_classA) == True] 
# print(classA_clue_check.shape) #(594, 20) 아마 set으로 count하면 575일듯! 중복된 항목 제거하면 575임
# classA_clue_check.to_csv("classA_clue_check_290323.csv", encoding = "utf-8-sig") 

# sent_dep 예시: ['modi', 'modi', 'clau', 'modi', 'clau', 'root', 'punc']

classA_sent_dep = classA_clue_check.sent_dep
classA_sent_dep_val = classA_sent_dep.values 
classA_sent_dep_val_list = classA_sent_dep_val.tolist() 

# print(len(classA_sent_dep_val_list)) # 594 richtig

classA_etri_sent = classA_clue_check.etri_sent
classA_etri_sent_val = classA_etri_sent.values 
classA_etri_sent_val_list = classA_etri_sent_val.tolist() 

# print(len(classA_etri_sent_val_list)) # 594 richtig

# sent_dep에 있는 clau의 개수를 세서 새로운 열로 추가함 
        
clau_count = [dep.count('clau') for dep in classA_clue_check['sent_dep']]
clau_count_add = classA_clue_check.assign(clau = clau_count) 

# print(clau_count_add.shape) # (594, 21) richtig

# clau_count_add.to_csv("clau_count_add_290323.csv", encoding = "utf-8-sig") 


In [35]:
# df4_class_A에서 class A에 속하는 conjunction을 지닌 predicate 740개들 중에 clau의 개수 310323


# sent_dep 예시: ['modi', 'modi', 'clau', 'modi', 'clau', 'root', 'punc']

df4_classA_sent_dep = df4_class_A.sent_dep
df4_classA_sent_dep_val = df4_classA_sent_dep.values 
df4_classA_sent_dep_val_list = df4_classA_sent_dep_val.tolist() 

# print(len(df4_classA_sent_dep_val_list)) # 740 richtig

df4_clau_count = [dep.count('clau') for dep in df4_class_A['sent_dep']]
# print(len(df4_clau_count)) # 740 richtig

df4_clau_count_add = df4_class_A.assign(clau = df4_clau_count) 

# df4_clau_count_add.to_csv("df4_clau_count_add_310323.csv", encoding = "utf-8-sig") 

df4_clau_freq = df4_clau_count_add['clau'].value_counts()

# df4_clau_freq.to_csv("df4_clau_freq_310323.csv", encoding = "utf-8-sig") 

df4_clau_freq


1    464
2    212
3     52
4      9
5      3
Name: clau, dtype: int64

In [36]:
# clau가 하나로 카운트 되는 464개의 predicates에 대해서 몇 개가 etri_sent가 같은지 확인하기 

df4_clau_one = df4_clau_count_add[df4_clau_count_add['clau'] == 1]
# print(df4_clau_one.shape) # (464, 21) richtig

df4_clau_one_etri_sent = df4_clau_one.etri_sent
df4_clau_one_etri_sent_val = df4_clau_one_etri_sent.values 
df4_clau_one_etri_sent_val_list = df4_clau_one_etri_sent_val.tolist() 

# print(len(df4_clau_one_etri_sent)) # 464 richtig (중복되는 etri_sent가 없음!)

df2_FilterA = df2_exclude_orig[df2_exclude_orig["Filter"] == 'A'] 

# print(df2_FilterA.shape) # (11644, 23) richtig

df2_FilterA_etri_sent = df2_FilterA.etri_sent
df2_FilterA_etri_sent_val = df2_FilterA_etri_sent.values 
df2_FilterA_etri_sent_val_list = df2_FilterA_etri_sent.tolist() 

# print(len(df2_FilterA_etri_sent_val_list)) # 11644 richtig

# print(len(set(df4_clau_one_etri_sent_val_list))) # 464 (중복되는 etri_sent가 없음!)
# print(len(set(df2_FilterA_etri_sent_val_list))) # 11644 (중복되는 etri_sent가 없음!)

# df2_FilterA에서 df4_clau_one_etri_sent_val_list 안에 들어 있는 case를 extract하고 이것의 subject_by_rule을 알아내기

df2_FilterA_root = df2_FilterA[df2_FilterA.etri_sent.isin(df4_clau_one_etri_sent_val_list)]

# print(df2_FilterA_root.shape) # (365, 23)

# clau가 하나로 count되는 464개의 predicates 중 이미 resolutionssystem에 의해 할당된 referents가 있는 case가 365개라는 뜻 (전체의 78,66%)! 310323 
# 나머지 99개는 manuell하게 확인해보기 (explicit한 주어가 있는지, 없는 경우에도 S1과 S2의 주어가 같은지)

df2_FilterA_root_subject_by_rule = df2_FilterA_root.subject_by_rule
df2_FilterA_root_subject_by_rule_val = df2_FilterA_root_subject_by_rule.values 
df2_FilterA_root_subject_by_rule_val_list = df2_FilterA_root_subject_by_rule_val.tolist() 
# print(len(df2_FilterA_root_subject_by_rule_val_list)) # 365 richtig

df2_FilterA_root_verb = df2_FilterA_root.verb
df2_FilterA_root_verb_val = df2_FilterA_root_verb.values 
df2_FilterA_root_verb_val_list = df2_FilterA_root_verb_val.tolist() 
# print(len(df2_FilterA_root_verb_val_list)) # 365 richtig

df2_FilterA_root_etri_sent = df2_FilterA_root.etri_sent
df2_FilterA_root_etri_sent_val = df2_FilterA_root_etri_sent.values 
df2_FilterA_root_etri_sent_val_list = df2_FilterA_root_etri_sent_val.tolist() 
# print(len(df2_FilterA_root_etri_sent_val_list)) # 365 richtig

filterA_subject_by_rule_dict = dict(zip(df2_FilterA_root_etri_sent_val_list, df2_FilterA_root_subject_by_rule_val_list)) 
# print(len(filterA_subject_by_rule_dict)) # 365 richtig 

filterA_verb_dict = dict(zip(df2_FilterA_root_etri_sent_val_list, df2_FilterA_root_verb_val_list)) 
# print(len(filterA_verb_dict)) # 365 richtig 

# df4_clau_one에 filterA에 있던 정보인 subject_by_rule과 verb copy하기 310323

filter_A_subject_by_rule_keys = list(filterA_subject_by_rule_dict.keys())

filter_A_copied_subject_by_rule = [] 

for sent in df4_clau_one['etri_sent']:
    if sent in filter_A_subject_by_rule_keys:
        filter_A_copied_subject_by_rule.append(filterA_subject_by_rule_dict[sent])
    
# print(len(filter_A_copied_subject_by_rule)) # 365 richtig

filter_A_verb_dict_keys = list(filterA_verb_dict.keys())

filter_A_copied_verb = []

for sent in df4_clau_one['etri_sent']:
    if sent in filter_A_verb_dict_keys:
        filter_A_copied_verb.append(filterA_verb_dict[sent])

# print(len(filter_A_copied_verb)) # 365 richtig

# df4_clau_one에서 etri_sent가 df2_FilterA_root_etri_sent_val_list인 것만 df로 우선 만들어야 함 (464 -> 365)

df4_clau_one_365 = df4_clau_one[df4_clau_one.etri_sent.isin(df2_FilterA_root_etri_sent_val_list)]
# print(df4_clau_one_365.shape) # (365, 21) richtig


# 열이 늘어나야 함! 21 - 22 - 23             
clau_one_365_add_subject_by_rule = df4_clau_one_365.assign(copied_subject_by_rule = filter_A_copied_subject_by_rule) 
clau_one_365_add_subject_and_verb = clau_one_365_add_subject_by_rule.assign(copied_verb = filter_A_copied_verb) 

# print(clau_one_365_add_subject_by_rule.shape) # (365, 22) richtig
# print(clau_one_365_add_subject_and_verb.shape) # (365, 23) richtig

# clau_one_365_add_subject_and_verb.to_csv("clau_one_365_add_subject_and_verb_310323.csv", encoding = "utf-8-sig") 

# df4_clau_one.to_csv("df4_clau_one_020423.csv", encoding = "utf-8-sig") 



In [37]:
# 365개 대상으로 copied_subject_by_rule와 zero_referents 비교 020423

# 이 중 copied_subject_by_rule가 none인 20개 case도 manuell하게 주어가 같은지 확인해야 함


# 1. copied_subject_by_rule '1ph-' / zero_referents '1ph-'

clau_one_365_add_subject_and_verb_wir = clau_one_365_add_subject_and_verb[clau_one_365_add_subject_and_verb['copied_subject_by_rule'] == '1ph-']
# print(clau_one_365_add_subject_and_verb_wir.shape) # (4, 23) richtig

clau_one_365_add_subject_and_verb_wir_match = clau_one_365_add_subject_and_verb_wir[clau_one_365_add_subject_and_verb_wir['zero_referents'] == '1ph-']
# print(clau_one_365_add_subject_and_verb_wir_match.shape) # (4, 23) richtig

# '1ph-'(wir)가 copy된 case는 총 4으로 이 중 4개 (100%)가 주어를 공유하는 것으로 분석됨

# 2. copied_subject_by_rule '1sh+' / zero_referents '1ph+' / 1sh+

clau_one_365_add_subject_and_verb_ich = clau_one_365_add_subject_and_verb[clau_one_365_add_subject_and_verb['copied_subject_by_rule'] == '1sh+']
# print(clau_one_365_add_subject_and_verb_ich.shape) # (200, 23) richtig

zero_referents_first_person = ['1ph+', '1sh+']

clau_one_365_first_person_match = clau_one_365_add_subject_and_verb_ich[clau_one_365_add_subject_and_verb_ich.zero_referents.isin(zero_referents_first_person) == True]

# print(clau_one_365_first_person_match.shape) # (188, 23) richtig

# right prediction: 200개 중 188개 match (94%)

clau_one_365_first_person_not_match = clau_one_365_add_subject_and_verb_ich[clau_one_365_add_subject_and_verb_ich.zero_referents.isin(zero_referents_first_person) == False]

# print(clau_one_365_first_person_not_match.shape) # (12, 23) 200 - 188 = 12 richtig

# wrong prediction: 200개 중 12 (6%)

# '1sh+'(ich)가 copy된 case는 총 200으로 이 중 188개 (94%)가 주어를 공유하는 것으로 분석됐고, 12개 (6%)가 주어를 공유하지 않는 것으로 나타남
# clau_one_365_first_person_match와 clau_one_365_first_person_not_match를 각각 추가 열로 넣을 수 있다면 이걸 통해서 예문, conjunction 다 뽑을 수 있음 020423


# 3. copied_subject_by_rule '2sh+' / zero_referents '2sh+' 

clau_one_365_add_subject_and_verb_Sie = clau_one_365_add_subject_and_verb[clau_one_365_add_subject_and_verb['copied_subject_by_rule'] == '2sh+']
# print(clau_one_365_add_subject_and_verb_Sie.shape) # (141, 23) richtig

clau_one_365_add_subject_and_verb_Sie_match = clau_one_365_add_subject_and_verb_Sie[clau_one_365_add_subject_and_verb_Sie['zero_referents'] == '2sh+']
# print(clau_one_365_add_subject_and_verb_Sie_match.shape) # (132, 23) richtig

# right prediction: 141개 중 132개 match (93,62%)

clau_one_365_add_subject_and_verb_Sie_not_match = clau_one_365_add_subject_and_verb_Sie[clau_one_365_add_subject_and_verb_Sie['zero_referents'] != '2sh+']
print(clau_one_365_add_subject_and_verb_Sie_not_match.shape) # (9, 23) 141 - 132 richtig

# wrong prediction: 141개 중 9개 (6,38%)

# '1sh+'(Sie)가 copy된 case는 총 141으로 이 중 132개 (93,62%)가 주어를 공유하는 것으로 분석됐고, 9개 (6,38%)가 주어를 공유하지 않는 것으로 나타남
# clau_one_365_first_person_match와 clau_one_365_first_person_not_match를 각각 추가 열로 넣을 수 있다면 이걸 통해서 예문, conjunction 다 뽑을 수 있음 020423


# 4. copied_subject_by_rule 'none' 

clau_one_365_add_subject_and_verb_none = clau_one_365_add_subject_and_verb[clau_one_365_add_subject_and_verb['copied_subject_by_rule'] == 'none']
# print(clau_one_365_add_subject_and_verb_none.shape) # (20, 23) richtig

# 이 중 copied_subject_by_rule가 none인 20개 case도 manuell하게 주어가 같은지 확인해야 함




# df2_NA_root_first_person_match = df2_NA_root_ich[df2_NA_root_ich.zero_referents.isin(first_person_list) == True]
# df2_NA_root_second_person_match = df2_NA_root_Sie[df2_NA_root_Sie.zero_referents.isin(second_person_list) == True]

# # df2_NA_root_first_person_match.shape # (400, 18) richtig
# # df2_NA_root_second_person_match.shape # (46, 18) richtig

# conditionlist = [
#     (df2_NA_root['new_rule_root'] == '1sh+') & (df2_NA_root['zero_referents'].isin(first_person_list) == True),
#     (df2_NA_root['new_rule_root'] == '1sh+') & (df2_NA_root['antecedent'].isin(first_person_ant_list) == True),
#     (df2_NA_root['new_rule_root'] == '2sh+') & (df2_NA_root['zero_referents'].isin(second_person_list) == True),
#     (df2_NA_root['new_rule_root'] == '1sh+') & (df2_NA_root['zero_referents'].isin(first_person_list) == False),
#     (df2_NA_root['new_rule_root'] == '2sh+') & (df2_NA_root['zero_referents'].isin(second_person_list) == False)] 

# choicelist = ['richtig_first', 'richtig_first_ant', 'richtig_second', 'wrong_first', 'wrong_second']

# df2_NA_root['prediction'] = np.select(conditionlist, choicelist, default='N/A')

# # df2_NA_root.head()

# # df2_NA_root.to_csv("df2_NA_check_root_prediction_160323.csv", encoding = "utf-8-sig") 

(9, 23)


In [38]:
# clau가 1개인 464개 중 복사할 주어가 있는 364개가 아닌 나머지 99개, S2에 왜 복사할 주어가 없는지 확인해 보기

df4_clau_one_99 = df4_clau_one[df4_clau_one.etri_sent.isin(df2_FilterA_root_etri_sent_val_list) == False]

# print(df4_clau_one_99.shape) # (99, 21) richtig

# df4_clau_one_99.to_csv("df4_clau_one_99_020423.csv", encoding = "utf-8-sig") 

In [39]:
# Null이 있다고 annotated된 전체 17,266개의 predicate (df2)을 대상으로 
# S1의 sentence ID (df4_clau_one_etri_sent_val_list)와 같은 value를 지니는 root인 predicate이 몇 개나 있는지 확인 020423
# 이들은 S2의 Subjekt가 explcit하지 않는 경우임

# print(df2.shape) # (17266, 23)
# print(len(df4_clau_one_etri_sent_val_list)) # 464

# 기존 list는 int였는데, df2의 etri_sent가 str이여서, 기존 list의 요소들을 int에서 str으로 바꿔줌

df4_clau_one_etri_sent_val_list_str = list(map(str, df4_clau_one_etri_sent_val_list))
# print(len(df4_clau_one_etri_sent_val_list_str)) # 464 richtig

df2_root = df2[df2["verb_syn"] == 'root'] 
# print(df2_root.shape) # (12783, 23)

df2_root_clau_one = df2_root[df2_root.etri_sent.isin(df4_clau_one_etri_sent_val_list_str)]
# print(df2_root_clau_one.shape) # (431, 23) 464 중 431은 92,89%

# df2_root_clau_one.to_csv("df2_root_clau_one_020423.csv", encoding = "utf-8-sig") 

df2_root_clau_one_etri_sent = df2_root_clau_one.etri_sent
df2_root_clau_one_etri_sent_val = df2_root_clau_one_etri_sent.values 
df2_root_clau_one_etri_sent_val_list = df2_root_clau_one_etri_sent_val.tolist() 

# print(len(df2_root_clau_one_etri_sent_val_list)) # 431 ricthig, type: str

# print(len(set(df4_clau_one_etri_sent_val_list_str))) # 464 richtig
# print(len(set(df2_root_clau_one_etri_sent_val_list))) # 431 richtig

complement = list(set(df4_clau_one_etri_sent_val_list_str) - set(df2_root_clau_one_etri_sent_val_list)) # type: str
# print(len(complement)) # 33 richtig

# 464에서 431 뺀 33 케이스가 explicit한 Subjekt가 있을 확률이 높음 
# 따로 df로 만들어서 확인 

# df2_root_explicit_sbj_check = df2_root[df2_root.etri_sent.isin(complement)]
# print(df2_root_explicit_sbj_check.shape)

df2_explicit_sbj_check = df2[df2.etri_sent.isin(complement)]
# print(df2_explicit_sbj_check.shape) # (35, 23) 왜 33 아니지?? df 만들어서 확

df2_explicit_sbj_check_sent_val = df2_explicit_sbj_check['etri_sent'].value_counts()
# print(df2_explicit_sbj_check_sent_val)

# 15292    2
# 18468    2
# 이 두 개 확인해보기 (etri_sent가 두 개 있는 케이스)

# 35개 중에서도 'sent_dep'에 subj 있는 것만 체크하면 18개 030423
# 실제로 16개임 (subj_yes: 1) / 18468의 경우 root의 주어가 실제로 있는 것이 아님
# 16개 중 4개만 S2의 explicit한 주어를 공유함

# 나머지 19개 (subj_yes: 0) 15292와 18468 포함 
# 실제로 17개로 쳐야 함
# 따라서 전체는 16 + 17 = 33 richtig
# 17개 중에는 9개만 주어를 공유함 

# print(len(set(complement))) # 33

# df2_explicit_sbj_check.to_csv("df2_explicit_sbj_check_020423.csv", encoding = "utf-8-sig") 


In [40]:
# S2의 Subjekt가 explcit하지 않는 경우, 만약 이들이 Resolutionsystem의 input이라면 규칙에 의해 할당한 referent를 복사하는 게 가능할 듯
# 이것이 몇 퍼센트가 되는지 확인해야 함

# input이 아닌 경우 manuell하게 S1과 S2의 주어가 같은지 확인해 봐야 할 듯

# 431개는 S1과 S2 둘 다 Nullsubjekt가 있을 확률이 높으므로, 둘의 zero_referents, antecedent를 비교해봐야 함

# for element in df2_root_clau_one_etri_sent_val_list:
#     print(type(element)) # list의 요소 type 'str'

# print(df4_clau_one.dtypes) # etri_sent type 'int64'

df2_root_clau_one_etri_sent_val_list_int = [int (i) for i in df2_root_clau_one_etri_sent_val_list] 
# print(len(df2_root_clau_one_etri_sent_val_list_int)) # 431 richtig

# df4_clau_one에서 df2_root_clau_one_etri_sent_val_list에 속하는 case의 df 우선 만들어 놓기, sollte 431 / 030423
# df4_clau_one_431

df4_clau_one_431 = df4_clau_one[df4_clau_one.etri_sent.isin(df2_root_clau_one_etri_sent_val_list_int)]
# print(df4_clau_one_431.shape) # (431, 21) richtig

# df2_root_clau_one에서 root의 verb, 주어, zero_referents, antencedents 추가하기 (기존 코드 활용) 030423
# 근데 subject_by_rule 아니고, zero_referents랑 antecedents 불러와서 이 것을 비교할 것임

# print(df2_root_clau_one.shape) # (431, 23)

df2_root_clau_one_zero_referents = df2_root_clau_one.zero_referents
df2_root_clau_one_zero_referents_val = df2_root_clau_one_zero_referents.values 
df2_root_clau_one_zero_referents_val_list = df2_root_clau_one_zero_referents_val.tolist() 
# print(len(df2_root_clau_one_zero_referents_val_list)) # None 포함되어 있음 / 431

df2_root_clau_one_antecedent = df2_root_clau_one.antecedent
df2_root_clau_one_antecedent_val = df2_root_clau_one_antecedent.values 
df2_root_clau_one_antecedent_val_list = df2_root_clau_one_antecedent_val.tolist() 
# pint(len(df2_root_clau_one_antecedent_val_list)) # None 포함되어 있음 / 431

df2_root_clau_one_verb = df2_root_clau_one.verb
df2_root_clau_one_verb_val = df2_root_clau_one_verb.values 
df2_root_clau_one_verb_val_list = df2_root_clau_one_verb_val.tolist() 
# print(len(df2_root_clau_one_verb_val_list)) # 431

df2_root_clau_one_zero_referents_dict = dict(zip(df2_root_clau_one_etri_sent_val_list_int, df2_root_clau_one_zero_referents_val_list)) 
# print(len(df2_root_clau_one_zero_referents_dict)) # 431 richtig 

df2_root_clau_one_antecedent_dict = dict(zip(df2_root_clau_one_etri_sent_val_list_int, df2_root_clau_one_antecedent_val_list)) 
# print(len(df2_root_clau_one_antecedent_dict)) # 431 richtig 

df2_root_clau_one_verb_dict = dict(zip(df2_root_clau_one_etri_sent_val_list_int, df2_root_clau_one_verb_val_list)) 
# print(len(df2_root_clau_one_verb_dict)) # 431 richtig 


# df4_clau_one_431에 root에 있던 정보인 zero_referents, antecedent, verb copy하기 030423

df2_root_clau_one_zero_referents_keys = list(df2_root_clau_one_zero_referents_dict.keys())

copied_zero_referents = [] 

for sent in df4_clau_one_431['etri_sent']:
    if sent in df2_root_clau_one_zero_referents_keys:
        copied_zero_referents.append(df2_root_clau_one_zero_referents_dict[sent])
    
# print(len(copied_zero_referents)) # 431 richtig

df2_root_clau_one_antecedent_keys = list(df2_root_clau_one_antecedent_dict.keys())

copied_antecedent = [] 

for sent in df4_clau_one_431['etri_sent']:
    if sent in df2_root_clau_one_antecedent_keys:
        copied_antecedent.append(df2_root_clau_one_antecedent_dict[sent])
    
# print(len(copied_antecedent)) # 431 richtig

df2_root_clau_one_verb_keys = list(df2_root_clau_one_verb_dict.keys())

copied_verb = [] 

for sent in df4_clau_one_431['etri_sent']:
    if sent in df2_root_clau_one_verb_keys:
        copied_verb.append(df2_root_clau_one_verb_dict[sent])
    
# print(len(copied_verb)) # 431 richtig


# df4_clau_one_431의 열이 늘어나야 함! (431, 21) - 22 - 23 - 24           
df4_clau_one_431_zero_referents = df4_clau_one_431.assign(copied_zero_referents = copied_zero_referents) 
df4_clau_one_431_antecedent = df4_clau_one_431_zero_referents.assign(copied_antecedent = copied_antecedent) 
df4_clau_one_431_verb = df4_clau_one_431_antecedent.assign(copied_verb = copied_verb) 

# print(df4_clau_one_431_zero_referents.shape) # (431, 22) richtig
# print(df4_clau_one_431_antecedent.shape) # (431, 23) richtig
# print(df4_clau_one_431_verb.shape) # (431, 24) richtig


# df4_clau_one_431_verb.to_csv("df4_clau_one_431_copied_done_030423.csv", encoding = "utf-8-sig") 



In [41]:
# 1. 첫 번째로 할 것은 우선 zero_referents가 simple하게 같은 case가 아마 대다수, 이걸 알아내고, 나머지 제외한 거 어떻게 implement할 건지 확인할 수 있음 030423

# case 1: zero_referents == copied_zero_referents

df4_clau_one_431_verb['match'] = df4_clau_one_431_verb['etri_sent'][(df4_clau_one_431_verb['zero_referents'] == df4_clau_one_431_verb['copied_zero_referents'])]
    

# df4_clau_one_431_verb.to_csv("df4_clau_one_431_string_match_check_030423.csv", encoding = "utf-8-sig") 

# 431개 중 412개 zero_referents string match 됨 (95,59%)

# 나머지 19개 maneull check

In [86]:
# conjunction 관련 implement 050423

# 1. 우선 두 개의 csv 불러오기, 왜냐하면 manuell하게 추가된 열들이 존재하기 때문
# 1.1. "df4_clau_one_431_string_match_check_030423.csv" # 33개
# 1.2. "df2_explicit_sbj_check_020423.csv" # 431개

data_1 = pd.read_csv("df4_clau_one_431_string_match_check_030423.csv", index_col=0)
# print(data_1.shape) # (431, 26)

data_2 = pd.read_csv("df2_explicit_sbj_check_020423.csv", index_col=0)
# print(data_2.shape) # (33, 25) 


,etri_sent,sent,verb,share_yes,applied_rule,subject_by_rule,zero_referents,antecedent,group,rule_order,rule,connective,base_morpheme,base_morpheme_second,connective_true,zp_hono,zp_mood,sent_dep,subj_yes,zp_markable_ID,zero_type,tense_zp,verb_syn,verb_head,Filter
16250,6198,아래층에서 담배를 펴서 창문으로 연기가 들어옵니다 .,펴서,0,NaN,NaN,사람,NaN,NaN,NaN,NaN,class_A,펴,(아/어)서,1,NaN,NaN,"['modi', 'dobj', 'clau', 'modi', 'subj', 'root...",1,markable_4760,extra,NaN,clau,6,C
15871,5480,가끔 운송 시 내부 충격을 받아서 이런 일이 발생하기도합니다 .,받아서,0,NaN,NaN,NaN,음반을,NaN,NaN,NaN,class_A,받,(아/어)서,1,NaN,NaN,"['modi', 'modi', 'modi', 'modi', 'dobj', 'clau...",1,markable_5176,inter,NaN,clau,9,C
2094,10605,어제 너무 많이 먹어서 소화가 안됩니다 .,먹어서,0,NaN,NaN,1sh+,NaN,NaN,NaN,NaN,class_A,먹,(아/어)서,1,NaN,NaN,"['modi', 'modi', 'modi', 'clau', 'subj', 'root...",1,markable_4910,extra,NaN,clau,6,C
1657,11391,지금 요리를 다 치워서 확인이 안되는데 .,치워서,0,NaN,NaN,1sh+,NaN,NaN,NaN,NaN,class_A,치우,(아/어)서,1,NaN,NaN,"['modi', 'dobj', 'modi', 'clau', 'subj', 'root...",1,markable_5475,extra,NaN,clau,6,C
790,10170,술을 마셔서그런지 기분이 좋군요 .,마셔서그런지,0,NaN,NaN,1sh+,NaN,NaN,NaN,NaN,class_A,마시,(아/어)서,0,NaN,NaN,"['dobj', 'clau', 'subj', 'root', 'punc']",1,markable_5266,extra,NaN,clau,4,C
6523,17891,"네, 아침부터 팔려서 지금 딱 세 개 상품이 남았어요 .",팔려서,1,NaN,NaN,NaN,쥬얼리,NaN,NaN,NaN,class_A,팔리,(아/어)서,1,NaN,NaN,"['modi', 'modi', 'clau', 'modi', 'modi', 'modi...",1,markable_4664,inter,NaN,clau,9,C
13332,29532,등산 중에 발을 다쳐서 친구가 못 움직이고있습니다 .,다쳐서,1,NaN,NaN,친구,NaN,NaN,NaN,NaN,class_A,다치,(아/어)서,1,NaN,NaN,"['modi', 'modi', 'dobj', 'clau', 'subj', 'modi...",1,markable_5110,extra,NaN,clau,7,C
6333,18468,잠시 저쪽에 가서 약을 발라도 괜찮을까요 ?,가서,1,NaN,NaN,2sh+,NaN,NaN,NaN,NaN,class_A,가,(아/어)서,1,NaN,NaN,"['modi', 'modi', 'clau', 'dobj', 'subj', 'root...",0,markable_5100,extra,NaN,clau,6,C
21,261,방해해서 죄송합니다 .,방해해서,1,NaN,NaN,1sh+,NaN,NaN,NaN,NaN,class_A,방해하,(아/어)서,1,NaN,NaN,"['clau', 'root', 'punc']",0,markable_5136,extra,NaN,clau,2,C
3483,12909,친절하신 분 만나서 기분좋네요 .,만나서,1,NaN,NaN,1sh+,NaN,NaN,NaN,NaN,class_A,만나,(아/어)서,1,NaN,NaN,"['modi', 'dobj', 'clau', 'root', 'punc']",0,markable_5082,extra,NaN,clau,4,C


In [89]:
# 2. 1.2에 Fall 1(16개, 'subj_yes': 1)과  Fall 2(17개, 'subj_yes': 0)가 섞여 있음

data_2_Fall_1 = data_2[data_2['subj_yes'] == 1]
data_2_Fall_2 = data_2[data_2['subj_yes'] == 0]

# print(data_2_Fall_1.shape) # (16, 25) richtig
# print(data_2_Fall_2.shape) # (17, 25) richtig

# Fall 1 (data_2)에서 'share_yes'를 기준으로 'base_morpheme_second'의 value_counts

Fall_1_share_yes = data_2_Fall_1[data_2_Fall_1['share_yes'] == 1]
# print(Fall_1_share_yes.shape) # (4, 25) richtig

Fall_1_share_no = data_2_Fall_1[data_2_Fall_1['share_yes'] == 0]
# print(Fall_1_share_no.shape) # (12, 25) richtig


# Fall 2 (data_2) 에서 'share_yes'를 기준으로 'base_morpheme_second'의 value_counts

data_2_Fall_2_share_yes = data_2_Fall_2[data_2_Fall_2['share_yes'] == 1]
# print(data_2_Fall_2_share_yes.shape) # (9, 25) # richtig

data_2_Fall_2_share_no = data_2_Fall_2[data_2_Fall_2['share_yes'] == 0]
# print(data_2_Fall_2_share_no.shape) # (8, 25) richtig

# data_2 전체: 4 + 12 + 9 + 8 = 33 richtig

# Fall 2 (data_1) 에서 'share_yes'를 기준으로 'base_morpheme_second'의 value_counts

data_1_Fall_2_share_yes = data_1[data_1['share_yes'] == 1]
# print(data_1_Fall_2_share_yes.shape) # (423, 26) # richtig

data_1_Fall_2_share_no = data_1[data_1['share_yes'] == 0]
# print(data_1_Fall_2_share_no.shape) # (8, 26) # richtig

# data_1 전체: 423 + 8 = 431 richtig

# Fall 1와 Fall 2 (data_1 + data_2)에서 'share_yes'를 기준으로 'base_morpheme_second'의 value_counts (meeting 끝나고 산책하고 여기 다시 시작 050423)

# 'base_morpheme_second'가 맞는지 확인해서 아닌 케이스 수정해야함
# 같이 묶을 수 있는 케이스도 묶어야 함 


# 전체 1. (data_2, Fall_1)
Fall_1_connective = data_2_Fall_1['base_morpheme_second'].value_counts()
# print(Fall_1_connective) # 전체 16 richtig

Fall_1_share_yes_connective = Fall_1_share_yes['base_morpheme_second'].value_counts() 
# print(Fall_1_share_yes_connective)

Fall_1_share_no_connective = Fall_1_share_no['base_morpheme_second'].value_counts() 
# print(Fall_1_share_no_connective)

# Fall_1_connective.to_csv("Fall_1_connective_050423.csv", encoding = "utf-8-sig") 
# Fall_1_share_yes_connective.to_csv("Fall_1_share_yes_connective_050423.csv", encoding = "utf-8-sig") 
# Fall_1_share_no_connective.to_csv("Fall_1_share_no_connective_050423.csv", encoding = "utf-8-sig") 


# 전체 2. (data_2, Fall_2)
data_2_Fall_2_connective = data_2_Fall_2['base_morpheme_second'].value_counts()
# print(data_2_Fall_2_connective) # 전체 17 richtig

data_2_Fall_2_share_yes_connective = data_2_Fall_2_share_yes['base_morpheme_second'].value_counts() 
# print(data_2_Fall_2_share_yes_connective)

data_2_Fall_2_share_no_connective = data_2_Fall_2_share_no['base_morpheme_second'].value_counts() 
# print(data_2_Fall_2_share_no_connective)

# data_2_Fall_2_connective.to_csv("data_2_Fall_2_connective.csv", encoding = "utf-8-sig") 
# data_2_Fall_2_share_yes_connective.to_csv("data_2_Fall_2_share_yes_connective_050423.csv", encoding = "utf-8-sig") 
# data_2_Fall_2_share_no_connective.to_csv("data_2_Fall_2_share_no_connective.csv", encoding = "utf-8-sig") 

# 전체 3. (data_1, Fall_2)
data_1_Fall_2_connective = data_1['base_morpheme_second'].value_counts()
# print(data_1_Fall_2_connective) 

data_1_Fall_2_share_yes_connective = data_1_Fall_2_share_yes['base_morpheme_second'].value_counts() 
# print(data_1_Fall_2_share_yes_connective)

data_1_Fall_2_share_no_connective = data_1_Fall_2_share_no['base_morpheme_second'].value_counts() 
# print(data_1_Fall_2_share_no_connective)

# data_1_Fall_2_connective.to_csv("data_1_Fall_2_connective.csv", encoding = "utf-8-sig") 
# data_1_Fall_2_share_yes_connective.to_csv("data_1_Fall_2_share_yes_connective_050423.csv", encoding = "utf-8-sig") 
# data_1_Fall_2_share_no_connective.to_csv("data_1_Fall_2_share_no_connective.csv", encoding = "utf-8-sig") 




# Fall_1_connective = pd.DataFrame(data_2_Fall_1_freq)
# Fall_1_connective = data_2_Fall_1_freq.rename_axis('connective').reset_index(name='counts')


In [313]:
## 1. classA_clue_check에서 우선 해당 문장의 clau가 몇 개인지 살펴보고 clau가 하나인 경우 root의 주어 복사 가능할 것 같음 290323
## 우선 이 case가 몇 개인지 확인해보기

clau_freq = clau_count_add.groupby(['clau']).count()  

# 전체 594 중

# clau  etri_sent
#  1: 365
#  2: 173     
#  3: 44
#  4: 9
#  5: 3

clau_freq_new = clau_count_add['clau'].value_counts()  # hmm...

# clau_freq_new

clau_count_add


# clau_freq

# clau_freq.to_csv("clau_count_add_groupby_290323.csv", encoding = "utf-8-sig") 

,etri_sent,sent,verb,applied_rule,zero_referents,antecedent,rule_order,connective,base_morpheme,base_morpheme_second,connective_true,zp_hono,zp_mood,sent_dep,zp_markable_ID,zero_type,tense_zp,verb_syn,verb_head,Filter,clau
66,136,"네, 세탁실에 가져다놓으시면 내일아침까지 세탁해서 드릴거예요 .",세탁해서,NaN,1ph+,NaN,NaN,class_A,세탁하,아서,1,NaN,NaN,"['modi', 'modi', 'clau', 'modi', 'clau', 'root...",markable_5026,extra,NaN,clau,6,C,2
105,434,주차장입구의 오른쪽은 C 구역 이니 그쪽에 가서 찾아보세요 .,가서,NaN,2sh+,NaN,NaN,class_A,가,서,1,NaN,NaN,"['modi', 'dobj', 'modi', 'scom', 'clau', 'modi...",markable_5278,extra,NaN,clau,8,C,2
144,452,차안에 열쇠를 두고 내렸는데 어떻게 하죠 ?,두고,NaN,1sh+,NaN,NaN,class_A,두,고,1,NaN,NaN,"['modi', 'dobj', 'clau', 'clau', 'modi', 'root...",markable_5287,extra,NaN,clau,4,C,2
145,164,저희 축제의 전통의상을 입고 촬영할수있어요 .,입고,NaN,2sh+,NaN,NaN,class_A,입,고,1,NaN,NaN,"['modi', 'modi', 'dobj', 'clau', 'root', 'punc']",markable_5046,extra,NaN,clau,5,C,1
179,730,다시 먹어보고 맞지않으면 다시 부르겠습니다 .,먹어보고,NaN,1sh+,NaN,NaN,class_A,먹어보,고,1,NaN,NaN,"['modi', 'clau', 'clau', 'modi', 'root', 'punc']",markable_5458,extra,NaN,clau,3,C,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17103,8258,그럼 튀김을 넣어서 주세요 .,넣어서,NaN,2sh+,NaN,NaN,class_A,넣,어서,1,NaN,NaN,"['modi', 'dobj', 'clau', 'root', 'punc']",markable_4972,extra,NaN,clau,4,C,1
17145,8195,사진을 현상하러 오셨나요 ?,현상하러,NaN,2sh+,NaN,NaN,class_A,현상하,러,1,NaN,NaN,"['dobj', 'clau', 'root', 'punc']",markable_4928,extra,NaN,clau,3,C,1
17157,8214,여길 다 둘러보고 다른 곳으로 가야겠네요 .,둘러보고,NaN,1sh+,NaN,NaN,class_A,둘러보,고,1,NaN,NaN,"['dobj', 'modi', 'clau', 'modi', 'modi', 'root...",markable_4939,extra,NaN,clau,6,C,1
17217,8836,에스컬레이터를 타고 2 층으로 가시면 오른쪽에 있습니다 .,타고,NaN,2sh+,NaN,NaN,class_A,타,고,1,NaN,NaN,"['dobj', 'clau', 'modi', 'modi', 'clau', 'modi...",markable_5291,extra,NaN,clau,5,C,2


In [222]:
## 2. clau가 1개로 count되는 365개의 case에 대해서 (594개의 61,45%)우선 resolutionsystem에서 이미 복원된 주어를 어떻게 복사할지 구현하기 290323
## 우선 etir_sent가 중복되지 않는다면 일이 더 쉬워지므로 etri_sent 하나만 있는 case와 clau가 1인 case의 intersection을 살펴보기! 290323

clau_count_add_counter = {}  
for value in classA_etri_sent_val_list:
    try: clau_count_add_counter[value] += 1
    except: clau_count_add_counter[value] = 1
        
# print(len(clau_count_add_counter)) # 575! 594개 중 중복된 항목 제거하면 575 richtig

classA_etri_sent_over_two = [] # 18 (etri_sent가 중복되는 classA)
classA_etri_sent_one = [] # 557 (etri_sent가 하나만 있는 classA)

for key, value in clau_count_add_counter.items():
    if value >= 2:
        classA_etri_sent_over_two.append(key)
    if value == 1:
        classA_etri_sent_one.append(key)
        
# print(len(classA_etri_sent_over_two)) # 18
# print(len(classA_etri_sent_one)) # 557


18
557


In [233]:
# 1. clau가 1인 etri_sent # (365, 22)

clau_count_one = clau_count_add[clau_count_add.clau == 1] 
# print(clau_count_one.shape) # (365, 22) richtig

clau_one_etri_sent = clau_count_one.etri_sent
clau_one_etri_sent_val = clau_one_etri_sent.values 
clau_one_etri_sent_val_list = clau_one_etri_sent_val.tolist() 

# print(len(clau_one_etri_sent_val_list)) # 365

intersection_etri_sent_clau_one = list(set(classA_etri_sent_one) & set(clau_one_etri_sent_val_list)) 

# print(len(intersection_etri_sent_clau_one)) # 365

simple_case = clau_count_add.etri_sent.isin(intersection_etri_sent_clau_one)
clau_count_add = clau_count_add.assign(simple = simple_case) # (683, 17)

# print(clau_count_add.shape) # (594, 22)

# 2. clau가 2 이상인 etri_sent # (229, 22)

clau_count_not_one = clau_count_add[clau_count_add.clau != 1] 

# print(clau_count_not_one.shape) # (229, 22) 594 - 365 = 229 richtig


# 실제로 규칙이 제안하는 referent가 맞지만 extra가 아니라서 직접적으로 정답이 비교 안되는 case (case3)
# df3_not_case1_not_singplural.xlsx에서 referent_correct sheet 

referent_correct = pd.read_excel('case2A_B_C_11052022.xlsx', sheet_name = 'referent_correct') 
referent_correct_idx = referent_correct['idx']
referent_correct_idx_val = referent_correct_idx.values 
referent_correct_idx_val_list = referent_correct_idx_val.tolist() # 124 --> case3 list 요소의 type float (list 값이 nan으로 나옴)

# print(len(referent_correct_idx_val_list)) # 1241인칭 right prediction

df2_exclude_case3 = df2_exclude_orig[df2_exclude_orig.index.isin(referent_correct_idx_val_list) == True]
# print(df2_exclude_case3.shape) # (120, 23)

# clau_count_add.to_csv("clau_count_add_2_290323.csv", encoding = "utf-8-sig") # 290323
# simple 열 추가됨: clau 1이면 True, clau 2 이상이면 False

(594, 22)

In [258]:
# 3. intersection_etri_sent_clau_one를 지닌 case를 df2_FilterA_B에서 찾아서 subject_by_rule 정보 가져오기!!

df2_FilterA_B_intersection_etri_sent = df2_FilterA_B[df2_FilterA_B.etri_sent.isin(intersection_etri_sent_clau_one) == True] 
# print(df2_FilterA_B_intersection_etri_sent.shape) # (367, 23)

df2_FilterA_B_intersection_subject_by_rule = df2_FilterA_B_intersection_etri_sent.subject_by_rule
# df2_FilterA_B_intersection_subject_by_rule은 subject_by_rule이 index와 함께 있는 자료 e.g 31 2sh+ (이 경우 etri_sent는 255임)

clau_count_one_intersection_etri_sent = clau_count_one[clau_count_one.etri_sent.isin(intersection_etri_sent_clau_one) == True] 
# print(clau_count_one_intersection_etri_sent.shape) # (365, 22)

# df2_FilterA_B_intersection_etri_sent.to_csv("df2_FilterA_B_intersection_etri_sent_290323.csv", encoding = "utf-8-sig") 
# clau_count_one_intersection_etri_sent.to_csv("clau_count_one__intersection_etri_sent_290323.csv", encoding = "utf-8-sig") 

## 이 두 개가 차이가 나는 etri_sent 찾기 

df2_FilterA_B_intersection_etri_sent = df2_FilterA_B_intersection_etri_sent.etri_sent
df2_FilterA_B_intersection_etri_sent_val = df2_FilterA_B_intersection_etri_sent.values 
df2_FilterA_B_intersection_etri_sent_val_list = df2_FilterA_B_intersection_etri_sent_val.tolist()

# print(len(df2_FilterA_B_intersection_etri_sent_val_list)) # 367

clau_count_one_intersection_etri_sent = clau_count_one_intersection_etri_sent.etri_sent
clau_count_one_intersection_etri_sent_val = clau_count_one_intersection_etri_sent.values 
clau_count_one_intersection_etri_sent_val_list = clau_count_one_intersection_etri_sent_val.tolist()

# print(len(clau_count_one_intersection_etri_sent_val_list)) # 365

# df2_FilterA_B_intersection_etri_sent_val_list 이 리스트에서 중복되는 값이 있는지 찾기)

from collections import Counter

counter = Counter(df2_FilterA_B_intersection_etri_sent_val_list)

counter_dict = dict(counter)

# for key, value in counter_dict.items():
#     if value == 2:
#         print(key)

# 1. "df2_FilterA_B_intersection_etri_sent_290323.csv"에서는

# 13661 보통 1인분 *주문하시고* 밥을 주문해서 *드시거든요*.
# 27222 일단 방송으로 보내고 경비원들 *풀어서* *찾아보겠습니다*.

# 2. "clau_count_one__intersection_etri_sent_290323.csv"에서는

# 13661 보통 1인분 주문하시고 밥을 *주문해서* 드시거든요. (index: 3564)
# 27222 일단 방송으로 *보내고* 경비원들 풀어서 찾아보겠습니다. (index: 12272)

# 이 두 개를 제외해야 하는 거 아닌가? clau가 1이 아님

# print(len(set(df2_FilterA_B_intersection_etri_sent_val_list))) # 365
# print(len(set(clau_count_one_intersection_etri_sent_val_list))) # 365

etri_sent_duplicate_set = {13661, 27222}


In [329]:
# etri_sent_duplicate을 제외하고는 진짜 simple하게 복사할 수 있는 듯 (dict 사용도 가능 아니면 직접적으로 df 합치기도 가능할 듯) 300323

etri_sent_remove_list = [i for i in df2_FilterA_B_intersection_etri_sent_val_list if i not in etri_sent_duplicate_set] # str이 아니라 integer인듯 300323
# print(len(etri_sent_remove_list)) # 363 rightig (367 - 4) 중복되는 두 개의 etri_sent에 속하는 문장은 총 4개 

# 양쪽에서 우선 etri_sent_duplicate만 제외하고 subject_by_rule dict로 가지고 있기! (e.g. {etri_sent: subject_by_rule})

# df2_FilterA_B에서 etri_sent_remove_list를 지닌 subject_by_rule을 dict로 만들어보기

df2_FilterA_B_etri_sent = df2_FilterA_B[df2_FilterA_B.etri_sent.isin(etri_sent_remove_list)]
# print(df2_FilterA_B_etri_sent.shape) # (363, 23) richtig

df2_FilterA_B_subject_by_rule = df2_FilterA_B_etri_sent.subject_by_rule
df2_FilterA_B_subject_by_rule_val = df2_FilterA_B_subject_by_rule.values 
df2_FilterA_B_subject_by_rule_val_list = df2_FilterA_B_subject_by_rule_val.tolist() 
# print(len(df2_FilterA_B_subject_by_rule_val_list)) # 363 richtig

df2_FilterA_B_verb = df2_FilterA_B_etri_sent.verb
df2_FilterA_B_verb_val = df2_FilterA_B_verb.values 
df2_FilterA_B_verb_val_list = df2_FilterA_B_verb_val.tolist() 
# print(len(df2_FilterA_B_verb_val_list)) # 363 richtig

subject_by_rule_dict = dict(zip(etri_sent_remove_list, df2_FilterA_B_subject_by_rule_val_list)) 
# print(len(subject_by_rule_dict)) # 363 richtig (원래의 코드와 달리 dictinary update하지 않음) 300323

verb_dict = dict(zip(etri_sent_remove_list, df2_FilterA_B_verb_val_list)) 
# print(len(verb_dict)) # 363 richtig (원래의 코드와 달리 dictinary update하지 않음) 300323


{164: '촬영할수있어요',
 365: '드릴게요',
 737: '환승하셔야합니다',
 744: '지워주세요',
 975: '앉으면되나요',
 252: '열어드리겠습니다',
 1056: '올라가봐야겠어요',
 868: '오면됩니까',
 476: '처리해드릴게요',
 352: '연락드리겠습니다',
 1030: '구경할수있습니다',
 640: '타야하나요',
 97: '가야하나요',
 9: '가겠습니다',
 207: '갈게요',
 461: '연락해드릴게요',
 334: '계산해도되나요',
 10153: '먹고싶어요',
 9757: '기다리시겠어요',
 9975: '기다려주세요',
 9782: '기다리세요',
 10174: '가봐야겠네요',
 9799: '말씀해드릴게요',
 9917: '계산해주세요',
 10130: '이용해야겠네요',
 10223: '가져다드리겠습니다',
 10393: '신기해요',
 10237: '기다리세요',
 10395: '생각해볼게요',
 9743: '기다리시겠어요',
 9395: '검사를해주어야합니다',
 9494: '가야하나요',
 9439: '타야하나요',
 9445: '가야하나요',
 9467: '가야하나요',
 10056: '해야하죠',
 10058: '구입해야겠네요',
 10071: '기다려주세요',
 10893: '담으시면됩니다',
 11079: '수리해드리도록하겠습니다',
 11354: '올라가보도록하겠습니다',
 11003: '들어갈수있겠어요',
 10867: '검색할수있습니다',
 11469: '가려고해요',
 11128: '가야해요',
 10446: '제출해주세요',
 11292: '주세요',
 11460: '결정할게요',
 10708: '시간되세요',
 10502: '주나요',
 10832: '가야하나요',
 11657: '모르겠습니다',
 11658: '여쭤보세요',
 11931: '이동하실수있습니다',
 11927: '이동하십시오',
 11943: '들어있어야되나요',
 12394: '갈게요',
 11777: '가

In [290]:
# 이제 clau_count_one (365, 22)에 copied subject_by_rule 추가해주기!
# 여기에서도 우선 총 2개의 문장 제외 돼야 하는 거 아닌가? etri_sent가 13661, 27222인 케이스들

# clau_count_one 자체에서도 2문장 제외시켜야 함 

etri_sent_duplicate_list = [13661, 27222]

clau_count_one_new = clau_count_one[clau_count_one.etri_sent.isin(etri_sent_duplicate_list) == False]
#print(clau_count_one.shape) # (365, 22) richtig
#print(clau_count_one_new.shape) # (363, 22) richtig

etri_sent_remove_list2 = [i for i in clau_count_one_intersection_etri_sent_val_list if i not in etri_sent_duplicate_set] 
# print(len(etri_sent_remove_list2)) # 363 richtig (365 - 2)

clau_one_etri_sent = clau_count_one_new[clau_count_one_new.etri_sent.isin(etri_sent_remove_list2)]
# print(clau_one_etri_sent.shape) # (363, 22) richtig

subject_by_rule_keys = list(subject_by_rule_dict.keys())

copied_subject_by_rule = [] 

for sent in clau_count_one_new['etri_sent']:
    if sent in subject_by_rule_keys:
        copied_subject_by_rule.append(subject_by_rule_dict[sent])
    
# print(len(copied_subject_by_rule)) # 363

copied_verb = []

for sent in clau_count_one_new['etri_sent']:
    if sent in subject_by_rule_keys:
        copied_verb.append(verb_dict[sent])

# print(len(copied_verb)) # 363

# 열이 늘어나야 함! 22 - 23 - 24             
clau_count_one_add_subject_by_rule = clau_count_one_new.assign(copied_subject_by_rule = copied_subject_by_rule) 
clau_count_one_add_subject_and_verb = clau_count_one_add_subject_by_rule.assign(copied_verb = copied_verb) # (432, 19)

# print(clau_count_one_add_subject_by_rule.shape) # (363, 23)
# print(clau_count_one_add_subject_and_verb.shape) # (363, 24) richtig


# clau_count_one_add_subject_and_verb.to_csv("clau_count_one_add_subject_and_verb_300323.csv", encoding = "utf-8-sig") 


In [ ]:
# 나중에 copied_subject_by_rule과 zero_referents 비교할 때 string으로 되어 있는 antecedent도 있고, sing/plural도 고려해야 함


In [ ]:
# error case 1: 규칙 groupA가 잘 적용되지 않아서 copied subject_by_rule가 틀린 경우 (규칙 group A를 더 최적화 해야함 - 완료)
# error case 2: 규칙 groupA가 제시하는 copied subject_by_rule이  틀린 경우 
# error case 3: class A에 속하지만 subject sharing property가 적용될 수 없는 connective (대부분 너무 용법이 많아서)
# error case 4: sing <-> plural이 애매한 경우 (사실은 정답이 될 수 있음)
 ## e.g.
 # 6977	18887
 # 7230	19044
 # 16498	6124
 # 12976	28027
 # 16251	6393
 ## e.g.15471	5720	어제밤에 뉴왁 공항에 내렸고 오늘은 나이아가라폭포를 봤습니다 .	내렸고	봤습니다	1sh+		{우리는}
# error case 5: 직접적으로 연결된 절 아님 
 ## e.g. 3923	14684	넘어져서 무릎을 다쳤는데요, 약을 좀 주십시오 .	넘어져서	주십시오
 ## e.g. 7134	19390	매콤하고 새콤합니다 . 한국사람 입에도 맞을거같아요 .	매콤하고	맞을거같아요
# error case 6: zero_referents 외의 경험자가 주어가 될 수 있는 경우
 ## e.g. 10639	24940	생각보다 덥지도않고 즐거웠어요 .	덥지도않고	즐거웠어요	1sh+	날씨
 ## e.g. 7862	20132	죄송한데요, 좀 작은거같아서 불편하네요 .	작은거같아서	불편하네요	1sh+	사이즈
# error case 7: antecedent 외의 경험자가 주어가 될 수 있는 경우
 ## e.g. 1632	11029	편하고 좋습니다 .	편하고	좋습니다	1sh+		{치마가}


In [31]:
# Fall 1: group value_counts (ranking 1-3)
want_statement_Fall_one = df2_case1[df2_case1["group"] == "want statement"]
want_statement_Fall_one_value_count = want_statement_Fall_one["rule"].value_counts() 
# want_statement_Fall_one_value_count.to_csv("want_statement_Fall_one_value_count_260922.csv", encoding = "utf-8-sig") 

mood_derivable_Fall_one = df2_case1[df2_case1["group"] == "mood derivable"]
mood_derivable_Fall_one_value_count = mood_derivable_Fall_one["rule"].value_counts() 
# mood_derivable_Fall_one_value_count.to_csv("mood_derivable_Fall_one_value_count_260922.csv", encoding = "utf-8-sig") 

clau_Fall_one = df2_case1[df2_case1["group"] == "clau"]
clau_Fall_one_value_count = clau_Fall_one["rule"].value_counts() 
# clau_Fall_one_value_count.to_csv("clau_Fall_one_value_count_260922.csv", encoding = "utf-8-sig") 

In [36]:
# Fall 2: Person match (ranking 1-3)
# Fall 2.1 first person match
politeness_Fall_two = df2_first_person_match[df2_first_person_match["group"] == "politeness"]
politeness_Fall_two_value_count = politeness_Fall_two["rule"].value_counts() 
# politeness_Fall_two_value_count.to_csv("politeness_Fall_two_value_count_260922.csv", encoding = "utf-8-sig") 

want_statement_Fall_two = df2_first_person_match[df2_first_person_match["group"] == "want statement"]
want_statement_Fall_two_value_count = want_statement_Fall_two["rule"].value_counts() 
# want_statement_Fall_two_value_count.to_csv("want_statement_Fall_two_value_count_260922.csv", encoding = "utf-8-sig") 

clau_Fall_two = df2_first_person_match[df2_first_person_match["group"] == "clau"]
clau_Fall_two_value_count = clau_Fall_two["rule"].value_counts() 
# clau_Fall_two_value_count.to_csv("clau_Fall_two_value_count_260922.csv", encoding = "utf-8-sig") 

# # Fall 2.2 second person match
mood_derivable_Fall_two_two = df2_second_person_match[df2_second_person_match["group"] == "mood derivable"]
mood_derivable_Fall_two_two_value_count = mood_derivable_Fall_two_two["rule"].value_counts() 
# mood_derivable_Fall_two_two_value_count.to_csv("mood_derivable_Fall_two_two_value_count_260922.csv", encoding = "utf-8-sig") 

clau_Fall_two_two = df2_second_person_match[df2_second_person_match["group"] == "clau"]
clau_Fall_two_two_value_count = clau_Fall_two_two["rule"].value_counts() 
# clau_Fall_two_two_value_count.to_csv("clau_Fall_two_two_value_count.csv", encoding = "utf-8-sig") 

possibility_Fall_two_two = df2_second_person_match[df2_second_person_match["group"] == "possibility"]
possibility_Fall_two_two_value_count = possibility_Fall_two_two["rule"].value_counts() 
# possibility_Fall_two_two_value_count.to_csv("possibility_Fall_two_two_value_count_260922.csv", encoding = "utf-8-sig") 


In [38]:
# Fall 3 antecedent match
want_statement_Fall_three = df2_case3_antecedent_match[df2_case3_antecedent_match["group"] == "want statement"]
want_statement_Fall_three_value_count = want_statement_Fall_three["rule"].value_counts() 
# want_statement_Fall_three_value_count.to_csv("want_statement_Fall_three_value_count_260922.csv", encoding = "utf-8-sig") 

experience_Fall_three = df2_case3_antecedent_match[df2_case3_antecedent_match["group"] == "experience"]
experience_Fall_three_value_count = experience_Fall_three["rule"].value_counts() 
# experience_Fall_three_value_count.to_csv("experience_Fall_three_value_count_260922.csv", encoding = "utf-8-sig") 

clau_Fall_three = df2_case3_antecedent_match[df2_case3_antecedent_match["group"] == "clau"]
clau_Fall_three_value_count = clau_Fall_three["rule"].value_counts() 
# clau_Fall_three_value_count.to_csv("clau_Fall_three_value_count_260922.csv", encoding = "utf-8-sig") 


In [56]:
# 그냥 df2_exclude에서 제거해보기

df2_NA_none = df2_exclude[(df2_exclude.subject_by_rule == "N/A") | (df2_exclude.subject_by_rule == "none") ]
print(df2_NA_none.shape) #(3718, 21)

df2_NA_none_idx = df2_NA_none.index.tolist()
print(len(df2_NA_none_idx)) # 3718

df2_not_error_idx_all = df2_NA_none_idx + referent_match_all_idx
print(len(df2_not_error_idx_all))

df2_possible_error = df2_exclude.drop(df2_not_error_idx_all, axis = 0)
print(df2_possible_error.shape) # (375, 21) yeah!!!!

# df2_possible_error.to_csv("df2_possible_error_180922.csv", encoding = "utf-8-sig") 


(3718, 21)
3718
15508
(375, 21)


In [70]:
# df2_possible_error와 error_category 지닌 데이터의 idx 비교하기

df2_possible_error_idx = df2_possible_error.index.tolist()
print(len(df2_possible_error_idx)) # 375 richtig

df2_total_error_with_category = pd.read_csv("df2_total_error_with_category_110922.csv", index_col=0)
print(df2_total_error_with_category.shape) # (376, 8)

df2_total_error_with_category_idx = df2_total_error_with_category.index.tolist()
print(len(df2_total_error_with_category_idx)) # 376

df2_error_category = df2_total_error_with_category['category']
# print(df2_error_category)

intersection = list(set(df2_possible_error_idx) & set(df2_total_error_with_category_idx))
print(len(intersection)) # 375 

# difference 하나 찾기!

difference = list(set(df2_total_error_with_category_idx) - set(df2_possible_error_idx))
difference # [5699]

# df2_possible_error에 category 추가하기 index를 중심으로

df2_exclude_add_category = pd.concat([df2_possible_error, df2_error_category], axis=1)

print(df2_exclude_add_category.shape) # (376, 22)

# "df2_exclude_add_category_180922.csv" # 375개인 것 (5699 index 가진 case 삭제) manuell 만듦


375
(376, 8)
376
375
(376, 22)


In [71]:
# "df2_exclude_add_category_180922.csv" 불러와서 error category frequent 만들기

error_total = pd.read_csv('df2_exclude_add_category_180922.csv')

error_total_freq = error_total['category'].value_counts() 
print(error_total['group'].size) # 375 richtig
 
# error_total_freq.to_csv("error_total_freq_180922.csv", encoding = "utf-8-sig") 

375


In [45]:
# 2. df2_exclude(15883)에서 error_category 두 개로 나눔, 실제 error인것과 precision과 coverage에 다시 추가되어야 하는 것

#exclude_add

file_name = 'error_category_636_082022.csv'#여기를 원래 열의 수로 바꿔야 할 듯 그리고 category열을 추가 (성공! 130922)
error_category = pd.read_csv(file_name, index_col=0) # index_col=0을 하면서 "Unnamed: 0" 없어짐
# print(error_category.shape) # (636, 8)

error_category_column = error_category['category']
# print(error_category_column)

error_category_index = error_category.index.tolist()
# print(len(error_category_index)) # 636

df2_exclude_add_column = df2_exclude[df2_exclude.index.isin(error_category_index) == True]
# print(df2_exclude_add_column.shape) # (636, 21)

df2_exclude_add_category = pd.concat([df2_exclude_add_column,error_category_column], axis=1)
# print(df2_exclude_add_category.shape) # (636, 22)

# dataframe에서는 object가 string임
# error_category = error_category.applymap(str) # excel의 모든 열의 값을 string으로 바꿈
# error_category.dtypes

# 1. df2_error_add의 index 저장
# error category가 "check for antecedent (right case)", "not error (ambiguity between sing and plur)", "not error (ambiguity between sugg and impe)", "check for antecedent (right case)"

not_error_df = df2_exclude_add_category[df2_exclude_add_category.category.isin(["check for antecedent (right case)", "not error (ambiguity between sing and plur)", "not error (ambiguity between sugg and impe)", "check for antecedent (right case)", "not error (wrong zero referent)"]) == True]
print(not_error_df.shape) # (322, 22)

error_df = df2_exclude_add_category[df2_exclude_add_category.category.isin(["check for antecedent (right case)", "not error (ambiguity between sing and plur)", "not error (ambiguity between sugg and impe)", "check for antecedent (right case)", "not error (wrong zero referent)"]) == False]
print(error_df.shape) # (314, 22)

not_error_idx = not_error_df.index.tolist()
print(len(not_error_idx)) # 322
error_idx = error_df.index.tolist()
print(len(error_idx)) #314

# error_df.to_csv("error1_df.csv", encoding = "utf-8-sig") 

(322, 22)
(314, 22)
322
314


In [48]:
# 2. df2_exclude(15883)에서 우선 error인 것만 제외하고 

df2_exclude_error = df2_exclude[df2_exclude.index.isin(error_idx) == False]
excluded_error = df2_exclude[df2_exclude.index.isin(error_idx) == True]
print(df2_exclude_error.shape) # (15569, 21) "15883-15569=314" finally richtig 
print(excluded_error.shape) # (314, 21) 

excluded_error_idx = excluded_error.index.tolist()
print(len(excluded_error_idx)) # 314
# df2_exclude_error.to_csv("df2_exclude_error_030922.csv", encoding = "utf-8-sig") # 여기의 index를 다시 찾아서 error_df에서 해당 error들만으로 다시 df 만들어야 함 => excluded_error_df

excluded_error_df = error_df[error_df.index.isin(excluded_error_idx) == True]
print(excluded_error_df.shape) # (314, 8)
# excluded_error_df.to_csv("excluded_error_df_050922.csv", encoding = "utf-8-sig") # "excluded_error_df.csv" 이걸로 error_analysis 하면 됨

df2_exclude_error.shape

(15569, 21)
(314, 21)
314
(314, 22)


(15569, 21)

In [50]:
# 3. not_error인 경우 df2_exclude_error에 새로운 열을 추가해서 original과 from_not_error 구분해주고 final 파일 만들어서 이것으로 precision 등 계산하면 됨

# df2_exclude_error (15569, 21)에 not_error_df (322개) 추가 

df2_exclude_not_error_df = df2_exclude[df2_exclude.index.isin(not_error_idx) == True]
print(df2_exclude_not_error_df.shape) #(322, 21)

df2_total = [df2_exclude_error, not_error_df]
df2_not_error_added_df = pd.concat(df2_total)

print(df2_not_error_added_df.shape) # (15891, 22) 15569 + 322 = 15891 richtig

from_error_TF = []

for idx in df2_not_error_added_df.index:
    if idx in not_error_idx:
        from_error_TF.append('from_error')
    else: 
        from_error_TF.append('original') 
        
df2_exclude_add_from_error = df2_not_error_added_df.assign(from_error_TF = from_error_TF) 

print(df2_exclude_add_from_error.shape) # (15891, 23)

# df2_exclude_add_from_error

from_error_TF_count = df2_exclude_add_from_error.groupby(['from_error_TF']).count() 
# print(from_error_TF_count) # from error: 322 richtig


# df2_exclude_add_from_error.to_csv("df2_final_group1_df.csv", encoding = "utf-8-sig") # "df2_final_group1_df" 이거 규칙별 final check 하고 three columns 만들어서 precision 계산

(322, 21)
(15891, 22)
(15891, 23)


In [52]:
# 1. "df2_exclude_add_from_error" (15891, 23) 를 가지고 총 세 개의 케이스로 맞는 케이스를 분리해서 precision, recall 등을 계산하고
# case 1: subject_by_rule과 zero_referent가 string match
# cas3 2: singular <-> plural, 인칭만 맞는 케이스 (이건 실제로 context가 필요한 케이스이므로 맞는 케이스로 넣음)
# case 3: subject_by_rule과 antecedent가 같은 경우 (antecedent에 다양한 케이스가 존재할 수 있음)

# 나중에 규칙이 적용된 전체 데이터셋 크기는 error까지 다 합친 것으로 해야함!!!!
# 1. simple string match (case1)

df2_final_group1_case1 = df2_exclude_add_from_error[df2_exclude_add_from_error.subject_by_rule == df2_exclude_add_from_error.zero_referents] 
print(df2_final_group1_case1.shape) # (11422, 23)

# df2_final_group1_case1.to_csv("df2_final_group1_case1.csv", encoding = "utf-8-sig") 


(11422, 23)


In [54]:
# 2.  singular <-> plural, person match (case2) * case1을 제외하고 찾기
### case2A_B_C_02052022_xlsx에서 sheet0("singplural")에 있는 idx

df2_final_group1_not_case1 = df2_exclude_add_from_error[df2_exclude_add_from_error.subject_by_rule != df2_exclude_add_from_error.zero_referents] 
# print(df2_final_group1_not_case1.shape) # (4469, 23) 15891 - 11422 = 4469 richtig

df2_final_group1_not_case1_idx = df2_final_group1_not_case1.index.tolist()
# print(len(df2_final_group1_not_case1_idx)) # 4469

df2_final_group1_case2_first_person = df2_final_group1_not_case1[df2_final_group1_not_case1['subject_by_rule'].str.contains('1')] 
df2_final_group1_case2_second_person = df2_final_group1_not_case1[df2_final_group1_not_case1['subject_by_rule'].str.contains('2')] 

# df2_final_group1_case2_first_person.to_csv("df2_final_group1_case2_first_person.csv", encoding = "utf-8-sig") 
# df2_final_group1_case2_second_person.to_csv("df2_final_group1_case2_second_person.csv", encoding = "utf-8-sig") 

# print(df2_final_group1_case2_first_person.shape) #(659, 23) richtig
# print(df2_final_group1_case2_second_person.shape) #(93, 23) richtig

df2_final_group1_case2_first_person_match = df2_final_group1_case2_first_person[(df2_final_group1_case2_first_person['zero_referents'] == '1ph-') | (df2_final_group1_case2_first_person['zero_referents'] == '1ph+')] 
df2_final_group1_case2_second_person_match = df2_final_group1_case2_second_person[(df2_final_group1_case2_second_person['zero_referents'] == '2ph+') | (df2_final_group1_case2_second_person['zero_referents'] == '2sh-')]

# print(df2_final_group1_case2_first_person_match.shape) # (381, 23) richtig
# print(df2_final_group1_case2_second_person_match.shape) # (67, 23) richtig

df2_final_group1_case2_first_person_match_idx = df2_final_group1_case2_first_person_match.index.tolist()
df2_final_group1_case2_second_person_match_idx = df2_final_group1_case2_second_person_match.index.tolist()

# print(len(df2_final_group1_case2_first_person_match_idx)) # 381
# print(len(df2_final_group1_case2_second_person_match_idx)) # 67

# df2_final_group1_case2_first_person_match.to_csv("df2_final_group1_case2_first_person_match.csv", encoding = "utf-8-sig") 
# df2_final_group1_case2_second_person_match.to_csv("df2_final_group1_case2_second_person_match.csv", encoding = "utf-8-sig") 



In [56]:
# 3. subject_by_rule과 antecedent 같은 경우 (case3) 
# df2_final_group1_not_case1에서 df2_final_group1_case2_first_person_match와 df2_final_group1_case2_second_person_match를 제외한 것에서 시작

df2_final_group1_case2_first_second_person_match_idx = df2_final_group1_case2_first_person_match_idx + df2_final_group1_case2_second_person_match_idx
# print(len(df2_final_group1_case2_first_second_person_match_idx)) # 448 (381+67 = 448 richtig)

df2_final_group1_case3_check = df2_final_group1_not_case1[df2_final_group1_not_case1.index.isin(df2_final_group1_case2_first_second_person_match_idx) == False]

# print(df2_final_group1_case3_check.shape) # (4021, 23) 4469-448 = 4021 richtig
# df2_final_group1_case3_check.to_csv("df2_final_group1_case3_check.csv", encoding = "utf-8-sig") 


# df2_final_group1_case3_check에서 3개의 subcases # total 233:
# 3.1 subject_by_rule "1sh+" & antecedent "{저}", "{저는}", "{저도}", "{저희가}", "{저희들도}", "{전}", "{제가}" (check the dtypes if string match not works) # 232
# 3.2 subject_by_rule "1ph-" & antecedent "{우리}" # 1
# 3.3 subject_by_rule "2sh+" # no match

# right_antecedent인 case의 idx를 하나의 csv파일로 만듦 (이걸 다시 만들어야 함 110922)
 
antecedent_correct = pd.read_excel('right_antecedent_idx_050922.xlsx') 
antecedent_correct_idx = antecedent_correct['idx']
antecedent_correct_idx_val = antecedent_correct_idx.values 
antecedent_correct_idx_val_list = antecedent_correct_idx_val.tolist() # 233 richtig

# print(len(antecedent_correct_idx_val_list))

df2_final_group1_case3 = df2_final_group1_case3_check[df2_final_group1_case3_check.index.isin(antecedent_correct_idx_val_list) == True]
df2_final_group1_error = df2_final_group1_case3_check[df2_final_group1_case3_check.index.isin(antecedent_correct_idx_val_list) == False]

# print(df2_final_group1_case3.shape) # (233, 23)
# print(df2_final_group1_error.shape) # (3788, 23) 4021-233 = 3788 richtig

# df2_final_group1_case3.to_csv("df2_final_group1_case3.csv", encoding = "utf-8-sig") 
# df2_final_group1_error.to_csv("df2_final_group1_error.csv", encoding = "utf-8-sig") # 이거랑, 위의 error 합쳐야함, 

In [58]:
# df2_final_group1_error에서 N/A랑 none 제외시켜야 함

# 1. df2_final_group1_error(3788 cases)에서 subject_by_rule이 N/A인 case 분리 => group2 규칙 적용 대상 (총 2049 cases) 
df2_final_group1_to_group2 = df2_final_group1_error[df2_final_group1_error.subject_by_rule == "N/A"]
# print(df2_final_group1_to_group2.shape) # (2049, 23) richtig (전체 df2_final_group1_df에서도 같은 숫자)

# 2. df2_final_group1_error 에서 subject_by_rule이 none인 case 분리 => 규칙 적용이지만 실제 대화 참여자랑 관계없는 case, 이것에 대한 분석도 필요함
df2_final_group1_non_discoure_participate_rule = df2_final_group1_error[df2_final_group1_error.subject_by_rule == "none"]
# print(df2_final_group1_non_discoure_participate_rule.shape) # (1668, 23) richtig (전체 df2_final_group1_df에서도 같은 숫자)

error_check_idx1 = df2_final_group1_to_group2.index.tolist()
error_check_idx2 = df2_final_group1_non_discoure_participate_rule.index.tolist()

# print(len(error_check_idx1)) # 2049
# print(len(error_check_idx2)) # 1668

error_check_idx_total = error_check_idx1 + error_check_idx2
# print(len(error_check_idx_total)) # 3717 (2049+1668 = 3717 richtig)

# 3. 실제 error일 확률이 높은 case 찾아서 error catetory 추가하고, exluded_error랑 합쳐서, 이 데이터에 대해서 error analysis하면 됨
df2_final_group1_error_check = df2_final_group1_error[df2_final_group1_error.index.isin(error_check_idx_total) == False]
# print(df2_final_group1_error_check.shape) # (71, 23) richtig 3788 -3717 = 71 richtig

# df2_final_group1_error_check.to_csv("df2_final_group1_error_check_110922.csv", encoding = "utf-8-sig") # 110922 다시 확인


In [88]:
# df2_final_group1_error_check에서 다시 df2_exclude_add_from_error에 추가돼야 하는 case 찾기

not_error_df2 = df2_final_group1_error_check[df2_final_group1_error_check.category.isin(["check for antecedent (right case)", "not error (ambiguity between sing and plur)", "not error (ambiguity between sugg and impe)", "check for antecedent (right case)", "not error (wrong zero referent)"]) == True]
# print(not_error_df2.shape) # (9, 23)

error_df2 = df2_final_group1_error_check[df2_final_group1_error_check.category.isin(["check for antecedent (right case)", "not error (ambiguity between sing and plur)", "not error (ambiguity between sugg and impe)", "check for antecedent (right case)", "not error (wrong zero referent)"]) == False]
# print(error_df2.shape) # (62, 23) 71 -9 = 62 richtig

# # 1. not_error_df2를 제외한 error_df2 (62, 23) 완성 (이전의 error_df (314, 8)과 합쳐서 error_analysis하기 )

# error_df2.to_csv("error_df2_110922.csv", encoding = "utf-8-sig") 

df2_total_error_list = [error_df2, error_df]
df2_total_error = pd.concat(df2_total_error_list)
# print(df2_total_error.shape) # (376, 23) 62 + 314 = 376 richtig

# df2_total_error.to_csv("df2_total_error_110922.csv", encoding = "utf-8-sig") 
# 이걸 다시 카테고리가 추가된 csv 파일로 manuell하게 만들기 "df2_total_error_with_category_110922.csv"
# 여기에 또 not error 포함되어 있음 (5 cases) "df2_final_group1_df.csv"에는 이미 포함되어 있는지 확인해 보기 # index 13은 없는 것 같음

df2_total_error_with_category = pd.read_csv("df2_total_error_with_category_110922.csv", index_col=0)
# print(df2_total_error_with_category.shape) # (376, 8)
# 여기에서 category column만 가져와서 df2_total_error에 추가해서 df2_total_error_add_category 새로 만들기

df2_total_error_category = df2_total_error_with_category['category']
# print(df2_total_error_category.shape) (376,)

# df2_total_error.to_csv("df2_total_error_130922.csv", encoding = "utf-8-sig") 
# df2_total_error_category.to_csv("df2_total_error_category_130922.csv", encoding = "utf-8-sig") 

# 1. df2_total_error에서 category 열을 제외 -> df2_total_error_drop_category
# 2. df2_total_error_drop_category와 df2_total_error_category를 axis=1로 concat하기

df2_total_error_drop_category = df2_total_error.drop(['category'], axis=1)
# print(df2_total_error_drop_category.shape) # (376, 22)

df2_total_error_add_category = pd.concat([df2_total_error_drop_category, df2_total_error_category], axis=1)
# print(df2_total_error_add_category.shape) (376, 23)

# df2_total_error_add_category.to_csv("df2_total_error_add_category_130922.csv", encoding = "utf-8-sig") 

not_error_df3 = df2_total_error_add_category[df2_total_error_add_category.category.isin(["check for antecedent (right case)", "not error (ambiguity between sing and plur)", "not error (ambiguity between sugg and impe)", "check for antecedent (right case)", "not error (wrong zero referent)"]) == True]
print(not_error_df3.shape) # (5, 23)
# print(not_error_df3.index) # [9, 13, 16, 17, 47]

error_df3 = df2_total_error_add_category[df2_total_error_add_category.category.isin(["check for antecedent (right case)", "not error (ambiguity between sing and plur)", "not error (ambiguity between sugg and impe)", "check for antecedent (right case)", "not error (wrong zero referent)"]) == False]
print(error_df3.shape) # (371, 23) 376-5 = 371 richtig

# error_df3.to_csv("df2_total_error_final_110922.csv", encoding = "utf-8-sig") 


(5, 23)
(371, 23)


In [90]:
# error_total_group1_df를 category count해보기 
error_category_freq = error_df3.groupby(['category']).count() 

# error_category_freq

# "Fehlerkategorie_110922.xlsx" in "Schreiben_neu" folder

In [119]:
# 2. df2_exclude_add_from_error (15891, 23)에는 이미 not_error_df2의 index를 가진 문장들이 포함되어 있음

# 확실하게 하기 위해서 not_error_df2와 not_error_df3을 추가해보기 (intersection도 활용해도 좋음)
# "df2_final_group1_df.csv"를 precision, recall 구하는 것에 사용하면 됨

df2_exclude_add_from_error_idx = df2_exclude_add_from_error.index.tolist()
print(len(df2_exclude_add_from_error_idx)) # 15891

not_error_df2_idx = not_error_df2.index.tolist()
print(len(not_error_df2_idx)) # 9

not_error_df3_idx = not_error_df3.index.tolist()
print(len(not_error_df3_idx)) # 5

not_error_df2_df3_idx = not_error_df2_idx + not_error_df3_idx
print(len(not_error_df2_df3_idx)) # 14

D = list(set(not_error_df2_df3_idx) - set(df2_exclude_add_from_error_idx))
print(D) # []

# print(df2_exclude_add_from_error.shape) # (15891, 23)

df2_exclude_add_from_error = df2_total_group1_for_analysis

df2_total_group1_for_analysis.to_csv("df2_total_group1_for_analysis_test1.csv", encoding = "utf-8-sig") 

print(df2_total_group1_for_analysis.shape) # (15891, 23)


15891
9
5
14
[]
(15891, 23)


In [102]:
# df2_total_group1_for_analysis (15891, 23) precision과 recall 계산하기

# df2_total_group1_for_analysis 에서 우선 N/A랑 none 따로 dataframe으로 만들어 놓기

# 1. df2_total_group1_for_analysis에서(15892 cases)에서 subject_by_rule이 N/A인 case 분리 => group2 규칙 적용 대상 (총 2049 cases) 
df2_group1_to_group2 = df2_total_group1_for_analysis[df2_total_group1_for_analysis.subject_by_rule == "N/A"]
df2_NA_idx = df2_group1_to_group2.index.tolist()
print(len(df2_NA_idx)) # 2049

# print(df2_group1_to_group2.shape) # (2049, 23) richtig 

# 2. df2_total_group1_for_analysis에서 subject_by_rule이 none인 case 분리 => 규칙 적용이지만 실제 대화 참여자랑 관계없는 case,이것에 대한 분석도 필요함
df2_non_discourse_participant = df2_total_group1_for_analysis[df2_total_group1_for_analysis.subject_by_rule == "none"]
df2_non_discourse_participant_idx = df2_non_discourse_participant.index.tolist()
print(len(df2_non_discourse_participant_idx)) # 1668

# print(df2_non_discourse_participate.shape) # (1668, 23) richtig 

# df2_group1_to_group2.to_csv("df2_group1_to_group2_110922.csv", encoding = "utf-8-sig") 
# df2_non_discourse_participant.to_csv("df2_non_discourse_participate_110922.csv", encoding = "utf-8-sig") 

2049
1668


In [120]:
# 나중에 규칙이 적용된 전체 데이터셋 크기는 error까지 다 합친 것으로 해야함!!!!
# 1. simple string match (case1)

df2_case1 = df2_total_group1_for_analysis[df2_total_group1_for_analysis.subject_by_rule == df2_total_group1_for_analysis.zero_referents] 
df2_case1_idx = df2_case1.index.tolist()
print(len(df2_case1_idx)) # 11422

print(df2_case1.shape) # (11422, 23)

# df2_case1.to_csv("df2_case1_110922.csv", encoding = "utf-8-sig") 

11422
(11422, 23)


In [121]:
# 2.  singular <-> plural, person match (case2) * case1을 제외하고 찾기

df2_not_case1 = df2_total_group1_for_analysis[df2_total_group1_for_analysis.subject_by_rule != df2_total_group1_for_analysis.zero_referents] 
print(df2_not_case1.shape) # (4470, 23) 15892 - 11422 = 4470 richtig

df2_not_case1_idx = df2_not_case1.index.tolist()
# print(len(df2_not_case1_idx)) # 4470

df2_first_person = df2_not_case1[df2_not_case1['subject_by_rule'].str.contains('1')] 
df2_second_person = df2_not_case1[df2_not_case1['subject_by_rule'].str.contains('2')] 

print(df2_first_person.shape) #(659, 23) 
print(df2_second_person.shape) #(94, 23) 

df2_first_person_match = df2_first_person[(df2_first_person['zero_referents'] == '1ph-') | (df2_first_person['zero_referents'] == '1ph+')] 
df2_second_person_match = df2_second_person[(df2_second_person['zero_referents'] == '2ph+') | (df2_second_person['zero_referents'] == '2sh-')]

print(df2_first_person_match.shape) # (381, 23) 
print(df2_second_person_match.shape) # (67, 23) 

df2_first_person_match_idx = df2_first_person_match.index.tolist()
df2_second_person_match_idx = df2_second_person_match.index.tolist()

# print(len(df2_first_person_match_idx)) # 381
# print(len(df2_second_person_match_idx)) # 67

# df2_first_person_match.to_csv("df2_first_person_match.csv", encoding = "utf-8-sig") 
# df2_second_person_match.to_csv("df2_second_person_match.csv", encoding = "utf-8-sig") 


(4469, 23)
(659, 23)
(93, 23)
(381, 23)
(67, 23)


In [130]:
# 3. subject_by_rule과 antecedent 같은 경우 (case3) 
# df2_not_case1 에서 df2_first_person_match 와 df2_second_person_match 를 제외한 것에서 시작

df2_first_second_person_match_idx = df2_first_person_match_idx + df2_second_person_match_idx
# print(len(df2_first_second_person_match_idx)) # 448 (381+67 = 448 richtig)

df2_case3_check = df2_not_case1[df2_not_case1.index.isin(df2_first_second_person_match_idx) == False]

# print(df2_case3_check.shape) # (4022, 23) 4470-448 = 4022 richtig
# df2_case3_check.to_csv("df2_case3_check.csv", encoding = "utf-8-sig") 


# df2_case3_check 에서 3개의 subcases # total 233:
# 3.1 subject_by_rule "1sh+" & antecedent "{저}", "{저는}", "{저도}", "{저희가}", "{저희들도}", "{전}", "{제가}" (check the dtypes if string match not works) # 232
# 3.2 subject_by_rule "1ph-" & antecedent "{우리}" # 1
# 3.3 subject_by_rule "2sh+" # no match 

# right_antecedent인 case의 idx를 하나의 csv파일로 만듦 (위의 코드에도 존재함)

df2_case3 = df2_case3_check[df2_case3_check.index.isin(antecedent_correct_idx_val_list) == True]
df2_case3_idx = df2_case3.index.tolist()
# print(len(df2_case3_idx)) # 233

# df2_error_final_check = df2_case3_check[df2_case3_check.index.isin(antecedent_correct_idx_val_list) == False]

print(df2_case3.shape) # (233, 23)
# print(df2_error_final_check.shape) # (3789, 23) 4022-233 = 3789 richtig

# df2_case3.to_csv("df2_case3.csv", encoding = "utf-8-sig") 


(233, 23)


In [112]:
# df2_total_group1_for_analysis 에다가 case_class 열 추가하기!
# 1.	to_group2
# 2.	not_discourse_participant
# 3.	case1
# 4.	case2
# 5.	case3
# 6.	N/A (전체는 15892인데 72개가 무엇일까?) # 또 다른 방법은 1-5까지의 모든 인덱스를 합치고 다시 차집합 확인하기 

df2_total_group1_for_analysis_idx = df2_total_group1_for_analysis.index.tolist()
print(len(df2_total_group1_for_analysis_idx)) # 15892

index_total_idx_list = df2_NA_idx+ df2_non_discourse_participant_idx+ df2_case1_idx + df2_first_second_person_match_idx + df2_case3_idx
print(len(index_total_idx_list)) # 15820

Difference = list(set(df2_total_group1_for_analysis_idx) - set(index_total_idx_list))
print(len(Difference)) # 62

df2_difference = df2_total_group1_for_analysis[df2_total_group1_for_analysis.index.isin(Difference) == True]
print(df2_difference.shape) # (71, 23)

# 이 71개가 지금 뭔지 모르겠고 그리고 왜 difference 62와 숫자 차이가 있는지 모르겠지만, 우선 case 1-3까지 rule 카테고리 등 분석, 우선 표로 만들기

# df2_difference.to_csv("df2_difference.csv", encoding = "utf-8-sig") 


15891
15820
62
(71, 23)


In [134]:
rule_freq_case1 = df2_case1['group'].value_counts() 
print(df2_case1['group'].size) # 11422 richtig
 
# rule_freq_case1.to_csv("rule_freq_case1.csv", encoding = "utf-8-sig") 

11422


In [133]:
rule_freq_first_person_match = df2_first_person_match['group'].value_counts() 
print(df2_first_person_match['group'].size) # 381 richtig

rule_freq_second_person_match = df2_second_person_match['group'].value_counts() 
print(df2_second_person_match['group'].size) # 67 richtig

rule_freq_case3 = df2_case3['group'].value_counts() 
print(df2_case3['group'].size) # 233 richtig


# rule_freq_first_person_match.to_csv("rule_freq_first_person_match.csv", encoding = "utf-8-sig") 
# rule_freq_second_person_match.to_csv("rule_freq_second_person_match.csv", encoding = "utf-8-sig") 
# rule_freq_case3.to_csv("rule_freq_case3.csv", encoding = "utf-8-sig") 


381
67
233


In [122]:
rule_freq_case3 = df2_case3['rule'].value_counts() 
# print(rule_freq_case3)

# rule_freq_case3.to_csv("rule_freq_case3.csv", encoding = "utf-8-sig") 

want verbal suffix                            34
cognition predicate                           12
clau_neunde                                   11
want periphrastic construction                11
possibility periphrastic construction/plus    10
suggestory verbal suffix                       9
permission periphrastic construction           8
obligation periphrastic construction           8
progress aspect                                5
sensation predicate                            5
feeling predicate                              4
guess periphrastic construction                4
politeness auxiliary verb                      3
politeness predicate                           3
impossibility periphrastic construction        3
aspect auxiliary verb                          1
clau_cognition                                 1
passive aspect                                 1
sugg                                           1
Name: rule, dtype: int64


In [123]:
rule_freq_difference = df2_difference['rule'].value_counts() 
# print(rule_freq_difference)

# rule_freq_difference.to_csv("rule_freq_difference.csv", encoding = "utf-8-sig") 

progress aspect                               30
hono total                                    14
want periphrastic construction                 7
want verbal suffix                             2
politeness auxiliary verb                      2
suggestory verbal suffix                       2
possibility periphrastic construction/plus     1
obligation periphrastic construction           1
clau_neunde                                    1
sugg                                           1
performative predicate                         1
Name: rule, dtype: int64


In [53]:
rule_freq = df2_exclude_error['applied_rule'].value_counts() 
#rule_order = df2_exclude_error['rule_order'].value_counts() 
rule_unique = df2_exclude_error['applied_rule'].unique()
#print(rule_unique)
print(rule_freq)
# rule_freq.to_excel('rule1_freq_with_order.xlsx', encoding = "utf-8-sig")
# 이미 기존 "df2_exclude_error"에 rule_order는 추가되어 있는 상태이므로 이 데이터셋에 rule_freq를 추가하는 것이 하나의 방법일 듯

rule_freq_dict = rule_freq.to_dict()

#df2_exclude_error_rule_freq = df2_exclude_error.assign(rule_count = rule_freq) 
# df2_exclude_error_rule_freq.shape # (15247, 23)
#df2_exclude_error_rule_freq.to_excel('df2_exclude_error_rule_freq.xlsx', encoding = "utf-8-sig")

rule_freq_dict_keys = list(rule_freq_dict.keys()) #8개 / J_root는 총 23개 - list로 무사히 바꿈 (dict_kyes의 속성을 지니지 않음)
# print(len(rule_freq_dict_keys)) #173

rule_count = []

for s in df2_exclude_error['applied_rule']:
    if s in rule_freq_dict_keys:
        rule_count.append(rule_freq_dict[s])
    else: 
        rule_count.append('NAN') 
        
df2_exclude_error_rule_freq = df2_exclude_error.assign(rule_count = rule_count)
print(df2_exclude_error_rule_freq.shape) # (15247, 22)

#df2_exclude_error_rule_freq.to_excel('df2_exclude_error_rule_freq.xlsx', encoding = "utf-8-sig")
#df2_exclude_error.shape


df2_three_columns = df2_exclude_error_rule_freq[['rule_order','applied_rule','rule_count']]
#df2_three_columns.to_excel('df2_three_columns.xlsx', encoding = "utf-8-sig")

impe was applied,                          2434
                                           2049
2.19.2 exclude_predicate1 was applied,     1495
2.2 want was applied,                      1000
3.1.2 suggestory was applied,               581
                                           ... 
8.3 want was applied,                         1
2.20 want was applied,                        1
7.3 judgement was applied,                    1
clau(euni_variant1) was applied,              1
0.3 clau(cognition3) was applied,             1
Name: applied_rule, Length: 173, dtype: int64
(15247, 22)
